# Modules required for pipeline

In [4]:
#!pip install --upgrade pdb-tools
#!pip3 install atomium

#!pip install sympy
#!pip install UpSetPlot
#!pip install -U kaleido

In [203]:
#!pip install pypdb
import requests
import os
import string
from collections import defaultdict
from collections import Counter
import re
import shutil
import hail as hl
# Import from installed package
#from pypdb.clients.pdb.pdb_client import *
import dask.dataframe as dd
import json
import Bio
import pandas as pd
from Bio.PDB import PDBParser, PDBIO, Select, MMCIFParser
from Bio.SeqIO import PirIO
from Bio import pairwise2
from io import StringIO
from modeller import *
from modeller.automodel import *
import matplotlib.pyplot as plt
import matplotlib.style as style
import logging
import subprocess
import shlex
from subprocess import PIPE, run
import numpy as np
import math
from bs4 import BeautifulSoup  #required later to download SIFT files.
import atomium
from sympy import Union, Interval
from itertools import compress
from sklearn import metrics
from sklearn.cluster import AffinityPropagation
from sklearn.datasets import make_blobs
from sklearn.cluster import MeanShift
import matplotlib.gridspec as gridspe
import plotly.express as px
import plotly.io as pio
from bravado.client import SwaggerClient
from Bio import AlignIO
from pycanal import Canal
import hdbscan
#lot import generate_counts
#from upsetplot import UpSet
#from upsetplot import from_memberships
#from upsetplot import from_contents
#from upsetplot import from_indicators
from scipy import stats
from sklearn.cluster import OPTICS, cluster_optics_dbscan

#logging.getLogger("requests").setLevel(logging.WARNING)


# Basehelper function:

+ get_url
+ get_gene_name_uniprot / get_gene_name redundant function.
+ get_uniprot_id
+ get_fasta_seq

In [6]:
def get_url(url):
    try:
        response = requests.get(url)  
        if not response.ok:
            print(response.text)
    except:
        response.raise_for_status()
        #sys.exit()
        
    return response

In [7]:
def get_gene_name_uniprot(uniprot_id:str):
    
    fields = "gene_primary"
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=tsv&fields={fields}&query={uniprot_id}"
    resp = get_url(URL)
    resp = resp.text
    #print(resp)
    resp = resp.split("\n")
    
    #result_lst = [x.split("\t") for x in resp]
    #result_lst = result_lst[0:-1]
    #result_sort = sorted(result_lst, key= lambda x : x[2])
    #resp = result_sort[0]
    #print(resp)
    return resp[1]

In [8]:
#get_gene_name_uniprot("A0A024RBG1")

In [9]:
def get_uniprot_id(uniprot_id:str):
    fields = "accession"
    
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=tsv&fields={fields}&query={uniprot_id}"
    resp = get_url(URL)
    resp = resp.iter_lines()
    for lines in resp:
        lines = lines.decode()
        if lines == "Entry":
            continue
        return lines

In [10]:
def get_gene_fasta(uniprot_id:str):

    print("we are in get gene fasta")
    "this is already overworked. should work."
    #uniprot_canonical_isoform = get_uniprot_id(uniprot_id=uniprot_id)
    
    fields = "sequence"
    
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=fasta&fields={fields}&query={uniprot_id}"
    resp = get_url(URL)
    resp = resp.iter_lines(decode_unicode=True)
    
    seq = ""
    
    i = 0
    for lines in resp:
        if i > 0:
            seq += lines
            #print(lines)
        i += 1
    
    #print(seq)
    return seq


In [11]:
#pdb_id_target= "6lle"
#get_rcsb_fasta(pdb_id_target=pdb_id_target, chain="A")
#
#query = "P16615"
#main_prot_seq = get_gene_fasta(query)
#print(main_prot_seq)
#print(len(main_prot_seq))

In [12]:
def get_gene_fasta_from_pdb_id(uniprot_id:str, chain:str):
    
    #uniprot_canonical_isoform = get_uniprot_id(uniprot_id=uniprot_id)
    
    fields = "sequence"
    
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=fasta&fields={fields}&query={uniprot_id}"
    resp = get_url(URL)
    resp = resp.iter_lines(decode_unicode=True)
    
    res = ""
    i = 0
    seq = 0
    aln_code = []
        
    for lines in resp:
        # if we encounter a new seq (e.g multi chain):
        if lines[0] == ">":
            #we check if its the first seq. We make out custom header
            continue
        else:
            res += lines
            
    return res


In [13]:
def get_fasta_and_write(uniprot_id:str, path:str):
    
    
    URL = f"https://www.rcsb.org/fasta/entry/{uniprot_id}/display"
    resp = get_url(URL)
    resp = resp.iter_lines(decode_unicode=True)
    
    #print(resp)
    
    res = ""
    i = 0
    seq = 0
    chain = string.ascii_uppercase 
    
    aln_code = []
    
    for lines in resp:
        # if we encounter a new seq (e.g multi chain):
        if lines[0] == ">":
            #we check if its the first seq. We make out custom header
            origin_header = lines.split("|")
            if seq == 0:
                res += f">{uniprot_id}_{chain[i]}\n"
                aln_code.append(f"{uniprot_id}_{chain[i]}")
                i += 1
                seq += 1
                continue
            #if its the nth seq, we add a \n beforehand.
            else:
                
                res += f"\n>{uniprot_id}_{chain[i]}\n"
                aln_code.append(f"{uniprot_id}_{chain[i]}")
                i += 1
                continue
        res += lines
    
    print(origin_header)
    print(res)
    #write to file and leave
    #with open(f"{path}x.fasta", "w") as fasta_out:
    #    fasta_out.write(res)
        
    
    
    #return aln_code

In [14]:
def get_hugo_name(uniprot_id:str):
    """
    Retrieve the HUGO gene symbol from a Uniprot identifier.

    Parameters:
    - uniprot_id (str): The Uniprot identifier for the protein.

    Returns:
    - str or None: The HUGO gene symbol corresponding to the Uniprot identifier.
                  Returns None if no gene symbol is found. """
                  
    
    fields = "gene_names"
    
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=tsv&fields={fields}&query={uniprot_id}"
    
    resp = get_url(URL)
    
    resp = resp.text
    
    resp = resp.replace("Gene Names", "")
    
    resp = resp.replace("\n", "")
    
    genes = resp.split(" ")

    #return None if we find nothing.
    return genes[0] if len(genes) != 0 else None

In [15]:
def get_cbioportal_info(gene_name:str, study_id="msk_impact_2017"):

    """
    Retrieve the mutation info from the cbioportal associated with our Uniprot ID.

    Parameters:
    - gene_name (str): The gene name identifier for the protein (HUGO format) Call first get_hugo_name(uniprot_id:str) to retrieve it.

    
    Returns:
    - pandas DataFrame or None: The HUGO gene symbol corresponding to the Uniprot identifier is used to search the cbioPortal API.
                  Returns None if no gene symbol is found. 
    
    """
    
    cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/v2/api-docs',
                                    config={"validate_requests":False,"validate_responses":False,"validate_swagger_spec": False})

    for a in dir(cbioportal):
        cbioportal.__setattr__(a.replace(' ', '_').lower(), cbioportal.__getattr__(a))
    
    muts = cbioportal.mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
    molecularProfileId=f"{study_id}_mutations", # {study_id}_mutations gives default mutations profile for study 
    sampleListId=f"{study_id}_all", # {study_id}_all includes all samples
    projection="DETAILED").result()
    
    
    # Create an empty DataFrame
    mutation_df = pd.DataFrame()
    
    
    gene_symbol_lst = []
    mutation_type_lst = []
    protein_change_lst = []
    sample_id_lst = []
    tumor_alt_count_lst = []
    tumor_ref_count_lst = []
    mutationStatus_lst = []
    norm_alt_cnt_lst = []
    norm_ref_cnt_lst = []
    alleleSpecificCopyNumber_lst = []
    driver_filt_lst = []
    driver_filt_annot_lst = []
    driver_tier_filt_lst = []
    driver_tier_filt_annot_lst = []

    
    # Populate the DataFrame with mutation information
    for mutation in muts:
        #print(mutation)
        gene_symbol = mutation.gene.hugoGeneSymbol
        mutation_type = mutation.mutationType
        protein_change = mutation.proteinChange
        sample_id = mutation.sampleId
        tumor_alt_count = mutation.tumorAltCount
        tumor_ref_count = mutation.tumorRefCount
        mutation_status = mutation.mutationStatus
        norm_alt_cnt = mutation.normalAltCount
        norm_ref_cnt = mutation.normalRefCount
        alleleSpecificCopyNumber = mutation.alleleSpecificCopyNumber
        driver_filt = mutation.driverFilter
        driver_filt_annot = mutation.driverFilterAnnotation
        driver_tier_filt = mutation.driverTiersFilter
        driver_tier_filt_annot = mutation.driverTiersFilterAnnotation
        
# 'driverFilterAnnotation', 'driverTiersFilter', 'driverTiersFilterAnnotation',


        
        gene_symbol_lst.append(gene_symbol)
        mutation_type_lst.append(mutation_type)
        protein_change_lst.append(protein_change)
        sample_id_lst.append(sample_id)
        tumor_alt_count_lst.append(tumor_alt_count) 
        tumor_ref_count_lst.append(tumor_ref_count)
        mutationStatus_lst.append(mutation_status)
        norm_alt_cnt_lst.append(norm_alt_cnt)
        norm_ref_cnt_lst.append(norm_ref_cnt)
        alleleSpecificCopyNumber_lst.append(alleleSpecificCopyNumber)
        driver_filt_lst.append(driver_filt)
        driver_filt_annot_lst.append(driver_filt_annot)
        driver_tier_filt_lst.append(driver_tier_filt)
        driver_tier_filt_annot_lst.append(driver_filt_annot)
        
        
    mutation_df["gene_symbol"] = gene_symbol_lst
    mutation_df["mutation_type"] = mutation_type_lst
    mutation_df["prot_change"] = protein_change_lst
    mutation_df["sample_id"] = sample_id_lst
    mutation_df["tumor_alt_count"] = tumor_alt_count_lst
    mutation_df["tumor_ref_count"] = tumor_ref_count_lst
    mutation_df["norm_alt_count"] = tumor_alt_count_lst
    mutation_df["norm_ref_count"] = tumor_ref_count_lst
    mutation_df["mutation_status"] = mutationStatus_lst
    mutation_df["alleleSpecificCopyNumber"] = alleleSpecificCopyNumber_lst
    mutation_df["driver_filt"] = driver_filt_lst
    mutation_df["driver_filt_annot"] = driver_filt_annot_lst
    mutation_df["driver_tier_filt"] = driver_tier_filt_lst
    mutation_df["driver_tier_filt_annot"] = driver_filt_annot_lst

    
    
    mutation_sub_df = pd.DataFrame()
    
    try:
        for mutation in mutation_df["gene_symbol"]:
            if mutation == gene_name:
                mutation_sub_df = mutation_df[mutation_df["gene_symbol"] == gene_name]
                mutation_sub_df = mutation_sub_df[mutation_sub_df["mutation_type"] == "Missense_Mutation"]
    except:
        print(f"gene name: {gene_name} was not found")
    
    return mutation_sub_df

In [16]:
#integrate get_rcsb_fasta into pipeline and substitute with get_gene_fasta!
#ToBeDone

def get_rcsb_fasta(pdb_id_target:str, chain:str):
    
    '''This function will take the uniprot id and the requiered chain and fetch ONLY the sequence of the
    corresponding chain. This will be used for modeller repair as template in order to ONLY build
    the required part of the protein (i.e part of the protein the structure covers) and not the fasta for the full length protein
    which would lead to a full reconstruction of e.g EGFR 1-1xxx instead of maybe only repairing 300-460'''
    
    URL = f"https://www.rcsb.org/fasta/entry/{pdb_id_target}/display"
    
    resp = get_url(URL)
    resp = resp.iter_lines(decode_unicode=True)
    
    
    #here we store each chain and their fasta respectively. If multiple chains are homo oligomer their fasta
    #are stored together.
    
    fasta_per_chains = defaultdict()
        
    for lines in resp:
        
        if lines[0] == ">":
            #we store all headers given they contain the chain info.
            #print(lines)
            origin_header = lines.split("|") #split by |
            #print(origin_header)
            chain_info = origin_header[1] #2nd entry is chain info
            chain_info = chain_info[6:] # we are only interested in A, B , C , D not the keyword "chains" 
            #print(chain_info)
            
            #lets try to capture author annotated chains as well.
            try:
                
                author_chain = chain_info[-2] #this is the authors chain info.
            
            except:
                
                author_chain = "Placeholder"
                
            chain_info = re.sub('auth.[A-Z]', '', chain_info)  #if the authors have other chain labels we dont care
            chain_info = chain_info.replace("[]","")  #this is removal from from the authors chain labels.
            
            chain_info = chain_info.replace(",","") # remove , and merge them
            chain_info = chain_info.replace(" ","") #make str out of those as key for a dict. "ABCD"
            continue
        
        else:
            fasta_per_chains[chain_info] = lines
            fasta_per_chains[author_chain] = lines
            
    
    print(fasta_per_chains)
    
    #now lets look the prior names of the chains before renumbering.
    
    
    #with open(f"{path}/reports/chain_relabeling_protocol.csv", "r") as chain_names_csv:
    #    for lines in chain_names_csv:
    #        splitted_csv = lines.split()
    #        if splitted_csv[0] == pdb_id_target:
    #            old = splitted_csv[1].split()
    #            new = splitted_csv[2].split()
                
    
    
    for fasta_chains, seq in fasta_per_chains.items():
        avail_chain = [x for x in fasta_chains]
        #print(avail_chain)
        if chain in avail_chain:
            return seq
            
    
    #try author chain annotation as well if not working.
    
    
    
    #return None if we dont find anything.
    return None

In [17]:
#pdb_id_target= "6lle"
#get_rcsb_fasta(pdb_id_target=pdb_id_target, chain="A")
#
#query = ""
#main_prot_seq = get_gene_fasta(query)
#print(len(main_prot_seq))

# Helper functions need overview to check which is needed.

# Following functions:

+ Setup directories and prep input files
+ Prepare templates
+ _write_report_template

In [18]:
def setup_directories_prepare_input_files(gene_name:str, prot_fasta:str, domain_dict:dict):
    
    #check if the directory is not already there.
    #setup all required directories now.
    
    #hardcorded anchor location.
    path = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/{gene_name}"
    
    
    #dont download twice structures that have 2 chains.
    """ We want to avoid downloading since its a bottleneck and speed up the pipeline.
        For that reason we will make a dict where each key = rcsb id and the values are the chains
        we need from that pdb."""
        
        
    download_dict = defaultdict(str)
    
    rcsb_ids2 = []
    
    for keys, vals in domain_dict.items():
        for rcsbs in vals:
            rcsb_ids2.append(rcsbs[0:4])
            download_dict[rcsbs[0:4]] += rcsbs[-1]
            
    #get rid of duplicates and store only relevant ids
    rcsb_unique_ids = list(set(rcsb_ids2))
    
    try:
        #make it and switch to it
        os.mkdir(path)
        #os.mkdir(f"{path}/fastas")
        os.mkdir(f"{path}/reports")
        os.mkdir(f"{path}/shifts")
        os.mkdir(f"{path}/mutational_mapping")
        #os.mkdir(f"{path}/no_gaps")
        #os.mkdir(f"{path}/missing_repairable_gaps")
        
    except:
        print("Dir already exists")
        #if we already have it, we dont need to go through it again
        
    #this part will contain the info for curl to get the pdb files.
    
    with open(f"{path}/pdbfile.txt", "w") as pdb_tar:
        #the input file for batch download is in format:
        #entries=pdb_code_chain, sep= ","   e.g 4rch_A,5juu_B...
        for rcsb_id in rcsb_unique_ids:
            
            pdb_tar.write(rcsb_id)
            
            pdb_tar.write(",")
    
    #we also store the main fasta of the canonical isoform for later mapping.
    with open(f"{path}/reports/main_isoform_fasta.txt", "w") as main:
        main.write(prot_fasta)
        
    
    with open(f"{path}/reports/chain_relabeling_protocol.csv", "w") as fh_chain:
        fh_chain.write("pdb_file,previous_chains,new_chains")
        fh_chain.write("\n")
    
    return download_dict

In [19]:
def prepare_templates(gene_name_target:str,
                      templates:list,
                      seq_sim:list,
                      query_start:list,
                      query_end:list,
                      temp_start:list,
                      temp_end:list,
                      path:str, oligodict:list):
    
    """Make function out of making templates and reference structures."""
    #setup
    
    
    #this retrieves the main len of the protein we are interested in
    main_target_seq_len = get_seq_len(path=f"{path}", rcsb_id="None", template=True)
    #181 for NUD4B
    
    #gives back the length of each template.
    temp_lengths = [f[0] - f[1] for f in zip(temp_end, temp_start)]
    
    homology_list = sorted(list(zip(templates,seq_sim, temp_lengths)), key= lambda x: x[1], reverse=True)
    #print(homology_list)
    
    try:
        
        #gather target templates that will be used to compute tm scores and align scores
        ref_list = []
        #here we store human hits.
        priority_list = []
        #if we retrieve not as many hits, we take what we get
        if len(homology_list) < 100:
            
            max_end = len(homology_list)
            
        #else we take max 100 potential templates
        else:
            
            max_end = 100
        #this will be used later to set a ref struc template.
        
        
        for entry , seq, lengths in homology_list[0:max_end]:
            
            #this is the rcsb_id
            prot_check = entry[0:4]
            
            #this retrieves gene_name
            prot_name = get_gene_name(prot_check)
            #this corresponds to the species the gene belongs to
            if prot_name[-5:] == "HUMAN":
                
                #if its human, we take it as template and break.
                reference_structure = entry+".pdb"
                #temp score will help to find ideal target
                temp_score_val = temp_score(seq_sim=float(seq),
                           seq_len=lengths,
                           main_seq_len=main_target_seq_len)
                #we store both seq similarity and potential .pdb file
                priority_list.append((reference_structure, prot_name, temp_score_val, lengths, float(seq)))
                continue
                
            #else we take as many hits we can and select the highest seq similarity out of those.
            reference = entry+".pdb"
            
            #temp score will help to find ideal target
            temp_score_val = temp_score(seq_sim=float(seq),
                       seq_len=lengths,
                       main_seq_len=main_target_seq_len)
            
            ref_list.append((reference, prot_name, temp_score_val, lengths, float(seq))) 
            
    except:
        print(f"We did not find structures.\n selected template was:{prot_name}")
        print(f"reference list contained: {ref_list}")
    
    
    
    if len(priority_list) != 0:
        #if we found a human template we set it as reference structure.
        priority_list_sorted = sorted(priority_list, key=lambda x : x[2], reverse=False)
        print(f"we found as reference structure(s): {priority_list[0][1]} -> {priority_list_sorted[0][0]}")
        
        #little helper function to write out which template was choosen.
        _write_report_template(path=path, 
                               template_gene_name=priority_list_sorted[0][1],
                               template_rcsb=priority_list_sorted[0][0],
                               template_length=priority_list_sorted[0][3],
                               seq_id =priority_list_sorted[0][4],
                               query_length=main_target_seq_len)
        
        return priority_list_sorted
    
    else:
        #else we take the next best thing that has highest seq similarity hoping this is mouse or something.
        ref_list_sorted = sorted(ref_list, key=lambda x : x[2], reverse=False)
        print(f"we found no human reference structure(s) but instead: {ref_list[0][1]} -> {ref_list_sorted[0][0]}")
        
        #little helper function to write out which template was choosen.
        _write_report_template(path=path, 
                               template_gene_name=ref_list_sorted[0][1],
                               template_rcsb=ref_list_sorted[0][0],
                               template_length=ref_list_sorted[0][3],
                               seq_id =priority_list_sorted[0][4],
                               query_length=main_target_seq_len,
                               position_dir = position_dir,
                               oligostate_dir=oligostate_dir)
        
        return ref_list_sorted
    

In [20]:
def _write_report_template(path:str, template_gene_name:str, 
                           template_rcsb:str, template_length:str, 
                           query_length:str, seq_id:str,
                           position_dir:str,
                           oligostate_dir:str):
    
    
    #the last dir corresponds to the gene_name of query.
    #this gives the split dirs that make up path: e.g /home/micnag/b would give ["home", "micnag", "b"]
    path_split = path.split(os.sep)
    
    
    with open(f"{path}/reports/selected_template_{oligostate_dir}_{position_dir}.tsv", "w") as fh_report:
        fh_report.write("Query_gene_name\tTemplate_gene_name\tTemplate_rcsb\tTemplate_length\tQuery_length\tSequence_identity")
        fh_report.write("\n")
        fh_report.write(f"{path_split[-1]}\t{template_gene_name}\t{template_rcsb}\t{template_length}\t{query_length}\t{seq_id}")
        

In [21]:
def get_rcsb_info(path:str):
    
    '''This function will take the uniprot id and the requiered chain and fetch ONLY the sequence of the
    corresponding chain. This will be used for modeller repair as template in order to ONLY build
    the required part of the protein (i.e part of the protein the structure covers) and not the fasta for the full length protein
    which would lead to a full reconstruction of e.g EGFR 1-1xxx instead of maybe only repairing 300-460'''
    
    pdb_list = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    pdb_list = [f[0:4] for f in pdb_list if f[-4:] == ".pdb"]
    
    try:
            
        os.mkdir(f"{path}/PCA_structures_info")
            
    except Exception as error:
            
        print(error)
    
    
    infodict = defaultdict()
    
    for pdb_id_target in pdb_list:
        
        URL = f"https://www.rcsb.org/fasta/entry/{pdb_id_target}/display"
        
        resp = get_url(URL)
        resp = resp.iter_lines(decode_unicode=True)

        #here we store each chain and their fasta respectively. If multiple chains are homo oligomer their fasta
        #are stored together.
            
        for lines in resp:

            if lines[0] == ">":
                #we store all headers given they contain the chain info.
                lines_grp = lines.split("|")
                pdb_id = lines_grp[0].replace(">","")
                pdb_id = pdb_id[0:4].lower()
                infodict[pdb_id] = lines_grp[1:]

    with open(f"{path}/PCA_structures_info/PCA_rcsb_headers.tsv", "w") as fh_out:
        for keys, vals in infodict.items():
            fh_out.write(keys)
            fh_out.write("\t")
            for entries in vals:
                fh_out.write(entries)
                fh_out.write("\t")
            fh_out.write("\n")
            

In [22]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/ADH1G_HUMAN/dimer/pos_1_375"

#get_rcsb_info(path)

# DOWNLOAD PDBS function

requires:

+ batch_download_modified.sh


In [23]:
def batch_download_pdbs(gene_name:str):
    
    """Function to download pdbs and if not possible.. download mmcif."""
    
    path = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/{gene_name}"
    
    #ugly but works.
    os.chdir("/home/micnag/bioinformatics/rcsb_retrieved_pdbs/")
    #this bash cmd will fetch all possible pdbs.    
    bash_curl_cmd = f"./batch_download_modified.sh -f {path}/pdbfile.txt -o {path} -p"
    
    bash_curl_cmd_rdy = bash_curl_cmd.split()
    
    #in the .sh file I piped to stdout if we did not manage to get a pdb.
    #stdout -> rcsbid_chain that did not work
    
    #if we already downloaded, dont download again!.
    
    check_path = f"{path}/monomeric"
    
    query_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f[-4:] == ".pdb"]
    
    #if we have already pdbs downloaded, dont do it again. IF monomeric dir already exists, we dont need to run it again.
    if len(query_files) == 0 and os.path.isdir(check_path) == False:
        result_pdbs = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                             universal_newlines=True)
        
        """THIS PART IS CONCERNING WITH MMCIF FILES AND NEEDS TO BE UPDATED LATER."""
        result_list = result_pdbs.stdout.split("\n")

        #now lets check if there are mmcif files to gather.
        if len(result_list) != 0:
            #first setup input file that is similar to pdbfile.txt and then rerun bash for mmcif
            with open(f"{path}/mmciffile.txt", "w") as mmcif_tar:
            
                for mmcifs in result_list[:-1]: #last one is always empty
                
                    mmcif_tar.write(mmcifs)
                
                    mmcif_tar.write(",")
                
            bash_curl_cmd = f"./batch_download_modified.sh -f {path}/mmciffile.txt -o {path} -c"
        
            bash_curl_cmd_rdy = bash_curl_cmd.split()
        
            result_pdbs = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                             universal_newlines=True)
    else:
        print(f"Dir has already some pdbs -> {len(query_files)}")

# Helper function / Utility functions

In [24]:
def _cleanup(path:str):
        
    #we dont need tmp pdb files given that all what we want is in dirs and no longer in this main dir.
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    for file in onlyfiles:
        os.remove(f"{path}/{file}")
    
    #also remove the shifts tmp folder to save disc space.
    try:
        shutil.rmtree(f'{path}/shifts')
    except Exception as error:
        print(error)

# Reduce redundancy and split proteins into domain chunks / group similar structure ranges together.

## _Split domains + make_groups

make_groups uses:

+ <span style="color:blue">_recursive_union_reduction</span>


In [25]:
def _split_domains(templates:list, temp_start:list, temp_end:list, query_start:list, query_end:list):
  
    print(len(templates))
    print("\n")
    
    #this will give the maximal possible range we will look into.
    abs_seq_min = min(temp_start)
    abs_seq_max = max(temp_end)
    
    abs_range = abs_seq_max - abs_seq_min
    tolerance = 0.2 *abs_range
    
    seq_lengths = []
    
    for start, end in zip(query_start, query_end):
        seq_lengths.append(end - start)

    
    seq_ranges = []
    
    testdict = defaultdict(list)
    
    for temp, start_q, end_q, length in zip(templates, query_start, query_end, seq_lengths):
        testdict[(start_q, end_q)].append(temp)
    
    start_stop_list = []
    val_list = []
    
    
    for keys, values in testdict.items():
        start_stop_list.append(keys)
        val_list.append(values)
    
    print(f"These are the ranges that we deal with: {start_stop_list} \n")
    
    if len(start_stop_list) != 1:
        split_dictionary = _make_groups(start_stop_list, val_list)    
    else:
        split_dictionary = defaultdict(list)
    
    print(f"These are the merged ranges: {[x for x in split_dictionary.keys()]}")
    
    
    single_interval_dict = defaultdict(list)
    
    #i = 0
    #for vals in val_list:
        #print(vals)
        #print(Interval(start_stop_list[0][0], start_stop_list[0][1]))
    #    single_interval_dict[Interval(start_stop_list[i][0], start_stop_list[i][1])] = vals
    #    i += 1
    
    #this contains the domains we need to split and each set of structures associated per domain.
    
    split_return_dict = defaultdict()
    
    for keys, vals in split_dictionary.items():
        #only care about more than 1 structure.
        if len(vals) > 1: 
            split_return_dict[keys] = vals
    
    cnt = 0
    for keys, vals in split_return_dict.items():
        cnt += len(vals)
        
    return split_return_dict

In [26]:
def _make_groups(datalst:list, val_list:list):
    
    storage_dict = defaultdict(list)
    
    for ranges, vals in zip(datalst, val_list):
        storage_dict[Interval(ranges[0],ranges[1])] = vals
    
    
    #works here still
    result_dict = defaultdict(list)
    
    intervals = [Interval(begin, end) for (begin, end) in datalst]
    
    
    #print("this are the intervals we deal with:")
    #print(intervals)
    #print("\n")
    
    #good so far. all 34 structures are in here for case NUDT4
    
    interval_dict = defaultdict()
    
    for interv in intervals:
        interval_dict[interv] = interv
        

    print(f"length of dictionary before recursive_reduction: {len(interval_dict)}")
    if len(interval_dict) != 1:
        result = _recursive_union_reduction(interval_dict, 0)
        print(f"length of dictionary after recursive_reduction: {len(result)}\n")
    else:
        result = interval_dict
        #for keys, vals in result.items():
        #    print(len(vals))
            
        return result
    
    
    interval_collection_dict = defaultdict(list)
    
    for keys, vals in result.items():
        #for each key we need to find all associated intervals within the key range e.g all subsets.
        
        for interval_member in intervals:
            condition = interval_member.is_subset(keys)
            condition_2 = interval_member.measure > 0.7*keys.measure
            if condition and condition_2:
                interval_collection_dict[keys].append(interval_member)
    
    #print(len(intervals))
    
    rcsb_dict = defaultdict(list)
    
    for keys, vals in interval_collection_dict.items():
        
        if type(vals) == list:
            rcsb_tmp = []
            for val in vals:
                tmp = storage_dict[val]
                #print(f"this is current key: {keys}")
                #print(f"this is current structures: {storage_dict[val]}")
                #print(tmp)
                for rcsbs in tmp:
                    rcsb_tmp.append(rcsbs)
            
        else:
            rcsb_tmp = []
            tmp = storage_dict[val]
            for rcsbs in tmp:
                rcsb_tmp.append(rcsbs)
        
            rcsb_dict[keys] = rcsb_tmp
        
        rcsb_dict[keys] = rcsb_tmp
        
        
        
    #for keys, vals in rcsb_dict.items():
        #print(keys)
        #print(vals)
        #print("\n")
    
    return rcsb_dict

In [27]:
def _recursive_union_reduction(dictionary_to_parse:dict, num_of_iterations:int):
    
    #print("this is dictionary to parse:")
    #print(dictionary_to_parse)
    
    if num_of_iterations < 20:
        
        #if everything merged together we dont need to continue
        if len(dictionary_to_parse) == 1:
            return dictionary_to_parse
        
        num_of_iterations += 1
        val_1 = []
        val_2 = []
        
        for keys, vals in dictionary_to_parse.items():
            val_1.append(keys)
            val_2.append(keys)
        
        union_dict = defaultdict(list)        
        
        for val in val_1:
            union_new = False
            for val_x in val_2:
                if val != val_x:
                    m_1 = val.measure
                    m_2 = val_x.measure
                    m_3 = val_x.intersect(val).measure
                    
                    #print(val, val_x)
                    #print(m_1, m_2, m_3)
                    condition_1 = m_3 <= m_1 and m_3 >= 0.8* m_1
                    condition_2 = m_3 <= m_2 and m_3 >= 0.8* m_2
                    #print(condition_1, condition_2)
                    if condition_1 and condition_2:
                        union_new = Union(val, val_x)
                        val_tmp_1 = val
                        val_tmp_2 = val_x
            
            if union_new:
                union_dict[union_new].append(val_tmp_1)
                union_dict[union_new].append(val_tmp_2)
                
            else:
        
                union_dict[val].append(val)

        dictionary_to_parse = union_dict
        
        dictionary_to_parse = _recursive_union_reduction(dictionary_to_parse, num_of_iterations)
    
    
    return dictionary_to_parse

# Split based on oligomers.

Input:

1. <span style="color:green">interval_dirs:dict</span>
2. <span style="color:green">oligostates:dict</span> 
3. <span style="color:green">path:str</span> 
4. <span style="color:green">rangedir:dict</span>

Output:
makes and sets up the required directories for the different oligomeric states.

Helper functions:

+ <span style="color:blue">get_seq_len</span>
+ <span style="color:blue">get_gene_name</span>
+ <span style="color:blue">temp_score</span>


In [28]:
def make_oligo_dirs(interval_dirs:dict, oligostates:dict, path:str, rangedir:dict):
    
    #build those out of the  /monomeric subdirectory!!!
    
    #intervaldir = f"{path}/monomeric"
    
    #print("this is interval dirs")
    #print(interval_dirs)
    
    intervaldirs = [ f for f in interval_dirs.keys()]
    
    #print(intervaldirs)
    
    oligodirdict = dict([("1",'monomer'),
                 ("2",'dimer'),
                 ("3",'trimer'),
                 ("4",'tetramer'),
                 ("5",'pentamer'),
                 ("6",'hexamer'),
                 ("7",'heptamer'),
                 ("8",'oktamer'),
                 ("9",'nonamer'),
                 ("10",'decamer'),
                 ("11",'undecamer'),
                 ("12",'dodecamer'),
                 ("13",'tridecamer'),
                 ("14",'tetradecamer'),
                 ("15",'pentadecamer'),
                 ("16",'hexadecamer'),
                 ("17",'heptadecamer'),
                 ("18",'oktadecamer'),
                 ("19",'nonadecamer'),
                 ("20",'eicosamer')])
    

    print("here start dict coll")
    print(intervaldirs)
    print(oligostates)
    print(rangedir)
    
    #grabs info about which directory to make.
    dirs_to_make = list(set([x for x in oligostates.values()]))
    
    for intervals in intervaldirs:
        start = intervals.start
        end = intervals.end
        
        pos_name = f"pos_{start}_{end}"
        for dirs in dirs_to_make:
            try:
                
                name = oligodirdict[str(dirs)]
                #print(f"{path}/{name}/{pos_name}")
                
                os.makedirs(f"{path}/{name}/{pos_name}")
                
            except Exception as error:
                print(error)
        
    
    #here might be an issue
    #print("this is oligostates dict")
    #print(oligostates)
    for keys, values in oligostates.items():
        try:
            #HERE SEEMS TO BE AN ISSUE WITH KEYS. CHECK.
            previous_path = f"{path}/merged_cleaned_files/{keys}.pdb"
            
            
            #range is 19-180 in first iteration.
            for ranges, vals in rangedir.items():
                for val in vals:
                    #current val: 6ckv_A, current key: 5wdd                    
                    #defaultdict(<class 'str'>, {'5wdd': 1, '6ckv_1': 1, '6ckv_2': 1, '6ckv_3': 1, '6ckv_4': 1, '6ckv_5': 1, '6ckv_6': 1, '6ckv_7': 1, '6ckv_8': 1, '6ckv_9': 1, '6ckv_10': 1, '6ckv_11': 1, '6ckv_12': 1, '6ckv_13': 1, '6ckv_14': 1, '6ckv_15': 1, '6ckv_16': 1, '6ckv_17': 1, '6ckv_18': 1, '6ckv_19': 1, '6ckv_20': 1, '6v4m': 1, '6yld': 2})
                    #defaultdict(None, {Interval(19, 180): ['6ckv_A', '5wdd_A', '5wdd_B', '6v4m_A', '6yld_A', '6yld_C']})
                    if val[0:4] == keys[0:4]:
                        start = ranges.start
                        end = ranges.end
                        
                        dir_name_to_use = f"pos_{start}_{end}"
            
                        #print(f"{path}/{oligodirdict[str(values)]}/{range_to_save}")
            
                        savepath = f"{path}/{oligodirdict[str(values)]}/{dir_name_to_use}"

                        #print(f"previous path: {previous_path}, savepath: {savepath}")
                        shutil.copy(previous_path, savepath)
            
        except Exception as error:
            print(error)
    
    try:
        #shutil.rmtree(f"{path}/merged_cleaned_files")
        pass
    except:
        pass

# Atomium part / Biological assemblies

### Major Subfunction

In [29]:
def get_biological_assemblies_atomium(path:str, gene_name:str, main_iso_seq:str,
                                     main_protein_seq:str):

    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    relevant_files = [f for f in onlyfiles if f[-3:] == "pdb" or f[-3:] == "cif"]
    relevant_files = [f for f in relevant_files if len(f) == 8]
    
    oligodict = dict([("1",'monomer'),
                 ("2",'dimer'),
                 ("3",'trimer'),
                 ("4",'tetramer'),
                 ("5",'pentamer'),
                 ("6",'hexamer'),
                 ("7",'heptamer'),
                 ("8",'oktamer'),
                 ("9",'nonamer'),
                 ("10",'decamer'),
                 ("11",'undecamer'),
                 ("12",'dodecamer'),
                 ("13",'tridecamer'),
                 ("14",'tetradecamer'),
                 ("15",'pentadecamer'),
                 ("16",'hexadecamer'),
                 ("17",'heptadecamer'),
                 ("18",'oktadecamer'),
                 ("19",'nonadecamer'),
                 ("20",'eicosamer')])
             
    letters = ["A", "B", "C", "D", "E", "F",
              "G", "H", "I", "J", "K", "L",
              "M", "N", "O", "P", "Q", "R",
              "S", "T", "U", "V", "W", "X", "Y", "Z"]
    
    oligostates = defaultdict(str)  # initialize a dict that keeps track of oligostates.
    
    if os.path.exists(f"{path}/merged_cleaned_files") == False:
        os.mkdir(f"{path}/merged_cleaned_files")
    
    for files in relevant_files:
        
        #lets try for each file (could be that some are empty and we require skip without breaking the loop)
        try:

            pdb1 = atomium.open(f'{path}/{files}')
            
            #if there are more models in there we have an NMR structure.
            model_len = pdb1.models
            
            #this means we have an NMR ensemble.
            if len(model_len) > 5:   #usually they are 20

                oligo_nmr = _NMR_ensemble(path = path,
                             files = files, gene_name=gene_name)
                #merge them into the normal oligostates dict.
                for keys, vals in oligo_nmr.items():
                    #keys = pdb_code _ idx e.g 2ro8_10    2ro8_11 etc.
                    oligostates[keys] = vals
                
                #continue needs to still be here.
                continue
                
            #else means we have NO NMR structure.
            else:
                
                oligostate = _non_NMR_structures(path=path,
                                   files=files,
                                   gene_name=gene_name,
                                   main_protein_seq=main_protein_seq)
                
                oligostates[files[0:4]] = oligostate
                
        except Exception as error:
            print(error)
            
            # we still continue with the next file if we cant open e.g because its empty file.
            continue
    
    #we return oligostates dict.
    print("this is oligostates")
    print(oligostates)
    return oligostates

In [30]:
#helper function for XRAY and CRYO-EM ensembles.
def _non_NMR_structures(path:str,
                        files:str,
                        gene_name:str,
                        main_protein_seq:str):
    
    """This function takes in the the pdb file that is xray or cryoem and rechains each chain. 
    Additionally, we merge the new labelled chains into a merged_pdb file for further use."""
    
    #open the pdb file
    pdb1 = atomium.open(f'{path}/{files}')
    
    assemblies = [pdb1.generate_assembly(n + 1) for n in range(len(pdb1.assemblies))]
    
    #we take the first one(this is the biological unit built from the asymmetric unit)
    assembly = assemblies[0]
    
    print(f'{path}/{files}')
    print(assembly)
    #this works for our purpose.
    
    seq_chains = []
    
    accepted_chains = []
    
    for x in assembly.chains():
        
        seq = x.sequence #the sequence of each chain
        
        seq_len = len(seq) #the len of each chain aka its number of residues
        
        chain_label = x.id #the chain identifier
        
        seq_chains.append((chain_label,seq_len)) 
    
    
    sorted_lens = sorted(seq_chains, key= lambda x: x[1], reverse=True) #reverse = true :largest first.

    acc_range = []
        
    for chains, lens in sorted_lens:
        #first is largest and always accepted
        if len(accepted_chains) == 0 or lens > 0.8 * min(acc_range):
            #then we continue to append only if its similar size to prevent small peptides from interfering.
            accepted_chains.append(chains)
            acc_range.append(lens)
        
    
    print(f"this is accepted chains: {accepted_chains}, and this is accepted ranges: {acc_range}")

    """
    if the chain contains more than 70% of the number of residues that the main fasta seq has we continue.
    this part might become tricky if we have only partial structures of full length stuff available. might be
    adjusted later.
    """
   
    oligostate = len(accepted_chains)  #this excludes small peptides ect from being mistaken as oligomers.
            
            
    path_list = []
            
    seen_chains = []
    
    for idx, chains in enumerate(assembly.chains()):
                
        chain_label = chains.id
        
        if chain_label in accepted_chains:  #if its accepted we save it.
            
            print(chain_label, accepted_chains)
            
            print(f"{path}/{files[0:4]}_{chain_label}_{idx}.pdb")

            chains.save(f"{path}/{files[0:4]}_{chain_label}_{idx}.pdb")
        
            path_to_pdb = f"{path}/{files[0:4]}_{chain_label}_{idx}.pdb"
        
            #gap_dict = check_gaps_single_struc(path_to_pdb)
            
            #repairability = repair_viability(gap_dict)
            
            #if repairability:
            #    try:
            #        #modeller block here.
            #        repairing_residues(path=path, 
            #                       pdb_id_target=f"{files[0:4]}_{chain_label}_{idx}")
            #        
            #    except Exception as error:
            #        print("the error occured in for idx, chains in enumerate(assembly.chains()):")
            #        print(error)
            
            print("we append now path")
            path_list.append(f"{path}/{files[0:4]}_{chain_label}_{idx}.pdb")
            seen_chains.append(chain_label)
            
            path_list = sorted(path_list, key=lambda x: int(x[-5]))
            #print(path_list)
            unique_chains = list(set(seen_chains))

            #for paths in path_list:

            _merge_pdb_chains(path_list, pdb_name=files[0:4],
                      gene_name=gene_name,
                      seen_chains=seen_chains, 
                      unique_chains=unique_chains)
    
            
        else: #we continue.
            """This means the part that is actually attached is not part of the protein but is instead a bound ligand!
            if this ligand analysis becomes relevant in the future in the context of mutational investigations
            we might adjust this part here."""
            continue
    
    #we return the oligostate of this file and merge it into dict as return value.
    return oligostate
    

In [31]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN"
#files = "4h1w.pdb"
#gene_name = "serca"


#main_prot_seq = get_gene_fasta("O14983")
#_non_NMR_structures(path=path,
#                        files=files,
#                        gene_name=gene_name,
#                        main_protein_seq=main_prot_seq)



In [32]:
#helper function for NMR ensembles.
def _NMR_ensemble(path:str, files:str, gene_name:str):

    
    """This function takes in the NMR ensemble and splits each state into a respective PDB file."""
    
    
    #open the pdb file
    pdb1 = atomium.open(f'{path}/{files}')
    
    assembly = pdb1.generate_assembly(1)
    
    
    oligostates = defaultdict()
    
    for idx, model in enumerate(pdb1.models):
                        
        path_list = []
        seen_chains = []
        
        #here we save the structure.
        model.save(f"{path}/{files[0:4]}_{idx+1}.pdb")
        
        #we append path
        path_list.append(f"{path}/{files[0:4]}_{idx+1}.pdb")
        #we append seen chains.
        for chains in assembly.chains():
            
            #here we grab the chain ID
            chain_label = chains.id
            fasta_seq = chains.sequence
            
            oligostates[f"{files[0:4]}_{idx+1}"] = len(assembly.chains())
            
            seen_chains.append(chain_label)
        
            unique_chains = list(set(seen_chains))
            
        #for paths in path_list:
        # we treat it like its a normal monomer (since NMR is single chain most of the time)
        _merge_pdb_chains(path_list, pdb_name=f"{files[0:4]}_{idx+1}", 
                          gene_name=gene_name,
                          seen_chains=seen_chains,
                          unique_chains=unique_chains)
        
        
    return oligostates
    

In [33]:
#path = "/home/micnag/bioinformatics/test/laura_test"
#files = "2ro8.pdb"
#gene_name = "NAC3_HUMAN"

#oligostate = _NMR_ensemble(path=path, 
#             files=files,
#             gene_name=gene_name)
#
#print(oligostate)

#### Testblock for atomium 

In [34]:
#test for get_biological_assemblies_atomium

#gene_name = "CALM1_HUMAN"
#path = "/home/micnag/bioinformatics/test/atomium_CALM1_testset"

#main_iso_seq = None

#print(get_biological_assemblies_atomium(gene_name=gene_name,
#                                  path=path,
#                                 main_iso_seq=main_iso_seq))

In [35]:
def remove_hetero_atoms(pdb_file:str, path:str):
    """This function will grab the correct chains and overwrite the pdbs with only correct chains."""
    
    #used to get rid of hetero atoms and wrong chain.
    #inherit from Select, pass additional arg to __init__: correct_chain id
    class NonHet_and_correct_chain_Select(Select):
        def __init__(self, *args):
            super().__init__(*args)
            
        #overload accept_residue inherited from Select with this conditional return
        def accept_residue(self, residue):
            return 1 if residue.id[0] == " " else 0
        
    
    #filelst    path
    #5ltu_A.pdb /home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN
    
    parser = PDBParser(QUIET=True)
    
    prot_name = f"{pdb_file[0:4]}"
    
    fullpath = f"{path}/{pdb_file}"
    
    
    print("we are here")
    print(fullpath)
    structure = parser.get_structure(prot_name, fullpath)
    
    
    non_canoncial_aas = defaultdict()
    
    
    canonical_aas = ['VAL', 'ILE', 'LEU', 'GLU', 'GLN' ,
                    'ASP', 'ASN', 'HIS', 'TRP', 'PHE', 'TYR', 
                    'ARG', 'LYS', 'SER', 'THR', 'MET', 'ALA', 
                    'GLY', 'PRO', 'CYS']
    
    
    for models in structure:
        for chains in models:
            
            chain = chains.get_id()
            
            for residues in chains:
                curr_res = residues.get_resname()
                curr_pos = residues.get_id()[1] #the position
                
                
                if curr_res not in canonical_aas:
                    print(curr_res)
                    non_canoncial_aas[curr_pos] = (curr_res, chain)
    
    
    
    #io object to save structure object to file
    io = PDBIO()
    io.set_structure(structure)
    
    savepath = fullpath
    
    #we save all structures to the monomeric category.
    io.save(savepath, NonHet_and_correct_chain_Select())
    
    return non_canoncial_aas

### **Function**:
<b><span style="color:orange">_merge_pdb_chains:</span></b>

*Input:*
1. <span style="color:green">path_list:str</span> 
2. <span style="color:green">pdb_name:str</span>
3. <span style="color:green">gene_name:str</span>
4. <span style="color:green">seen_chains:str</span>
5. <span style="color:green">unique_chains:str</span>

<span style="color:green"></span>

calls internally following helper functions:

+ <span style="color:blue">_homomer_check</span>
+ <span style="color:blue">_pure_oligomer_rechaining</span>
+ <span style="color:blue">_mixed_oligomer_rechaining</span>
+ <span style="color:blue">_monomeric_rechaining</span>


requires additionally:

+ <span style="color:red">pdb_merge.py</span> 
+ <span style="color:red">pdb_tidy.py</span>


In [36]:
def _merge_pdb_chains(path_list:str, pdb_name:str, gene_name:str,
                     seen_chains:str, unique_chains:str):
    
    #needs to be adjusted how to handle hetero-mers.
    
    #for testing purpose we put this here : return
    
    letterdict = {1 : "A", 2 : "B", 3 : "C", 4 : "D",
                 5 : "E", 6 : "F",7 : "G", 8 : "H",
                 9 : "I", 10 : "J",11 : "K", 12 : "L",
                 13 : "M", 14 : "N",15 : "O", 16 : "P",
                 17 : "Q", 18 : "R",19 : "S", 20 : "T",
                 21 : "U", 22 : "V",23 : "W", 24 : "X",
                 25 : "Y", 26 : "Z" }
    
    #that means we have duplicates.
    
    #oligomeric heterodimer case first!
    
    #default setting
    
    oligohomomer = False
    
    if len(seen_chains) != len(unique_chains):
        #means its an oligomer.
        oligomer = True
        #now check if homo or hetero oligomer.
        oligohomomer = _homomer_check(seen_chains, unique_chains)
    else:
        oligomer = False
        
    #means its an oligomer and a homo oligomer.
    if oligohomomer:
        
        new_chains = _pure_oligomer_rechaining(seen_chains=seen_chains,
                                   unique_chains=unique_chains,
                                   path_list=path_list,
                                   letterdict=letterdict)
    
    #means its an oligomer but a hetero oligomer.
    if oligohomomer == False and oligomer:
        
        new_chains = _mixed_oligomer_rechaining(seen_chains=seen_chains,
                                   unique_chains=unique_chains,
                                   path_list=path_list,
                                   letterdict=letterdict)
    
    #means its just either a single chain of the protein or some co crystallized binding complex protein as other chains.
    if oligomer == False:
        
        new_chains = _monomeric_rechaining(seen_chains=seen_chains,
                                   unique_chains=unique_chains,
                                   path_list=path_list,
                                   letterdict=letterdict)
     
    #to be continued
    
    
    #print("this is path list and chains")
    #print(path_list)
    #print(seen_chains)
    #print(new_chains)
    
    pdb_paths = ""
    
    for entries in path_list:
        pdb_paths += f"{entries} "
    
    #print("this is pdb_paths")
    
    #print(pdb_paths)
    

    outfile = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/{gene_name}"
    
    bash_merge = f"python /home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_merge.py {pdb_paths}"
        
    bash_curl_cmd_rdy = bash_merge.split()
    
    bash_tidy_cmd = f"python /home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_tidy.py {outfile}/merged_cleaned_files/{pdb_name}_merged.pdb"
    
    bash_tidy_cmd_rdy = bash_tidy_cmd.split()
    
    #print("this is the merge command")
    
    #print(bash_curl_cmd_rdy)
    
    with open(f"{outfile}/merged_cleaned_files/{pdb_name}_merged.pdb", "w") as fh_out:
        result_pdbs = run(bash_curl_cmd_rdy, stdout=fh_out, stderr=PIPE, 
                             universal_newlines=True)
        
    
    #print("this is the tidy command thats breaking the merge.")
    #print(bash_tidy_cmd_rdy)
    
    with open(f"{outfile}/merged_cleaned_files/{pdb_name}.pdb", "w") as fh_out2:
        results_tidy = run(bash_tidy_cmd_rdy, stdout=fh_out2, stderr=PIPE, 
                             universal_newlines=True)
    
    #we remove tmp intermediate files.
    #os.remove(f"{outfile}/merged_cleaned_files/{pdb_name}_merged.pdb")
    
    with open(f"{outfile}/reports/chain_relabeling_protocol.csv", "a") as fh_chain:
        outtext = f'{pdb_name},{" ".join(sorted(seen_chains, reverse=False))},{" ".join(new_chains)}'
        fh_chain.write(outtext)
        fh_chain.write("\n")

In [37]:
def _homomer_check(seen_chains:list, unique_chains:list):
    
    # ABCDEF    set: ABCDEF  vs unique:ABCDEF
    
    # AA   set : A vs unique A
    
    #if they both are the same set but the seen chain len is larger e.g [A A] vs [A] but as set
    #they will both be [A] and [A] -> Means its a homodimer.
    
    
    
    #this means they differ and now it could be either a mixed or a homo oligomer.
    #if len(unique chains == 1) this means the assymetric unit consists of 1 chain. and the
    #biological unit is simply generated through means of symmetry operations.
    
    if len(seen_chains) != len(unique_chains) and len(unique_chains) == 1:
        return True
    
    #this case covers the mixed oligomers. nevertheless they are oligomers!
    if len(seen_chains) != len(unique_chains) and len(unique_chains) != 1:
        return False
    


In [38]:
def _pure_oligomer_rechaining(seen_chains:list,
                                 unique_chains:list,
                                 path_list:list,
                                 letterdict:dict):    
    
    
    seen_chains = sorted(seen_chains, reverse=False)

    merged_chain_paths = zip(seen_chains, path_list)
    
    for idx, (chains, path_to_pdb) in enumerate(merged_chain_paths):        
        
        parser = PDBParser(QUIET=True)
                
        prot_name = f"default"
                
        #open the correct pdb and rechain it.
        structure_template = parser.get_structure(prot_name, path_to_pdb)
        
        new_chain = letterdict[idx+1]
        
        new_chain_seq = []
        
        for models in structure_template:
            for chains in models:
                
                chains.id = "_"
                
                chains.id = new_chain
                
                new_chain_seq.append(new_chain)
                
                io = PDBIO()
                
                io.set_structure(structure_template)
                
                io.save(path_to_pdb)
                
    return new_chain_seq

In [39]:
def _mixed_oligomer_rechaining(seen_chains:list,
                               unique_chains:list,
                               path_list:list,
                               letterdict:dict):
    
    seen_chains = sorted(seen_chains, reverse=False)

    merged_chain_paths = zip(seen_chains, path_list)
    
    chain_seq_len = len(seen_chains) #e.g 6
    
    shift = len(unique_chains) # e.g 3
    
    blocksize = int(chain_seq_len / shift) # e.g 2
    
    block_count = int(chain_seq_len/blocksize)
    
    # A A B B C C becomes A D B E C F
    
    # B B C C becomes A C B D 
    
    #i = 1
    
    # A D 
    
    # block 1 2 3 for A A B B C C 
    
    # 0 2 1 3
    
    j = 0
    new_chain_seq = []
    for blocks in range(1, block_count+1):
        # each block has 2 members:
        #first iteration: 1 
        # second iteration: 2
        # third iteration: 3
        for i in range(0, blocksize):
            # first iteration A D
            # second iteration B E
            # third iteration C F
            new_chain = letterdict[blocks+i*shift]
            new_chain_seq.append(new_chain)
            path_to_pdb = path_list[j]
            #continue in path
            j += 1
            
            #lets renumber the structure now.
            parser = PDBParser(QUIET=True)
            
            prot_name = f"default"
            
            #open the correct pdb and rechain it.
            structure_template = parser.get_structure(prot_name, path_to_pdb)
            
            for models in structure_template:
                for chains in models:
                    chains.id = "_"
                    chains.id = new_chain
            
            io = PDBIO()
            
            io.set_structure(structure_template)
            io.save(path_to_pdb)

    return new_chain_seq

In [40]:
def _monomeric_rechaining(seen_chains:list,
                          unique_chains:list,
                          path_list:list,
                          letterdict:dict):
    
    merged_chain_paths = zip(seen_chains, path_list)
    
    new_chain_seq = []
    
    for idx, (chains, path_to_pdb) in enumerate(merged_chain_paths): 

        parser = PDBParser(QUIET=True)
                
        prot_name = f"default"
                
        #open the correct pdb and rechain it.
        structure_template = parser.get_structure(prot_name, path_to_pdb)
        
        new_chain = letterdict[idx+1]
        
        for models in structure_template:
            for chains in models:
                print("this is current chains")
                print(chains.id)
                chains.id = "_"
                
                chains.id = new_chain
                print("this is new chains")
                print(chains.id)
                
                new_chain_seq.append(new_chain)
                
                io = PDBIO()
                
                io.set_structure(structure_template)
                
                io.save(path_to_pdb)
                
    return new_chain_seq

# prepares references used for USAlign later downstream.

### Major Subfunction.

*Input:*

1. <span style="color:green">main_gene_name:str</span>
2. <span style="color:green">templates:list</span>
3. <span style="color:green">seq_sim:list</span>     
4. <span style="color:green">query_start:list</span>                      
5. <span style="color:green">query_end:list</span>
6. <span style="color:green">temp_start:list</span>                      
7. <span style="color:green">temp_end:list</span>                    
8. <span style="color:green">path:str</span>
9. <span style="color:green">oligodict:list</span>

*Output*:

<b><span style="color:brown">template_dict:dir</span></b>

In [41]:
def prepare_references(main_gene_name:str,
                      templates:list,
                      seq_sim:list,
                      query_start:list,
                      query_end:list,
                      temp_start:list,
                      temp_end:list,
                      path:str, oligodict:list,
                      custom_template=False):
    
    """Make function out of making templates and reference structures."""
    #setup
    
    print("we start prepare templates!!")
    
    #this retrieves the main len of the protein we are interested in
    main_target_seq_len = get_seq_len(path=f"{path}", rcsb_id="None", template=True)
    #181 for NUD4B
    
    #gives back the length of each template.
    temp_lengths = [f[0] - f[1] for f in zip(temp_end, temp_start)]
    
    homology_list = sorted(list(zip(templates,seq_sim, temp_lengths)), key= lambda x: x[1], reverse=True)
    
    homology_dict = defaultdict(tuple)
    
    #homology dict will be used later to fetch corresponding templates for each oligomer and each domain.
    for entries in homology_list:
        homology_dict[entries[0][0:4]] = (entries[1], entries[2])
    
    
    acceptable_dirnames = ["monomer",
                           "dimer",
                           "trimer",
                           "tetramer",
                           "pentamer",
                           "hexamer",
                           "heptamer",
                           "oktamer",
                           "nonamer",
                           "decamer",
                           "undecamer",
                           "dodecamer",
                           "tridecamer",
                           "tetradecamer",
                           "pentadecamer",
                           "hexadecamer",
                           "heptadecamer",
                           "oktadecamer",
                           "nonadecamer",
                           "eicosamer"
    ]
    
    intervaldirs = [ f for f in os.scandir(path) if f.is_dir() and f.name in acceptable_dirnames]
    
    intervalpathdict = defaultdict(list)
    
    for subentities in intervaldirs:
        intervals = [ f for f in os.scandir(subentities.path) if f.is_dir()]
        for interv in intervals:
            onlyfiles = [f for f in os.listdir(interv.path) if os.path.isfile(os.path.join(interv.path, f))]
            if len(onlyfiles) != 0:
                intervalpathdict[interv.path] = onlyfiles
                
    template_dict = defaultdict(str)
    
    #print("this is intervalpathdict")
    #print(intervalpathdict)
    
    
    for keys, vals in intervalpathdict.items():
        
        
        fin_dir_list = keys.split("/")
        
        #save position dir
        position_dir = fin_dir_list[-1]
        #and oligostate in order to write report later on.
        oligostate_dir = fin_dir_list[-2]
        
        ref_list = []
        #here we store human hits.
        priority_list = []
        #if we retrieve not as many hits, we take what we get
        
        if len(vals) < 100:
                
            max_end = len(vals)
        
        else:
            
            max_end = 100
        
        
        print(f"pos dir: {position_dir}, oligostate: {oligostate_dir}, val: {vals}")
        
        exact_hits = []
    
        #group 2: other species same gene.  e.g KRS1_DROSOPHILA instead of KRS1_HUMAN
        other_species_same_gene = []
    
        #group 3: other HUMAN but not exact gene. KRS2_HUMAN instead of KRS1
        other_human_similar_gene = []
    
        #we dont find anything from the above.
        suboptimal_templates = []
        
        for val in vals:
            
            try:
                
                prot_check = val[0:4] #pdb code without chain id
            
                prot_name = get_gene_name(prot_check)
            
                seq, lengths = homology_dict[prot_check][0], homology_dict[prot_check][1] 
            
                #print(seq, lengths)
                
                #this means its human and our main prot is always human. but this should be extended.
                #sometimes its better to have e.g MOUSE protein instead another less related human protein!
                
                
                
                #case 1
                if prot_name == main_gene_name:
                    temp_score_val = temp_score(seq_sim=float(seq),
                                                seq_len=lengths,
                                                main_seq_len=main_target_seq_len)
                                                
                    exact_hits.append((val, prot_name, temp_score_val))
                    continue
                
                #case 2
                if prot_name[:-6] == main_gene_name[:-6]:
                    temp_score_val = temp_score(seq_sim=float(seq),
                                                seq_len=lengths,
                                                main_seq_len=main_target_seq_len)
                                                
                    other_species_same_gene.append((val, prot_name, temp_score_val))
                    continue
                
                #case 3
                if prot_name[-5:] == "HUMAN":
                    temp_score_val = temp_score(seq_sim=float(seq),
                                                seq_len=lengths,
                                                main_seq_len=main_target_seq_len)
                                                
                    other_human_similar_gene.append((val, prot_name, temp_score_val))
                    continue
                
                #if its none of the above.
                temp_score_val = temp_score(seq_sim=float(seq),
                                                seq_len=lengths,
                                                main_seq_len=main_target_seq_len)
                
                suboptimal_templates.append((val, prot_name, temp_score_val))
                
            except Exception as error: 
                print(error)
        
        
        
        #case 0 : custom_template = True which means we want a search based on a specific pdb template.
        
        pdb_shorts = [x[0:4] for x in vals]  #just interested in 4 letter pdb codes.
        
        if custom_template in pdb_shorts:
            #we check if submitted struc is available at the current oligomer level.
            try:
                
                prot_check = custom_template[0:4] #only pdb code without chain id
                prot_name = get_gene_name(custom_template)
                
                #if we have it in the dict its fine. (it should be there if we did a proper seq search beforehand)
                seq, lengths = homology_dict[prot_check][0], homology_dict[prot_check][1] 
                
                #we calculate temp score 
                temp_score_val = temp_score(seq_sim=float(seq),
                                        seq_len=lengths,
                                        main_seq_len=main_target_seq_len)
                
                
                #we count it as a perfect hit.
                exact_hits.append((f"{custom_template[0:4]}.pdb", prot_name, 0))
                
            except Exception as error:
                print(error)
                continue
            
        
        
        
        if len(exact_hits) != 0:
            
            exact_sorted = sorted(exact_hits, key= lambda x: x[2])
            print("this is exact sorted")
            print(exact_sorted)
            
            with open(f"{path}/mutational_mapping/{oligostate_dir}_{position_dir}.csv", "w") as fh_out:
                #we take it from here
                #we only store the first potential structure for each range and oligomeric state!
                previous_path = f"{path}/{oligostate_dir}/{position_dir}/{exact_hits[0][0]}"
                #and store it here.
                savepath = f"{path}/mutational_mapping/{oligostate_dir}_{position_dir}_{exact_hits[0][0]}"
                shutil.copy(previous_path, savepath)
                fh_out.write(exact_hits[0][0])
                fh_out.write(",")
                fh_out.write(exact_hits[0][1])
                fh_out.write("\n")
        
        #print(exact_hits)
        #print(other_species_same_gene)
        #print(other_human_similar_gene)
        #print(suboptimal_templates)
        
        
        pos_oligomer = f'{oligostate_dir}/{position_dir}'
        template_dict[pos_oligomer] = [exact_hits, other_species_same_gene, other_human_similar_gene,
                                      suboptimal_templates]

    #for keys, vals in template_dict.items():
    #    print(keys)
    #    print(vals)
    #    print("\n")
    
    print("template dict is")
    print(template_dict)
    
    return template_dict


In [42]:
def get_seq_len(path:str, rcsb_id:str, template=False):
    
    seq_len = 0

    target = f"{path}/reports/{rcsb_id}.fasta"
    
    if template:
        target = f"{path}/reports/main_isoform_fasta"
    
    try:
        with open(target, "r") as fasta_file:
            for lines in fasta_file:
                if lines[0] != ">":
                    lines = lines.replace("\n","") #get rid of newline char
                    seq_len += len(lines) #add length of line to seq    
    except:
        #it did not work.
        return 0 
    
    return seq_len


In [43]:
def get_gene_name(uniprot_id:str):
    
    fields = "id"
    
    URL = f"https://rest.uniprot.org/uniprotkb/search?format=tsv&fields={fields}&query={uniprot_id}"
    resp = get_url(URL)
    resp = resp.text
    resp = resp.split("\n")
    return resp[1]

In [44]:
def temp_score(seq_sim:float, seq_len:float, main_seq_len:float):
    
    
    #seq_similarity * sequence len
    tmp = (seq_sim*seq_len)
    
    #to prevent sqrt 0 for ideal matches.
    eps = 1e-10
    
    #minimum squared diff between tmp and main_seq len is best
    temp_score = math.sqrt((tmp - main_seq_len+eps)**2)
    
    return temp_score

# SEQ shift compared to UNIPROT + renumber whole structures.


### function *get_shifts*:

requires:

+ <span style="color:red">get_shift.sh</span>

Output:

<span style="color:brown">shift_dict</span>


In [45]:
def new_get_shifts(pdbfolder:str):
    
    onlyfiles = [f for f in os.listdir(pdbfolder) if os.path.isfile(os.path.join(pdbfolder, f))]
    
    pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]


    link_path = "https://www.ebi.ac.uk/pdbe/api/mappings/uniprot"
    #we will fetch the results through request.
    shift_dict = defaultdict()
    
    for pdb in pdbs:
        searchp = f"{link_path}/{pdb[0:4]}"
        #print(searchp)
        resp = get_url(searchp)
        resp = resp.json()
        
        for pdb_id, pdb_info in resp.items():
            for uniprot_id, uniprot_info in pdb_info['UniProt'].items():
                for mapping in uniprot_info['mappings']:
                    chain_id = mapping['chain_id']
                    unp_start = mapping['unp_start']
                    unp_end = mapping['unp_end']
                    #print(chain_id)
                    #print(unp_start)
                    #print(unp_end)
                    author_start = mapping['start']['author_residue_number']
                    author_end = mapping['end']['author_residue_number']

                    #print(author_start)
                    #print(author_end)

                    if author_start == None:
                        author_start = unp_start
                    if author_end == None:
                        author_end = unp_end

                    shift_start = unp_start - author_start
                    shift_end = unp_end - author_end

                    
                    shift_dict[f"{pdb_id}_{chain_id}"] = shift_start 

    for keys, vals in shift_dict.items():
        print(keys, vals)

    with open(f"{pdbfolder}/shifts/shiftdict.txt", "w") as shift_out:
        for keys, vals in shift_dict.items():
            shift_out.write(keys)
            shift_out.write(str(vals))
            shift_out.write("\n")
    
    return shift_dict

In [46]:
def get_shifts(pdbfolder:str):
    
    savefolder = f"{pdbfolder}/shifts"
    basepathlst = pdbfolder.split("/")
    #print(basepathlst)
    
    basepath = "/".join(basepathlst[1:5])
    #print(basepath)
    
    os.chdir(f"/{basepath}")
    
    bash_correct_shift = f"./get_shift.sh {pdbfolder} {savefolder}"
    
    #print(bash_correct_shift)
            
    bash_command = bash_correct_shift.split()
    
    result = run(bash_command, stdout=PIPE, stderr=PIPE,
                universal_newlines=True)

    print(result.stdout)
    print(result.stderr)
    #print(result.stdout)
    
    try:
        
        shift_dict = defaultdict()
        
        with open(f"{pdbfolder}/shift_dict.txt", "r") as shiftdic_fh:
            for lines in shiftdic_fh:
                key, val = lines.replace("\n","").split(":")
                shift_dict[key] = val
                
    except Exception as error:
        print(error)
        
    #write a report file as well
    with open(f"{pdbfolder}/reports/shift_reports.txt", "w") as shiftout:
        for keys, vals in shift_dict.items():
            shiftout.write(keys)
            shiftout.write("\t")
            shiftout.write(vals)
            shiftout.write("\n")
        
    return shift_dict   

### function *renumber_whole_structures*:

requires:

+ <span style="color:red">pdb_shiftres_by_chain.py</span>


In [47]:
def renumber_whole_structures(shift_dict:dict, path:str):
    
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    relevant_files = [f[0:4] for f in onlyfiles if f[-3:] == "pdb" or f[-3:] == "cif"]
    
    #print(shift_dict)
    for files in relevant_files:
        for keys, vals in shift_dict.items():
            #print(files, keys[0:4])
            if files == keys[0:4] and vals != str(0):
                
                chain = keys[-1]
                shift = int(vals)
                
                filepath = f"{path}/{files}.pdb"
                
                #should we really shift by shift + 1??? or just shift?
                
                bash_cmd = f"python /home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_shiftres_by_chain.py {filepath} {shift} {chain}"
                
                bash_cmd_rdy = bash_cmd.split()
                
                with open(f"{filepath}_tmp", "w") as fh_tmp:
                    result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, 
                         universal_newlines=True)
                
                #now replace the original one with the temp file.
                os.replace(f"{filepath}_tmp", filepath)
                            
            else:
                continue

In [48]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/"

#shift_dict = get_shifts(pdbfolder=path)

#renumber_repaired_single_chain(shift_dict=shift_dict, path = path, pdb_name="7nnj_B_0")



# USALIGN + REPORT

### USAlign 

Input:

1. <span style="color:green">templates:dict</span>
2. <span style="color:green">gene_name:str</span>
3. <span style="color:green">path:str</span>
4. <span style="color:green"> **optional** report=True</span>


requires:

+ <span style="color:red">./USalign (.exe, C++)</span>


helper_functions:

+ <span style="color:blue">_get_tm_scores_and_rmsd</span>
+ <span style="color:blue">us_report</span>
+ <span style="color:blue">multiple_seq_alignment_pre_pca_processing</span>

In [49]:
def USAlign(templates:dict, gene_name:str, path:str,
            report=True)-> list:
    
    """ This function will call subprocess bash script USalign to compute Tms and RMSD values
        for our template and each other protein available on path.
        
        params:
                template: contains the potential template structure.
                
        RETURN: 
                list of tuple: (TEMPLATE, QUERY, TM2, RMSD)"""

    #here we save the report
    report_path = f"{path}"
    
    #this path is used to handle the structures for each ensemble at each oligomeric level of information
    result_list = []
    
    #example ('5ltu_A.pdb', 'NUDT4_HUMAN', 9.159999999899997, 179, 0.96)
    #index to try for selection of template (if highest ranked template number 1 does not work because file is unreadable, emprelevant_dirs    #try select template ranked 2)
    
    #we go through all potential templates
    
    overall_result_list = []
    
    
    #for each category (position and oligomer) we need to pick a template. 
    #naturally, ideal exact hits are first order priority.
    
    #vals == list of lists
    
    
    template_dict = defaultdict()
    
    min_seq_len_dict = defaultdict()
    
    print("this is templates")
    print(templates)
    
    for keys, vals in templates.items():
        #lists in vals: single list
        #if this condition is true we break the search and continue.
        found = False
        for lists in vals:
            #entries of single list.
            if found == True:
                break
            for x in lists:
                #lets append the first hit.
                template_dict[keys] = x
                found = True
                break
    
    for keys, vals in template_dict.items():
        pdb_id = vals[0]
        pdb_gene_name = vals[1]
        template_score = vals[2]
        
        #I used ">" as a seperator before for oligomer>position
        basepath = keys.split("/")
        
        dir_path_for_usalign = f"{path}/{basepath[0]}/{basepath[1]}" 
        
        print("this is dir path for usalign")
        print(dir_path_for_usalign)
        
        #this corresponds to our selected template.
        template = f"{dir_path_for_usalign}/{pdb_id}"
        
        #now lets compare it against all other structures in this directory
        pdb_files = [f for f in os.listdir(dir_path_for_usalign) if os.path.isfile(os.path.join(dir_path_for_usalign, f))]
        
        #if we have less than 2 files we dont need to any of the below.
        
        if len(pdb_files) < 2:
            
            continue
        
        for rcsb_file in pdb_files:
            #dont run USALIGN against itself..
            if rcsb_file != pdb_id:
                
                path_struc_1 = f"{dir_path_for_usalign}/{rcsb_file}"
                
                bash_tm_and_rmsd_calc = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/USalign {path_struc_1} {template} -outfmt 2"
            
            
            
                bash_command = bash_tm_and_rmsd_calc.split()
                
                try:

                    result = run(bash_command, stdout=PIPE, stderr=PIPE, 
                         universal_newlines=True)
    
                    
                    tm_2, rmsd = _get_tm_scores_and_rmsd(result.stdout) #both results are str.
                
                    
                    print(rcsb_file, pdb_id)
                    print(tm_2, rmsd)
                    if float(tm_2) > 0.5: #and float(rmsd) > 0.5:  #lets try less strict!
                        
                        us_report(rcsb_file, pdb_id, tm_2, rmsd, path,
                              template_score=template_score,
                              oligomer=basepath[0], pos=basepath[1])
                    #changed from f"{path_struc_1}" to path_struc_1
                    else:
                        os.remove(path_struc_1)
                    
                except Exception as error:
                    print(error)
    
    
    
    
        #now run the all against all
        #lets try version 1 for the full length all chains alignments.
        min_seq_len = multiple_seq_alignment_pre_pca_processing_version_2(path_to_pdbs=dir_path_for_usalign,
                                                  template=pdb_id)
        
        #set global minima to min seq len
        min_seq_len_dict[keys] = min_seq_len
        
    
    
    print("this is min seq len dict")
    print(min_seq_len_dict)
    return min_seq_len_dict, template_dict

### Us_report:

simple helper function that writes out a report:

sep="\t"

Scheme:

Query, Template, TM_2, RMSD, Template_score


In [50]:
def us_report(rcsb_file:str, pdb_id:str, tm_2:str, rmsd:str, path:str,
             oligomer:str, pos:str, template_score:str):
    
    
    path_to_assembly = f"{path}/reports/{oligomer}_{pos}.csv"
    
    if os.path.isfile(path_to_assembly) == False:
        with open(path_to_assembly, "w") as fh_start:
            fh_start.write("Query")
            fh_start.write(",")
            fh_start.write("Template")
            fh_start.write(",")
            fh_start.write("TM_2")
            fh_start.write(",")
            fh_start.write("RMSD")
            fh_start.write(",")
            fh_start.write("Template_score")
            fh_start.write("\n")
    
    with open(path_to_assembly, "a") as fh_out:
        fh_out.write(str(rcsb_file))
        fh_out.write(",")
        fh_out.write(str(pdb_id))
        fh_out.write(",")
        fh_out.write(str(tm_2))
        fh_out.write(",")
        fh_out.write(str(rmsd))
        fh_out.write(",")
        fh_out.write(str(template_score))
        fh_out.write("\n")

### _get_tm_scores_and_rmsd:

simple helper function that filters USAlign output and returns TM + RMSD vals as tuple:

(TM_2, RMSD)

In [51]:
def _get_tm_scores_and_rmsd(results:str):
    """ helper function to retrieve tm scores and rmsd
        We are only interested in TM_2 and RMSD.
        IF RMSD is HIGH and TM_2 HIGH that means we have a conformer.
        IF RMSD is LOW and TM_2 HIGH that means we have the same structure in the same conformer
        IF RMSD is HIGH and TM_2 LOW that means the structures are not related."""
    
    """['#PDBchain1', 'PDBchain2', 'TM1', 'TM2', 'RMSD', 'ID1', 'ID2', 'IDali', 'L1', 'L2', 'Lali\n/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/2duk_A.pdb:A', '/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/5ltu_A.pdb:A', 
    '0.8833', '0.9565', '0.94', '0.920', '1.000', '1.000', '138', '127', '127\n']"""
    
    res_list = results.split("\t")
    
    #this one is from the mobile protein
    tm_1 = res_list[12]
    #this one belongs to the target protein (the one we superimpose the mobile protein onto)
    tm_2 = res_list[13]
    
    #rmsd used to judge cutoff for trashing structures.
    rmsd = res_list[14]
    
    #we return tm_2 and rmsd
    return ((tm_2, rmsd))
    
    

### multiple_seq_alignment_pre_pca_processing

requires:

+ <span style="color:red">./USalign (.exe, C++)</span>

## PCA version 1

In [52]:
#testpath = "/home/micnag/bioinformatics/test/usaligntester"

#gap_dict = read_msa_file_version_1(path=testpath)



In [53]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/FAK1_HUMAN/monomer/pos_410_689"
#template_struc = "2v7a.pdb"
#print(_remove_deviating_length_strucs(path_to_pdbs=path, template_struc=template_struc))

# PCA PROCESSING PIPELINE 2 (full length all chains 1 vs 1 alignments)

In [54]:
def multiple_seq_alignment_pre_pca_processing_version_1(path_to_pdbs:str,
                                                  template:str):
    
    """-mm 1 -ter 0  for oligomers and all chains should be aligned."""
    
    
    min_seq_len = _remove_deviating_length_strucs(path_to_pdbs=path_to_pdbs, template_struc=template)
    
    
    only_pdbs = [f for f in os.listdir(path_to_pdbs) if os.path.isfile(os.path.join(path_to_pdbs, f))]
    
    with open(f"{path_to_pdbs}/chain_list.txt", "w") as chain_out:
        for pdbs in only_pdbs:
            #only for pdb files.
            if pdbs[-4:] == ".pdb":
                chain_out.write(pdbs)
                chain_out.write("\n")
    
    
    try:
        os.mkdir(f"{path_to_pdbs}/MSA_dir")
    
    except:
        print("already there")
    
    
    USalign_loc = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/"
    for pdbs in only_pdbs:
        if pdbs != template:
            
            bash_cmd = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/USalign {path_to_pdbs}/{pdbs} {path_to_pdbs}/{template} -mm 1 -ter 0"
            
            try:
                bash_cmd_rdy = bash_cmd.split()
                with open(f"{path_to_pdbs}/MSA_dir/{template[0:4]}_{pdbs[0:4]}.txt", "w") as msa_out:
                    result = run(bash_cmd_rdy, stdout=msa_out, stderr=PIPE, 
                        universal_newlines=True)
                
            except Exception as error:
                print(error)
                
    return min_seq_len
    

In [55]:
def multiple_seq_alignment_pre_pca_processing_version_2(path_to_pdbs:str,
                                                  template:str):
    
    """seq based alignment because we already did a structure based alignment."""
    
    #NOW we need to do a rigorous structure alignment different subroutine!
    
    
    
    #to get min seq len
    min_seq_len = _remove_deviating_length_strucs(path_to_pdbs=path_to_pdbs, template_struc=template)
    
    
    only_pdbs = [f for f in os.listdir(path_to_pdbs) if os.path.isfile(os.path.join(path_to_pdbs, f))]
    only_pdbs = [f for f in only_pdbs if f[-4:] == ".pdb"]
    
    with open(f"{path_to_pdbs}/chain_list.txt", "w") as chain_out:
        for pdbs in only_pdbs:
            #only for pdb files.
            if pdbs[-4:] == ".pdb":
                chain_out.write(pdbs)
                chain_out.write("\n")
    
    
    try:
        os.mkdir(f"{path_to_pdbs}/MSA_dir")
     
    except:
        print("already there")
    
    
    USalign_loc = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/"
    for pdbs in only_pdbs:
        if pdbs != template:
            
            print(pdbs, template)
            bash_cmd = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/USalign {path_to_pdbs}/{pdbs} {path_to_pdbs}/{template} -byresi 5"
            
            try:
                bash_cmd_rdy = bash_cmd.split()
                with open(f"{path_to_pdbs}/MSA_dir/{template[0:4]}_{pdbs[0:4]}.txt", "w") as msa_out:
                    result = run(bash_cmd_rdy, stdout=msa_out, stderr=PIPE, 
                        universal_newlines=True)
                
            except Exception as error:
                print(error)
    
    
    return min_seq_len

# Structure based alignment and cutting

In [153]:
def structure_based_cutting(path_to_pdbs:str, template:str, main_prot_seq:str):
    
    """This function will perform the structure based alignment based on the SUPERIMPOSER PDB.BIO class
        """
    
    onlyfiles = [f for f in os.listdir(path_to_pdbs) if os.path.isfile(os.path.join(path_to_pdbs, f))]
    
    pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
    #here we store our successful hits.
    try:
        new_directory_path = f"{path_to_pdbs}/PCA/clean_ensemble"
        os.makedirs(new_directory_path, exist_ok=True)
    except:
        print(f"we could not make the dir : {path_to_pdbs}/PCA/clean_ensemble")
    
    for pdb in pdbs:
        #we save one original one.
        
        pdb_parser = Bio.PDB.PDBParser(QUIET = True)
    
        sample_structure = pdb_parser.get_structure("sample", f"{path_to_pdbs}/{pdb}")
        
        ref_chain = [x.get_id() for x in sample_structure.get_chains()]
        
        io = Bio.PDB.PDBIO()
        
        io.set_structure(sample_structure)
        
        io.save(f"{path_to_pdbs}/PCA/original_{pdb[0:4]}_{''.join(ref_chain)}.pdb") #first save only in this dir.
        
        if pdb == template:
            template_return = f"original_{pdb[0:4]}_{''.join(ref_chain)}.pdb"
            
        
    #contains the new saved files in the new dir.
    onlyfiles1 = [f for f in os.listdir(f"{path_to_pdbs}/PCA") if os.path.isfile(os.path.join(f"{path_to_pdbs}/PCA", f))]
    
    pdbs1 = [f for f in onlyfiles1 if f[-4:] == ".pdb"]
    
       #lets check structure lengths.
    pdb_starts = []
    pdb_stops = []
    
    pdb_info_dict = defaultdict()
    
    for pdb in pdbs1:
        
        struc_start, struc_stop , sequence = select_c_alpha(f"{path_to_pdbs}/PCA/{pdb}")
        pdb_info_dict[pdb] = (struc_start, struc_stop, sequence)
        
        pdb_starts.append(struc_start)
        pdb_stops.append(struc_stop)
        
        
    #len_ref = select_c_alpha(f"{template}")  #template path is full.
 
    #pdb_lengths.append(len_ref) #we add it in the mix.
    
    
    cnts_start = Counter(pdb_starts)
    
    cnts_stop = Counter(pdb_stops)
    
    highest_occ_start = cnts_start.most_common()  #grabs the highest frequency start of structures
    highest_occ_stops = cnts_stop.most_common()  #grabs the highest frequency stop of structures
    
    print(highest_occ_start)
    print(highest_occ_stops)
    
    min_len = highest_occ_start[0][0] # this corresponds to the majority vote and its associated starts.
    max_len = highest_occ_stops[0][0] # this corresponds to the majority vote and its associated stops.
    
    print(min_len)
    print(max_len)
    
    #so here we did majority voting (1) boundaries are established.
    
    #for keys, vals in pdb_info_dict.items():
    #    print(keys, vals)    #val is: (start, stop, sequence(in full length!))
    
    solid_ensemble = [] #here we store good hits.
    
    to_be_checked = []
    #we need MAIN FASTA SEQ HERE
    
    #print(len(main_prot_seq))
    for pdb in pdbs1:
        
        """HERE WE WILL DO THE PAIRWISE ALIGNMENTS AGAINST THE PDB FASTA. PASS MAIN FASTA SEQ HERE AS ARGUMENT.
        WE WILL COMPARE ALL SEQ AGAINST THIS ONE."""
        
        seq_to_check = pdb_info_dict[pdb][2]

        aligned = pairwise2.align.globalxx(main_prot_seq, seq_to_check)
        
        best_alignment = max(aligned, key=lambda x: x.score)
        
        score = best_alignment.score
        print(pdb, len(seq_to_check) , score)
        
        #this means we have more than 80% identity on the sequence level aka it should be the same protein.
        if score > 0.8* len(seq_to_check):  
            solid_ensemble.append(pdb)
        else:
            to_be_checked.append(pdb)
            
        
        
    #print("this is solid ensemble")
    #print(solid_ensemble)
    #
    #print("this is to be checked")
    #print(to_be_checked)
    
    
    #now lets repair the solid ensemble.
    for pdb in solid_ensemble:
        
        struc_len = len(pdb_info_dict[pdb][2]) #this gives the length of the structure.
        
        structure_based_repair(path_to_pdb = f"{path_to_pdbs}/PCA", 
                               cutoff=10,
                               pdb_id=pdb,
                               min_len=min_len,
                               max_len=max_len,
                               structural_length = struc_len,
                               main_prot_seq=main_prot_seq,
                               use_main=True)  #we grab the respective sequence.
        
        #select_c_alpha(f"/home/micnag/result_test_struc_align/{pdb}")
    
    
    solid_ensemble = [f for f in os.listdir(f"{path_to_pdbs}/PCA/clean_ensemble") if os.path.isfile(os.path.join(f"{path_to_pdbs}/PCA/clean_ensemble", f))]
    
    for x in solid_ensemble:
        print(x)
    
    
    #solidify ensemble by removing strong outliers.
    
    lst_ground_truth = []
    for pdb in solid_ensemble:
        lst_ground_truth.append(f"{path_to_pdbs}/PCA/clean_ensemble/{pdb}")
    
    #lets check the RMSD WITHIN the best hits to remove strong outliers.
    
    #lets return the average rmsd and use this as criterion! NEEDS TO BE DONE ATFER LUNCH
    off_strucs, tot_mean, tot_sd = _calc_mean_tresholds_lst(lst_ground_truth)
    print(off_strucs)
    
    #remove off strucs. now compute mean rmsd against the ground truth assumed ensemble.
    
    
    try:
        os.mkdir(f"{path_to_pdbs}/PCA/repaired_ensemble")
    
    except:
        print(f"we could not make the dir : {path_to_pdbs}/PCA/repaired_ensemble")
    
    
    
    #this will land in the new dir repaired_ensemble
    for pdb in to_be_checked:
        
        struc_len = len(pdb_info_dict[pdb][2])
        #first check if needs to be repaired.
        structure_based_repair(path_to_pdb = f"{path_to_pdbs}/PCA", 
                               cutoff=10,
                               pdb_id=pdb,
                               min_len=min_len,
                               max_len=max_len,
                               structural_length = struc_len,
                               main_prot_seq=main_prot_seq,
                               use_main=False)
        
    #after repair we check rmsd.
    
    lst_to_check = [f for f in os.listdir(f"{path_to_pdbs}/PCA/repaired_ensemble") if os.path.isfile(os.path.join(f"{path_to_pdbs}/PCA/repaired_ensemble", f))]
    
    print("we check now rmsd for repaired alternative strucs")
    
    for pdb in lst_to_check:

        try:
            rmsd_avg = _calc_rmsd_dev(lst_to_pdbs=lst_ground_truth,
                                  pdb_to_check=f"{path_to_pdbs}/PCA/repaired_ensemble/{pdb}")
        
            #if rmsd is low we add it to the ensemble.
            if rmsd_avg < (tot_mean * 2 * tot_sd):
                print(f"we append {pdb} with rmsd avg: {rmsd_avg}, against {tot_mean * 2 * tot_sd} as threshold.")
                solid_ensemble.append(pdb)
                #we move it to another dir.
                shutil.copy(f"{path_to_pdbs}/PCA/repaired_ensemble/{pdb}", 
                        f"{path_to_pdbs}/PCA/clean_ensemble/{pdb}")
            
            else:
                print(f"we reject {pdb} with rmsd avg: {rmsd_avg}, against {tot_mean * 2 * tot_sd} as threshold.")
                os.remove(f"{path_to_pdbs}/repaired_ensemble/{pdb}")
        
        except Exception as error:
            print(error)
            
    
    return template_return
    #return pdbs to be fetched from the clean ensemble to another function.


In [57]:
def pca_laura_pipeline(path:str, template:str, protein:str):
    
    
    work_dir = path

    os.chdir(work_dir)
    
    pdb_files = [f for f in os.listdir(f"{path}/clean_ensemble") if os.path.isfile(os.path.join(f"{path}/clean_ensemble", f))]
    
    
    #store pdb ids for later identification.
    with open(f"{work_dir}/ensemble.txt", "w") as esmbl:
        for entries in pdb_files:
            esmbl.write(entries[9:-4])
            esmbl.write("\n")
    
    
    #we write a multi pdb file
    ms = Bio.PDB.Structure.Structure("master")

    i = 0

    hit_list = []
    for files in pdb_files:
        hit_list.append(files[9:-4])
        location = path +"/"+files
    
        parser = PDBParser(QUIET=True)
    
        prot_name = f"{files[0:-4]}"
        structure = parser.get_structure(prot_name, location)
        for model in structure:
            new_model = model.copy()
            new_model.id = i
            new_model.serial_num=i+1
            i += 1
            ms.add(new_model)
        
    
    pdb_io = Bio.PDB.PDBIO()
    pdb_io.set_structure(ms)
    #print("we saved")
    pdb_io.save(f"{work_dir}/multi_ensemble.pdb")
    
    #now we need to call lauras pipeline
    #make new dir and shuffle all required .exe and scripts to it.
    
    basepath = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/PCA_pipeline"

    try:
        
        #move executables to this new location.
        
        files_to_move = [f for f in os.listdir(basepath) if os.path.isfile(os.path.join(basepath, f))]
        
        for files in files_to_move:
            
            shutil.copy(f"{basepath}/{files}" , f"{work_dir}")
        
    except Exception as error:
        print(error)
    
    
    ##continue HERE TOMORROW
    
    #now we get_pca
    #part 1 pca
    
    bash_cmd = f"./getpca.sh multi_ensemble.pdb {template} refpdb_ref"
    
    #print(bash_cmd)
    os.chdir(work_dir)
    
    #print(os.getcwd())
    
    bash_cmd_rdy = bash_cmd.split()
    
    try:
        #print("we start part 1")
        result = run(bash_cmd_rdy, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            
        #print(result.stdout)
        #print(result.stderr)
        #print("part 1 worked")
    except:
        print(f"Script did not work. Here are the supplied parameters \n{bash_cmd_rdy}")
        
    #now we convert the evec files
    
    
    bash_cmd1 = f"./convert.sh refpdb_ref_pca.evec"
    
    bash_cmd1_rdy = bash_cmd1.split()
    
    #print("this is bashcmd1")
    #print(bash_cmd1_rdy)
    
    try:
        #print("we start part 2")
        result = run(bash_cmd1_rdy, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            
        #print(result.stdout)
        #print(result.stderr)
        #print("part 2 worked")
    except:
        print(f"Script did not work. Here are the supplied parameters \n{bash_cmd1_rdy}")
        
    #now to the projections.
    
    bash_cmd2 = f"./getproj.sh {template} refpdb_ref_pca.evec {protein} multi_ensemble.pdb"
    
    bash_cmd2_rdy = bash_cmd2.split()
    
    try:
        
        #print("we start part 3")
        result = run(bash_cmd2_rdy, stdout=PIPE, stderr=PIPE, universal_newlines=True)
            
        #print(result.stdout)
        #print(result.stderr)
        #print("part 3 worked")
    except:
        print(f"Script did not work. Here are the supplied parameters \n{bash_cmd2_rdy}")
        
        
    #work dir = 
    #protein = name of protein according to UNIPROT
    #i = num of structures in the multipdb. 
    return work_dir, protein, i

In [58]:
def _plot_PCA_new(input_dir:str, protein:str, num_struc:int):
    
    proj_dict = defaultdict()
    
    labels = []
    with open(f"{input_dir}/ensemble.txt", "r") as label_in:
        for lines in label_in:
            lines = lines.replace("\n","")
            labels.append(lines[-6:])
    
    
    var_list = []
    
    #they are already sorted.
    with open(f"{input_dir}/{protein}_evec3.dat", "r") as var_readin:
        for lines in var_readin:
            line = lines.split()
            if len(line) == 2:
                var_list.append(float(line[1]))

    expl_var = var_list / np.sum(var_list)
    
    with open(f"{input_dir}/{protein}.mode_12.proj", "r") as res_proj:
        for lines in res_proj:
            lines = lines.replace("\n","")
            #print(lines.split("      "))  #6 spaces ??? 
            lines = lines.split("      ")
            num = lines[1].replace(" ","")
            
            PC1 = float(lines[2].replace(" ",""))
            PC2 = float(lines[3].replace(" ",""))
            
            proj_dict[str(num)] = (PC1, PC2)
            
    
            
    PC_1 = []
    PC_2 = []
    for keys, vals in proj_dict.items():
        #print(vals)
        PC_1.append(vals[0])
        PC_2.append(vals[1])
    
    
    plot_df = pd.DataFrame(columns=["PC1", "PC2"])
    plot_df["PC1"] = PC_1
    plot_df["PC1"] = plot_df["PC1"] * (-1)
    plot_df["PC2"] = PC_2
    plot_df["labels"] = labels
    #michael9_test_df["PC2"] = michael9_test_df["PC2"] * (-1)
    #michael9_test_df["labels"] = labels
    
    fig = px.scatter(plot_df, x= "PC1",y= "PC2",
                hover_data=["labels"],labels={"labels": "labels"},
                custom_data = ["labels"])

    fig.update_traces(marker_size=10,
                    hovertemplate="<b>RCSB: %{customdata[0]}</b>",
                    hoverlabel=dict(
                    font_size=40),  # Set the hover label font size
                        
                        
                    mode="markers+text", selector=dict(type='scatter')    
                        
        )
    
    fig.update_layout(plot_bgcolor='lavender',
                      paper_bgcolor='white', 
                     width = 800,
                     height = 600,
                     xaxis_title=f'PC 1 {expl_var[0]*100:.2f}%',  # Set the x-axis label
                     yaxis_title=f'PC 2 {expl_var[1]*100:.2f}%',   # Set the y-axis label
                     legend_title_text='Clusters',  # Set the legend title
                     showlegend=True,  # Show the legend
                     title={
                     'text': f'PCA of {protein}: {num_struc} structures',  # Set the title text
                     'x': 0.5,  # Center the title horizontally (0 to 1)
                     'font': {'size': 30}  # Customize title font size and weight
                     }
                      
                     )
    
    
    pio.write_image(fig, f"{input_dir}/{protein}_PC_plot.png", format="png")
    
    #return fig.show()
    

In [59]:
#input_file = "/home/micnag/result_test_struc_align/PCA/SERCA.mode_12.proj"
#labels_file = "/home/micnag/result_test_struc_align/PCA/ensemble.txt"
#var_file = "/home/micnag/result_test_struc_align/PCA/SERCA_evec3.dat"
#protein = "SERCA"
#num_struc = 86

#_plot_PCA(input_file=input_file, 
#         labels_file=labels_file,
#          explained_var_file=var_file,
#         protein=protein,
#         num_struc=num_struc)

In [60]:
#path = "/home/micnag/result_test_struc_align/clean_ensemble"
#template = "/home/micnag/result_test_struc_align/clean_ensemble/original_2c9m_A.pdb"

#pca_laura_pipeline(path=path, template=template)

In [61]:
##### os.chdir("/home/micnag/") #just because there is an error is we stuck within that makes problems
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994"
#path = "/home/micnag/result_test_struc_align"
#query = "O14983"
#main_prot_seq = get_gene_fasta(query)
#template = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/2c9m.pdb"
#structure_based_cutting(path_to_pdbs=path, template=template, main_prot_seq=main_prot_seq, query=query)

In [62]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/monomer/pos_1_181"
#template ="/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/monomer/pos_1_181/2duk.pdb"

#structure_based_cutting(path_to_pdbs=path, template=template)

In [63]:
def _select_proper_length(path:str,start:int, stop:int, use_main:bool):
    
     #sel only c_alpha
    class range_only(Select):
        def __init__(self, *args):
            super().__init__(*args)
        
        def accept_atom(self, atom):
            return 1 if atom.id == "CA" else 0
        #overload accept_residue inherited from Select with this conditional return
        def accept_residue(self, residue):
            return 1 if residue.id[1] >= start and residue.id[1] <= stop else 0
        
    
    #filelst    path
    #5ltu_A.pdb /home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN
    
    #now we load it again
    parser = PDBParser(QUIET=True)
    
    prot_name = f"default"
    
    structure = parser.get_structure(prot_name, path)
    
    structure_len = structure.get_residues()
    
    struc_len = [x.get_id()[1] for x in structure_len]
    
    #this is start
    #1
    #this is stop
    #994
    #/home/micnag/result_test_struc_align/original_4kyt_A.pdb
    #992 994
    
    
    
    if use_main == False:    
        
        start = struc_len[0]
        stop = stop + start -1 # this corresponds to correcting for the overhang!
        
    print(f"we set as start: {start} and as stop: {stop}")
    
    if len(struc_len) < (stop-start)+1:
        
        #print(path)
        #print(struc_len,stop-start+1)
        #print(struc_len)
        #we remove it then.
        print(f"we remove {path}")

        #os.remove(path)
        
        return False
        
    else:
        
        io = PDBIO()
            
        io.set_structure(structure)
    
        #savepath needs to be changed.
        io.save(path, range_only())

In [64]:
#path = "/home/micnag/result_test_struc_align/original_5zmv_A.pdb"
#_select_proper_length(path=path, stop=994)


In [65]:
path_to_pdb_test = "/home/micnag/result_test_struc_align"


def structure_based_repair(path_to_pdb:str, pdb_id:str, min_len:int,
                          max_len:int, 
                          structural_length:int, 
                          use_main:bool,
                          main_prot_seq:str,
                          cutoff=10):
    
    print("we check gaps now")
    print(f"{path_to_pdb}/{pdb_id}")
    gaps = _gap_localization(f"{path_to_pdb}/{pdb_id}")
    
    print(gaps)
    if len(gaps) != 0:  #also repair if the structural length is less than the max len (might be missing a few res)
        
        #this dict contains as values : paths to potential repairing strucs and as [-1] entry the AVG RMSD OVER ALL TEMPLATES over all SAMPLE ALIGNMENTS
        #gap_dict = _gap_alignment(path_to_pdbs, gaps)
            
        #print(f"{path_to_pdbs}/{pdb}", gap_dict)
        
        #we repair all that have less than 10 gaps.
        gap_ranges = [y-x for x, y in gaps if (y < max_len and x > min_len)]  #basically only coiunt gaps between 1-994.not e.g 994-1100 because here we shall reconstruct and select only 1-944
        
        #print(gap_ranges)
        gap_sorted = sorted(gap_ranges, reverse=True) #largest first
        
        print(gap_sorted)
        if len(gap_sorted) == 0 or gap_sorted[0] <= 10 :
            print(f"pdb is : {pdb_id}, use_main : {use_main}")
            
            try:
            
                mini_repair_residues(path_to_pdb=f"{path_to_pdb}/{pdb_id}",
                            stop_pos=max_len, main_prot_seq=main_prot_seq, use_main=use_main)
            
            except:
                
                print(f"we remove {path_to_pdb}/{pdb_id}")
                os.remove(f"{path_to_pdb}/{pdb_id}")
                return
                
                
        else:
            #we thrash the structure.
            print(f"we remove {path_to_pdb}/{pdb_id}")
            os.remove(f"{path_to_pdb}/{pdb_id}")
            #and return back.
            return
        
        
    elif structural_length < max_len:      
        
        try:
            mini_repair_residues(path_to_pdb=f"{path_to_pdb}/{pdb_id}",
                            stop_pos=max_len, main_prot_seq=main_prot_seq, use_main=use_main)
        except:
            
            print(f"we remove {path_to_pdb}/{pdb_id}")
            os.remove(f"{path_to_pdb}/{pdb_id}")
            return
        
        
    #now we need to strip c_alpha and align again.
    
    print(f"we move now to selecting the proper length for {pdb_id}")
    
    keep_struc = _select_proper_length(path=f"{path_to_pdb}/{pdb_id}",
                         start=min_len,
                         stop=max_len,
                         use_main=use_main)
    
    
    #if we deleted it we dont need to continue
    if keep_struc == False:
        print("we removed the struc so we shall return!")
        return
    
    non_canonical = remove_hetero_atoms(path=f"{path_to_pdb}",
                           pdb_file=pdb_id)
    
    print(non_canonical)
    
    if len(non_canonical) != 0:
        for keys, vals in non_canonical.items():
            _mutate_non_standard_aa(path_to_pdb=f"{path_to_pdb}/{pdb_id}",
                               non_standard_residue=vals[0],
                                   residue=keys,
                                chain=vals[1])
    
    #here we shuffle everthing that worked properly into new dir.
    if use_main:
        print(f"we copy from {path_to_pdb}/{pdb_id} to : {path_to_pdb}/clean_ensemble/{pdb_id}")
        shutil.copy(f"{path_to_pdb}/{pdb_id}", f"{path_to_pdb}/clean_ensemble/{pdb_id}")
    else:
        print(f"we copy from {path_to_pdb}/{pdb_id} to : {path_to_pdb}/repaired_ensemble/{pdb_id}")
        shutil.copy(f"{path_to_pdb}/{pdb_id}", f"{path_to_pdb}/repaired_ensemble/{pdb_id}")
    
    
#structure_based_repair(path_to_pdb_test)pdb_info_dict

In [66]:
#os.chdir("/home/micnag/bioinformatics")

In [67]:
def get_rmsd(pdb_path:str, template:str):
    
    aligner = Bio.PDB.Superimposer()
    
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)
        
    sample_structure = pdb_parser.get_structure("sample", pdb_path)
            
    sample_model = sample_structure.get_atoms()
            
    sample_model_list = [x for x in sample_model]
            
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)
        
    template_structure = pdb_parser.get_structure("template", template)
            
    template_model = template_structure.get_atoms()
            
    template_model_list = [x for x in template_model]
    
    aligner.set_atoms(template_model_list, sample_model_list)
        
    aligner.apply(sample_model_list)
    
    return aligner.rms
    

In [68]:
def _calc_rmsd_dev(lst_to_pdbs:list, pdb_to_check:str):
    
    avg_rms = []
    
    for pdb in lst_to_pdbs:
        #'/home/micnag/result_test_struc_align/original_2o9j_A.pdb'
        res = get_rmsd(pdb_path=pdb, template=pdb_to_check)
        
        avg_rms.append(res)
    
    
    print(pdb_to_check,avg_rms)
    #lets return the mean rmsd
    return np.mean(avg_rms)

In [69]:
def _calc_mean_tresholds_lst(lst_to_pdbs:list):
    
    avg_rms = defaultdict(list)
    
    #each against each
    for temp in lst_to_pdbs:
        #we compute the average rmsd for each struc against all others.
        for pdb in lst_to_pdbs:
            res = get_rmsd(pdb_path=pdb, 
                  template=temp)
        
            avg_rms[pdb].append(res)
        

    total_means = []

    tot_mean_dict = defaultdict()
    for keys, vals in avg_rms.items():
        print(keys)
    
        tot = 0
        for v in vals:
            tot += v
    
        total_means.append(tot/len(vals))
        print(tot/len(vals))
        
        tot_mean_dict[keys] = tot/len(vals) 
        
    tot_mean = np.mean(total_means)
    tot_sd = np.std(total_means)
    
    off_strucs = [(keys, vals) for keys, vals in tot_mean_dict.items() if vals > tot_mean + 2*tot_sd]
    
    print(off_strucs)
    return (off_strucs, tot_mean, tot_sd)

In [70]:
path_to_pdb_test = "/home/micnag/result_test_struc_align"

def _calc_mean_tresholds(path_to_pdbs:str):
    
    onlyfiles = [f for f in os.listdir(path_to_pdbs) if os.path.isfile(os.path.join(path_to_pdbs, f))]

    avg_rms = defaultdict(list)
    
    #each against each
    for temp in onlyfiles:
        #we compute the average rmsd for each struc against all others.
        for pdb in onlyfiles:
            res = get_rmsd(pdb_path=f"{path_to_pdbs}/{pdb}", 
                  template=f"{path_to_pdbs}/{temp}")
        
            avg_rms[pdb].append(res)
        

    total_means = []

    tot_mean_dict = defaultdict()
    for keys, vals in avg_rms.items():
        print(keys)
    
        tot = 0
        for v in vals:
            tot += v
    
        total_means.append(tot/len(vals))
        print(tot/len(vals))
        
        tot_mean_dict[keys] = tot/len(vals) 
        
    tot_mean = np.mean(total_means)
    tot_sd = np.std(total_means)
    
    off_strucs = [(keys, vals) for keys, vals in tot_mean_dict.items() if vals > tot_mean + 2*tot_sd]
    print(off_strucs)

In [71]:
#_calc_mean_tresholds(path_to_pdb_test)

In [72]:
def _gap_localization(pdb_path:str):
    
    """Helper function to compute the start and stops of gaps for later potential reconstruction."""
    
    gaps = []
    
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)
    
    test_structure = pdb_parser.get_structure("test", pdb_path)
    
    test_res = test_structure.get_residues()
    
    i = 1
    for res in test_res:
        
        if i != int(res.get_id()[1]):
            
            j = i
            i = res.get_id()[1] + 1
            
            gap = [x for x in range(j,i-1)] #start until finish (inclusive last gap pos)
            
            #if there is no gap we continue.
            if len(gap) == 0:
                continue
            
            
            start, stop = gap[0], gap[-1]
            
            gaps.append((start, stop))
        
        i += 1
    
    
    return gaps

In [73]:
def _structures_to_select_as_reference(structure_pdbs:str, gaps:list):
    
    np.random.seed(7) #lets make it random. we dont want to parse the whole ensemble.
    
    #either 10% of ensemble as template or min 5:
    
    
    onlyfiles = [f for f in os.listdir(structure_pdbs) if os.path.isfile(os.path.join(structure_pdbs, f))]
    
    pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
    struc_templates = []
    
    rng_num = np.unique(np.random.randint(0, len(structure_pdbs)-1, 3*len(onlyfiles)))  #we draw 60 samples and keep only uniques
    
    i = 0 #we start with first member of rng_num
    
    while len(struc_templates) < 4:
        
        #ifonlyfilesexhausted all potential templates we shall return None
        if i == len(onlyfiles):
            return None
        
        pdb_parser = Bio.PDB.PDBParser(QUIET = True)
            
        struc_chosen = rng_num[i]
        
        test_structure = pdb_parser.get_structure("test", f"{structure_pdbs}/{pdbs[struc_chosen]}")  #this choses the structure that has idx i from rng_num
        
        test_res = test_structure.get_residues()
                                    
        max_len = [x for x in test_res] #just for max len purpose.
        
        #need to reset generator!!!
        test_res = test_structure.get_residues()
        
        
        ext_gap_upper = gaps[1] + 5 if gaps[1] + 5 < len(max_len) else len(max_len) # if it goes beyong possible we stick with end range.
        ext_gap_lower = gaps[0] - 5 if gaps[0] - 5 > 0 else gaps[0]  #min is pos 1. we cant go negative
        
        res_id_lst = [x.get_id()[1] for x in test_res if x.get_id()[1] >= ext_gap_lower and x.get_id()[1] <= ext_gap_upper] # +5 as buffer
        
        diff_required = ext_gap_upper - ext_gap_lower + 1
        
        
        # this is borders ext:
        #512 500
        #this is required
        #13

        #print("this is borders ext:")
        #print(ext_gap_upper, ext_gap_lower)
        #print("this is required")
        #print(diff_required)
        
        #print(gaps[1]+5 - gaps[0]- 5 + 1)
        #implement that we grab CA_IDX GAPS[0] to GAPS[1] instead of idx of atomlist (which can have gaps and then is shifted)
        
        if len(res_id_lst) == (diff_required):
            struc_templates.append(f"{structure_pdbs}/{pdbs[struc_chosen]}")
            
        #print(len(struc_templates))
        i += 1
        #reset if we get all templates for struc 1
        if len(struc_templates) == 3:
            i = 0
        
        
    #these we use as templates.    
    return struc_templates

In [74]:
def _gap_alignment(structure_pdbs:str, gaps:list):
    
    """This function will make an alignment locally around the gaps and compute RMSD.
    IF LOW RMSD: we seal the gap.
    IF HIGH RMSD: we cut the gap from all structures"""
    
    #structures are all aligned already. now we should do a local small alignment and then compute rmsd.
    
    gap_dict = defaultdict()
    
    for gap in gaps:
    
        ref_strucs = _structures_to_select_as_reference(structure_pdbs, gap)
    
        onlyfiles = [f for f in os.listdir(structure_pdbs) if os.path.isfile(os.path.join(structure_pdbs, f))]
    
        pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
        
        for j, template in enumerate(ref_strucs):
            
            #here we store for each template the rmsd
            template_rmsd = 0
            
            #print(template)
            
            pdb_parser = Bio.PDB.PDBParser(QUIET = True)
            
        
            template_structure = pdb_parser.get_structure("template", f"{template}")
            
            template_model = template_structure.get_atoms()
            
            ext_gap_upper = gap[1] + 5
            ext_gap_lower = gap[0] - 5
            
            
            template_atoms = [x for x in template_model if x.get_full_id()[3][1] >= ext_gap_lower 
                              and x.get_full_id()[3][1] <= ext_gap_upper]
            
            template_ids = [x.get_full_id()[3][1] for x in template_atoms]
            
            
            #print(template_atoms)
            #print(template_ids)
            
            
            template_model_list = [x for x in template_model]  #select atoms from start to finish but they have + 5 on both ends.
            
            #nested loop.
            
            #first gap all samples
            for i, samples in enumerate(pdbs):
                
                pdb_parser = Bio.PDB.PDBParser(QUIET = True)
            
                aligner = Bio.PDB.Superimposer()
        
                sample_structure = pdb_parser.get_structure("sample", f"{structure_pdbs}/{samples}")
            
                sample_model = sample_structure.get_atoms()
                
                sample_atoms = [x for x in sample_model if x.get_full_id()[3][1] >= ext_gap_lower 
                        and x.get_full_id()[3][1] <= ext_gap_upper]
                
                if len(sample_atoms) == len(template_atoms):
                    
                    #if they are the same we proceed.
                    
                    #we start procedure of alignment if this is done.
                    
                    aligner.set_atoms(fixed=template_atoms, moving=sample_atoms)
                    
                    aligner.apply(sample_atoms)
                    #print(template, samples)
                    #print(aligner.rms)
            
                    template_rmsd += aligner.rms
        #print(((template_rmsd/i))/j)
                    
                    
        #outer most we compute the average rmsd between all template and all structs.
        #j means we average over all templates.
        #ref strucs 0 is just the first suitable template.
        gap_dict[gap] = (template_rmsd/i)/j
        
        
    #print(gap_dict)
    return gap_dict

In [75]:
def mini_repair_residues(path_to_pdb:str, stop_pos:int, main_prot_seq:str, use_main:bool):
    """Function should take only those proteins that have
    a) gaps with less than 7 residues missing per gap.
    
    Input:
    path to pdb_folder
    gap_dict file
    
    Output:
    produces repaired structures
    """

    #onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    #pdbs = [corr_file for corr_file in onlyfiles if corr_file[-4:] == ".pdb"]
    
    log.none() #no stdout spam
    env = Environ() #setup env for modelling
    
    aln = Alignment(env) # setup alignment within the env.
    
    mdl = Model(env)  #setup the model object where we will append our pdbs into.
     
    current_pth = os.getcwd()
    
    pdb_id_target = path_to_pdb.split("/")
    #print("we are inside mini repair")
    #print(pdb_basep)

    #['', 'home', 'micnag', 'result_test_struc_align', 'original_7e7s_A.pdb']
    #print(pdb_basep)
    #/home/micnag/result_test_struc_align
    #print(pdb_id_target)
    #7e7s_A
    #print(pdb_id_chain)
    #A
    
    pdb_basep = pdb_id_target[0:-1]
    pdb_basep = "/".join(pdb_basep)
    # we need this for downstream split.
    pdb_id_target = pdb_id_target[-1][9:-4]
    
    pdb_id_chain = pdb_id_target[5]
    
    
    #this is the new version which grabs correct seq per chain.
    
    #if original ensemble: we fetch the right uniprot sequence else we fetch whatever other homolog protein it is.
    
    print(f"we have used main as: {use_main} and pdb target is : {pdb_id_target}")
    
    if use_main:
        #either we grab this!
        fasta_seq = main_prot_seq
        
        print(fasta_seq)
        print(len(fasta_seq))
    
    else:
        #or we grab this if its not the protein itself but another homolog.
        gene_name_id, gene_id = get_gene_name_uniprot(f"{pdb_id_target[0:4]}")
        
        print(gene_name_id, gene_id)
        
        gene_name = get_gene_fasta(gene_id) #gives back the gene_name
        
        print(gene_name)
        
        fasta_seq = get_gene_fasta(gene_name) #this should work.
        
    
    
    print(fasta_seq)
    
    
    
    #if len(fasta_seq) < stop_pos:
    #    #we dont need to progress than because the resulting structure will be shorter than the required one.
    #    return #we cant progress anyfurther then.
    
    try:
        #print("we make the dir:")
        #print(f"{pdb_basep}/{pdb_id_target}")
        os.mkdir(f"{pdb_basep}/{pdb_id_target}")
        
        #print("we made it")
        #print(f"{path}/{pdb_id_target}")
        #print(f"{path}/{pdb_id_target}/{pdb_id_target}")
        #print("we shuffle from:")
        #print(f"{pdb_basep}/{pdb_id_target}.pdb")
        
        #print("towards:")
        #print(f"{pdb_basep}/{pdb_id_target}/{pdb_id_target}.pdb")
        
        shutil.copy(f"{pdb_basep}/original_{pdb_id_target}.pdb", f"{pdb_basep}/{pdb_id_target}/{pdb_id_target}.pdb")
    except:
        print("already there")
    
    os.chdir(f"{pdb_basep}/{pdb_id_target}")
    print(os.getcwd())
    
    
    #start stop grab:
    
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)
        
    #sample struc
    sample_structure = pdb_parser.get_structure("sample", f"{pdb_basep}/{pdb_id_target}/{pdb_id_target}.pdb")
    
    sample_res = sample_structure.get_residues()
    
    sample_list = [x.get_id()[1] for x in sample_res]
    
    start = sample_list[0] #first residue.
    stop = len(fasta_seq) #max amount of fasta seq.
    
    print(start, stop)
    
    env.io.atom_files_directory = ['.','../.']

    #write fasta
    #grab start + end pos.
    
    #start, stop, chain = _start_stop_fasta(pdb_id_target=pdb_id_target, 
    #                                                   path=f"{pdb_basep}/{pdb_id_target}")
    
    code = f"{pdb_id_target}"
    
    #print("this will be start and stop")
    #print(start, stop)
    
    
    mdl.read(file=code, model_segment=(f"{start}:{pdb_id_chain}", f"{stop}:{pdb_id_chain}"))
    
    aln.append_model(mdl, align_codes=code, atom_files=code)
    
    #print("this is pdb_id_target and chain")
    #print(f"{pdb_id_target[0:4]}", chain)
    
    #this is the old version which grabs full fasta.
    #fasta_seq = get_gene_fasta_from_pdb_id(f"{pdb_id_target[0:4]}", chain=chain)
    
    #print("this is fasta seq")
    #print(fasta_seq)
    
    with open(f"./{pdb_id_target}x.fasta", "w") as fastaout:
        fastaout.write(f">{pdb_id_target}x\n")
        fastaout.write(fasta_seq)
    
    
    aln_code = f"{pdb_id_target}x"
    
    aln.append(file=f"./{pdb_id_target}x.fasta", align_codes=aln_code, 
               alignment_format="fasta")
    
    aln.salign(overhang=30, gap_penalties_1d=(-450, -50),
    alignment_type="tree", output="ALIGNMENT")
    
    ##aln.malign(gap_penalties_1d=(-500, -300))
    ##aln.malign3d(gap_penalties_3d=(0.0, 2.0))
    aln.write(file=f"{pdb_id_target}.ali")

    #lets model
    
    #we will fish one out of the pdb list and align it against all others.
    
    a = AutoModel(env,
                  alnfile = f"{pdb_id_target}.ali",
                  knowns = f"{pdb_id_target}",
                  sequence = aln_code)
    
    a.starting_model = 1
    a.ending_model = 1
    a.make()
    
    os.chdir(current_pth)
    
    #If successful:
    
    pdbs = []
    onlyfiles = [f for f in os.listdir(f"{pdb_basep}/{pdb_id_target}") if os.path.isfile(os.path.join(f"{pdb_basep}/{pdb_id_target}", f)) and f[-4:] == ".pdb"]
    pdbs_sorted = sorted(onlyfiles, key=len)
    
    #print(pdbs_sorted)
    #the old one needs to be removed.
    #print("this is the remove path")
    #print(f"{path}/{pdb_id_target}/{pdbs_sorted[0]}")
    #os.remove(f"{path}/{pdb_id_target}/{pdbs_sorted[0]}")
    #the new one needs to be renamed
    shutil.copy(f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[1]}",f"{pdb_basep}/{pdb_id_target}/original_{pdbs_sorted[1]}")
    os.rename(f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[1]}", f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[0]}")
    
    #If rebuilt, we need to RENUMBER AGAIN (TAKE CORRECT SHIFT INTO ACCOUNT)
    #now move the result back to the original location and overwrite the gap containing struc.
    #os.remove(f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[0]}")
    #print(f"we removed {path}/{pdbs_sorted[0]}")
    
    #DONT RENUMBER, but instead CUT OFF according to start stop. RENUMBERING NOT needed because FASTA is already providing 1-END numbering:
    
    
    #no longer needed here
    #start, stop = _new_cut_nums(start=start, stop=stop, gap_dict=gap_dict) 
    
    
    #_cut_proper_models(path_to_struc=f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[0]}",
    #               start_num = 1, end_num = min_length,
    #                   chain=chain)
    
    #_renumber_struc(path_to_struc=f"{path}/{pdb_id_target}/{pdbs_sorted[0]}",
    #               start_num = start, chain=chain)
    
    shutil.copy(f"{pdb_basep}/{pdb_id_target}/{pdbs_sorted[0]}",f"{pdb_basep}/original_{pdbs_sorted[0]}")
    #print(f"we moved {path}/{pdbs_sorted[0]}/{pdbs_sorted[0]} to {path}/{pdbs_sorted[0]}")
    #back to normal working dir
    
    #uncomment if DONE
    
    os.chdir(current_pth)
    
    shutil.rmtree(f"{pdb_basep}/{pdb_id_target}")
    
    #os.rename and maybe copy back to original file dest.
    
    #else. ignore and continue.
    

In [76]:
def _new_cut_nums(start:int, stop:int, gap_dict:dict):
    
    for keys, vals in gap_dict.items():
        start_n = keys[0]
        stop_n = keys[1]
        
        if start_n < start and not start_n < 0:
            start = start_n
        
        if stop_n > stop:
            stop = stop_n
        
        #in case its negative. we make it 1.
        if start < 0:
            start = 1
    
    print(start, stop)
    return start, stop

In [77]:
#path_to_pdb_test = "/home/micnag/result_test_struc_align"


#gaps = _gap_localization("/home/micnag/result_test_struc_align/2oa0.pdb_aligned.pdb")
#_gap_alignment(path_to_pdb_test, gaps)

In [78]:
#path_to_pdbs = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/PCA"
#template = f"{path_to_pdbs}/3w5b_A_CA.pdb"

#path_to_pdbs = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/"
#template = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/2c88.pdb"

#structure_based_cutting(path_to_pdbs=path_to_pdbs, 
#                       template=template)




# msa_file_readin

In [79]:
def read_msa_file_version_1(path:str):
    
    """Here we need to fetch information from N oligomer alignment files each containing info about 1 structure
    being aligned towards the template structure."""
    
    msa_dir = f"{path}/MSA_dir"
    
    only_msa = [f for f in os.listdir(msa_dir) if os.path.isfile(os.path.join(msa_dir, f))]
    
    #with this version, we need to analyze n - 1 alignment files for n structures with a designated template structure.
    
    seq_dict = defaultdict(list)
    
    for files in only_msa:
        
        headers = []
        seqs = []
        
        #each files needs to be read out.
        #each file contains fluffer + 2 sequences. ALL chains against all chains. First seq is always template.
        
        seq = False
        
        with open(f"{msa_dir}/{files}", "r") as msa: 
            i = 0
            for lines in msa:
                if lines[0:4] == "Name":
            
                    header_line = lines.split("/")  #split at whitespace 
                    pdb_code = header_line[-1][0:4]
                    
                    #this looks good
                    
                    headers.append(pdb_code)
                    
                if lines[0:4] == '(":"':
                    seq = True
                    continue
                #print("this is apparent correct line")
                #print(correct_lines)
                
                if seq:
                    
                    lines = lines.replace("*","")
                    sequence = lines.replace("\n","")
                    
                    tot_seq = len(sequence)
                    removed_gap = sequence.replace("-","")
                    remov_seq = len(removed_gap)
                    
                    seqs.append(sequence)
                    
               
            #print(headers)
            #print(seqs)
            #ugly solution but only 0 and 2 are seq rest is filler. 2 will be the template 0 will be the structure thats superimposed onto
            #template
            seqs = [seqs[0], seqs[2]]
            
            #print(headers)
            #print(seqs)
            
            for pdb_codes, seqs in zip(headers, seqs):
                #print(pdb_codes, seqs)
                seq_dict[pdb_codes].append(seqs)
                
            
            
    #print(seq_dict.keys())
    

    #seq dict will contain n times template seq and n-1 other sequences.
    #we will use this dict to cut out proper positions.
    
    
    '''check all gap positions.'''
    
    gaps_redundant = []
    deletion_list = []

    
    for pdbs, seqs in seq_dict.items():
        i = 0   #we count through the whole seq but only add gaps to the gaps_redundant list.
        #print(pdbs, seqs)
        print(len(seqs))
        if len(seqs) > 1:  #thats our template. because all others will have 1 and seqs will have n seqs (corresponding to n comparisons
            #between template and pdb)
            #for seqlst in seqs:
            #    seqlst_without_gaps = seqlst.replace("-","")
            #    
            #    gaplen = len(seqlst)-len(seqlst_without_gaps)
            #    if gaplen > 0.01 * len(seqlst):
            #        
            #        #we remove the structure:
            #        #print(pdbs)
            #        #os.remove()
            #        continue
            #    i = 1
            #    for seq in seqlst:
            #        if seq == "-":
            #            if i not in gaps_redundant:
            #                gaps_redundant.append(i)
            #            
            #            i += 1
            #            continue
            #        else:
            #            i += 1
            continue
                    
        else:
            #print(seqs)
            for seqlst in seqs:
                
                gapless = seqlst.replace("-","")
                withgap = len(seqlst)
                withoutgap = len(gapless)
                print(withgap, withoutgap)
                if (withgap - withoutgap) > 0.10 * withoutgap: #here we can experiment how strict we want to be.
                    print("this is too much gap")
                    print(pdbs)
                    print(withgap, withoutgap)
                    deletion_list.append(pdbs)
                    continue
                for aa in seqlst:
                    if aa == "-":  #which means there is a gap.
                        #print(aa)
                        if i not in gaps_redundant:
                            gaps_redundant.append(i)
                        
                        i += 1
                        continue
                    i += 1
    
    
    gaps_to_remove = sorted(list(set(gaps_redundant)))  #set to make sure there are no duplicates.
        
    #print("apparent gaps to remove")
    #print(gaps_to_remove)
    
    return gaps_to_remove, deletion_list

In [80]:
#test 

#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994"
#listlen = read_msa_file_version_1(path=path)
#print(len(listlen))

In [81]:
def _remove_deviating_length_strucs(path_to_pdbs:str, template_struc:str):
    
    """This function will take in the pdbs in the directory of interest and remove those that are simply too large.
    If they are too small this will also be removed.
    All length criteria are based on the selected reference structure."""
    
    
    #first the template_struc to set length standard.
    parser = PDBParser(QUIET=True)
    
    
    prot_path = f"{path_to_pdbs}/{template_struc}"
    
    print(prot_path)
    prot_name = "default"
    
    seq_length_ref = 0
    
    try:
        
        structure = parser.get_structure(prot_name, prot_path)
        
        for model in structure:
            for chain in model:
                for residue in chain:
                    seq_length_ref += 1
            
    except Exception as error:
        print(error)
        
    #now we parse through all of the rest and remove those that are less/more than 20% of the ref struc length.
    
    onlyfiles = [f for f in os.listdir(path_to_pdbs) if os.path.isfile(os.path.join(path_to_pdbs, f))]
    
    pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
    #we keep track of which is the shortest ref seq because we want to cut down later based on that size
    #to end up with uniform lengths.
    
    shortes_seq_len = seq_length_ref
    #now we remove all pdbs that dont fullfill the above criteria.
    for pdb in pdbs:
        
        try:
            
            seq_len = 0
            
            prot_path = f"{path_to_pdbs}/{pdb}"
            
            prot_name = "default"
            
            structure = parser.get_structure(prot_name, prot_path)
        
            for model in structure:
                for chain in model:
                    for residue in chain:
                        seq_len += 1
            
            
            
            print(pdb, seq_len)
            #if this is the case, we remove it.
            if seq_len < 0.9 * seq_length_ref or seq_len > 1.1 * seq_length_ref:  #here we can experiment how strict we want to be.
                
                print(f"we remove {pdb} because its seq_len is: {seq_len} and the refsef is : {seq_length_ref}")  
                os.remove(f"{path_to_pdbs}/{pdb}")
                
            else:
                #check if its shorter than the current shortest seq . This block only counts those that are not rejected
                #i.e nothing thats 80% or less than refseq.
                if seq_len < shortes_seq_len:
                    shortes_seq_len = seq_len
                
        except Exception as error:
            print(error)
            
    return shortes_seq_len

In [82]:
def _mutate_non_standard_aa(path_to_pdb:str,
                           non_standard_residue:str,
                           residue:int,
                           chain:str):
    
    """This function will take a pdb file and calls the mutate_py script to change a single aa per time.
    Additionally, this scripts output will be used to overwrite an existing pdb file in order to tandemly 
    mutate multiple aa in the same pdb file in consequtive function calls."""
    
    
    
    path_to_script = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_mutate.py"
    
    path_to_error_script = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_delresname.py"
    
    path_to_tidy = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_tidy.py"
    
    resulting_success = False
    
    #posttranslational_dict = {"PTR":"THR",
    #                          "SNC":"CYS",
    #                          "BFD":"ASP",
    #                          "PHD":"ASP",
    #                          "UNK":"ALA"
    #                          }
    #
    try:
        with open(f"{path_to_pdb}_new", "w") as pdb_out:
            
            bash_code = f"python {path_to_script} {path_to_pdb} {chain} {residue} {non_standard_residue} ALA"
    
            bash_cmd_rdy = bash_code.split() #ready for deploy 
                
            result = run(bash_cmd_rdy, stdout=pdb_out, stderr=PIPE, 
                         universal_newlines=True)
            
            
            resulting_success = True 
            
    except Exception as error:
        
        print(error)
        print("we try now to delete it")
        with open(f"{path_to_pdb}_new", "w") as pdb_out:
        
            bash_code = f"python {path_to_error_script} -{non_standard_residue} {path_to_pdb}"

            bash_cmd_rdy = bash_code.split() #ready for deploy 
        
            print(bash_cmd_rdy)
        
            result = run(bash_cmd_rdy, stdout=pdb_out, stderr=PIPE, 
                     universal_newlines=True)

            resulting_success = False
            
    #now we need to remove the input file and rename the output to original input (TBD)
 
    #in both cases we should tidy the file!
    
    try:
        with open(f"{path_to_pdb}", "w") as pdb_out:
            
            bash_code = f"python {path_to_tidy} {path_to_pdb}_new"
            
            print(bash_code)
            
            bash_cmd_rdy = bash_code.split() #ready for deploy 
                
            result = run(bash_cmd_rdy, stdout=pdb_out, stderr=PIPE, 
                         universal_newlines=True)
            
            
            
            print("we cleaned the file")
            print(f"outfile is at {path_to_pdb}")
            
    except Exception as error:
        print(error)
    
    
    
    
    try:
        #first we remove the old pdb file
        #os.remove(f"{path_to_pdb}")
        #then we replace the old one with the new one.
        #shutil.copy(f"{path_to_pdb}_new", path_to_pdb)
        #Then we remove the intermediate file.
        os.remove(f"{path_to_pdb}_new")
        pass
    except Exception as error:
        print(error)
    
    return resulting_success
    

In [83]:
path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/3ar6.pdb"
chain = "A"
residue_num = 1
non_standard_resi = "ACE"

#_mutate_non_standard_aa(path_to_pdb=path, 
#                       chain=chain,
#                       residue_number=residue_num,
#                       non_standard_residue=non_standard_resi)

### PRE PCA PROCESSING / FETCH PROPER RESIDUES

In [84]:
def prep_structures_pca(remove_pos:list, path_to_strucs:str,
                       max_seq_len:str, delete_pdbs:str):
    
    '''The kept positions are based on str position (not residue pos) so we need to
    change each structure to 1 based residue counting 
    and then select ONLY those residues that are NOT in the remove positions range'''
    
    #renumber first all structures.
    
    pdbs = []
    
    with open(f"{path_to_strucs}/chain_list.txt", "r") as chain_file:
        for lines in chain_file:
            if lines != "\n":
                #this will contain files as "4r3d.pdb"
                pdb = lines.replace("\n","")
                
                if pdb[0:4] in delete_pdbs:
                    print(f"{path_to_strucs}/{pdb}")
                    os.remove(f"{path_to_strucs}/{pdb}")  # we remove it if its deemed bad.
                    continue
                
                else:
                    pdbs.append(pdb)
    
    
    #now we rechain copies.
    
    try:
        os.mkdir(f"{path_to_strucs}/PCA")
    except Exception as error:
        print(f"{path_to_strucs}/PCA already exists")
    
    
    #loop over pdb list
    
    #seems to work.
    
    #required for the script to work.
    
    '''This block below needs to be tested.'''
    
 
    #first renumbering! because gap pos are 1 counted and start in all structures with 1 (e.g even if structure atom is normally positon 32 it will start with 1.
    
    pdb_reres_path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_reres.py"
    
    for pdb_entries in pdbs:
        
        bash_cmd = f"python {pdb_reres_path} -1 {path_to_strucs}/{pdb_entries}"
                
        bash_cmd_rdy = bash_cmd.split()
        
        try:
            with open(f"{path_to_strucs}/PCA/{pdb_entries}", "w") as fh_tmp:
                result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, 
                         universal_newlines=True)
            
            print(f"we renumbered {pdb_entries}")
        
        except Exception as error:
            print(f"we did not renumber {pdb_entries}")
        
            print(error)
    
    #first remove the HET Atoms.
    for pdb_entries in pdbs:
        
        non_canonical = remove_hetero_atoms(pdb_file=f"{pdb_entries}", path=f"{path_to_strucs}/PCA")
        
        #this means we have something special that needs to be taken care of.
        
        if non_canonical:
            
            #loop over dict that contains all positions that deviate from standard.
            
            #structure of dict is : key: position value: tuple: (aa_type, chain)
            print("This is non canonical:")
            print(non_canonical)
            
            
            remove_targets = []
            fulllst = []
            for keys, vals in non_canonical.items():
                print(keys)
                if keys in remove_targets:
                    continue
                else:
                    remove_targets.append(keys)
                    fulllst.append((keys, vals[0], vals[1]))

            for (pos, hits, chain) in fulllst:
                
                print(pos, hits, chain)
                #call function iteratively to replace step by step.
                result = _mutate_non_standard_aa(path_to_pdb=f"{path_to_strucs}/PCA/{pdb_entries}",
                       non_standard_residue=hits,residue=pos,chain = chain)
                
                if result == False:
                    print(f"we did not manage to delete and so we need to introduce a gap at {pos}")
                    #if we could not repair and were forced to delete it we append this and take it as a gap. otherwise its fine if it got "repaired" to ALA
                    remove_pos.append(pos)
                    
        else:
            continue
    
    
    
    print(remove_pos)
    

    #here ends removal of het atoms.
    
    #pdb_reres_path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_reres.py"
    #
    #for pdb_entries in pdbs:
    #    
    #    bash_cmd = f"python {pdb_reres_path} -1 {path_to_strucs}/{pdb_entries}"
    #            
    #    bash_cmd_rdy = bash_cmd.split()
    #    
    #    with open(f"{path_to_strucs}/PCA/{pdb_entries}", "w") as fh_tmp:
    #        result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, 
    #                     universal_newlines=True)
    #        
            
        
    #now only select those residues that we need.
    
    
    print("max seq len is:")
    print(max_seq_len)
    
    #we use this class and overwrite it for our purpose.
    class ResidueSelect(Bio.PDB.Select):
        def accept_residue(self, res):
            #we accept all residues that are in the kept-position list
            if res.id[1] in remove_pos or res.id[1] > max_seq_len:
                return False
            #but we reject those that are not shared between all structures.
            else:
                return True

    
    #now we select only those that we need and overwrite existing files.
    for pdb_entries in pdbs:
        
        parser = PDBParser(QUIET=True)
                
        prot_name = f"default"
                
        #open the correct pdb and rechain it.
        structure_template = parser.get_structure(prot_name, 
                            f"{path_to_strucs}/PCA/{pdb_entries}")
        
        io = PDBIO()
            
        io.set_structure(structure_template)
            
        io.save(f"{path_to_strucs}/PCA/{pdb_entries}", ResidueSelect())
        
    #this return will contain the number of CAs required for the next step.
    #second part of tuple contains the number of pdb structures in the ensemble.
    
    #pdbs 0 = ref struc.
    
    print("this is pdbs:")
    print(pdbs)
    
    #if its 0 we dont care any further.
    if len(pdbs) == 0:
        return None
    
    return pdbs[0] #else we return the template.
    

In [85]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994"
#max_seq_len = 10000
#prep_structures_pca(remove_pos=[], path_to_strucs=path,
#                       max_seq_len= max_seq_len, delete_pdbs="None")


In [86]:
#test

#rem_pos = [1,2,3]
#path_to_struc = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AACS_HUMAN/trimer/pos_16_671"

#max_seq_len = 1000


#prep_structures_pca(path_to_strucs=path_to_struc,max_seq_len=max_seq_len, remove_pos=rem_pos)

In [87]:
remove_gaps = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 27, 28, 32, 33, 35, 36, 37, 38, 47, 49, 50, 51, 52, 53, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 97, 98, 99, 100, 101, 122, 123, 124, 125, 148, 149, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368]

path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/FAK1_HUMAN/monomer/pos_410_689"

chains = "A"

ref_struc_pdb = "2v7a"

min_seq_len = 241
                
#then we will prepare those structures, cut gaps and will be left with uniform length
#pdbs that have 1:1 correspondence between each residue against all pdbs.
#ref_struc_pdb_id = prep_structures_pca(path_to_strucs=path, 
#                        remove_pos=remove_gaps,
#                        max_seq_len = min_seq_len) #min_seq_len is length of shortes structure in the ensemble.
                
                
#chains_of_ref = _get_chain_labels(path, ref_struc_pdb_id)
                
#dir_path_pca = f"{path}/PCA/"
                
                
#PCA_domenico_new(path_to_pdbs=dir_path_pca,
#                ref_struc_pdb_id=ref_struc_pdb_id[0:4],
#                chains_of_reference_struc=chains_of_ref)
    


In [88]:
path = f"/home/micnag/bioinformatics/python_scripts_test/2v7a.pdb"
chain = "A"

residues = 393
non_standard_residue = "PTR"


#_mutate_non_standard_aa(path_to_pdb=path,
#                       chain=chain,
#                       residue_number=residues,
#                       non_standard_residue=non_standard_residue)

#works


# PCA PLACE

In [89]:
def PCA_domenico_new(path_to_pdbs:str, ref_struc_pdb_id:str,
                    chains_of_reference_struc:str):
    
    
    """This function will take in the input directory with the pdb files that
    are going to be used for PCA.
    Domenicos pipeline consisting of 3 scripts needs to be MOVED around into the working dir
    and then removed back after the job is done.
    
    Input:
    
    path_to_pdbs: the directory that contains the pdbs.
    ref_struc_pdb_id: the 4 digit code OF THE REFERENCE STRUCTURE.
    
    exe are:
    
    + add_chain_label_ali
    + run_pca_domenico
    + write_CA
    
    the shellscript that combined them is:
    + PCA_pipeline.sh
    
    Additional files required there are:
    
    + pdbs
    + ensemble.txt
    
    All work needs to be done in the same directory (i.e the working dir) and we need to os.chdir() to this
    dir and afterwords os.chdir() back.
    
    Execution:

    bash PCA_pipeline.sh INPUT1 INPUT2

    INPUT1 : pdb id (4 digits) of reference structure
    INPUT2 : chains id (max 100 characters) of reference structure

    INPUT1 and INPUT2 need to be the same as in the first row of the "ensemble.txt" file

    """
    
          
    print("we are inside PCA_domenico_new")
          
        
    
    #check first which structures are present:
    #get_rcsb_info(path=path_to_pdbs)
    
    
    
    print(f"chains:{chains_of_reference_struc}")
    
    print(f"ref_struc_pdb:{ref_struc_pdb_id}")
          
          
    #first we need to make the working dir.
    
    basepath = "/home/micnag/bioinformatics/domenico_pca"

    try:
        
        #move executables to this new location.
        
        os.chdir(f"{path_to_pdbs}")
        shutil.copy(f"{basepath}/add_chain_label_ali",f"{path_to_pdbs}")
        shutil.copy(f"{basepath}/write_CA",f"{path_to_pdbs}")
        shutil.copy(f"{basepath}/run_pca_domenico",f"{path_to_pdbs}")
        shutil.copy(f"{basepath}/PCA_pipeline.sh",f"{path_to_pdbs}")
    
        #change into new working dir.
        os.chdir(f"{path_to_pdbs}")
        
    except Exception as error:
        print(error)
        os.chdir(f"{path_to_pdbs}")
        
    scriptcall = f"./PCA_pipeline.sh"
    
    
    #first we remove all deviating CA files (in case there are any
    _remove_deviating_ca(path_to_pdbs=path_to_pdbs,
                        ref_struc_pdb_id=ref_struc_pdb_id)
    
    
    onlyfiles = [f for f in os.listdir(f"{path_to_pdbs}/") if os.path.isfile(os.path.join(f"{path_to_pdbs}/", f))]
    only_pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
    
    #we need to create ensemble.txt which is required by the pipeline.
    #format : each line : <4 digit pdb> <chain(s)>
    with open(f"{path_to_pdbs}/ensemble.txt", "w") as ensemble_txt:
        for pdb_ids in only_pdbs:
            pdb_id = pdb_ids[:-4]  #we just need the 4 digit ids. 
            chain = chains_of_reference_struc #this has to be the same as our reference structure.
            
            append_str = pdb_id + " " + chain + "\n"
            print('append string is:', append_str)
            ensemble_txt.write(append_str)
    
    
    
    print(os.getcwd())
    
    print("these files are present in the directory:")
    
    
    onlyfiles = [f for f in os.listdir(f"{path_to_pdbs}/") if os.path.isfile(os.path.join(f"{path_to_pdbs}/", f))]
    
    print(onlyfiles)
    
    bash_cmd = f"{scriptcall} {ref_struc_pdb_id} {chains_of_reference_struc}"
    
    print(bash_cmd)
    
    bash_cmd_rdy = bash_cmd.split()
    
    try:
        result = run(bash_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                 universal_newlines=True)
            
        print(result.stdout)
        print(result.stderr)
            
    except:
        print(f"Script did not work. Here are the supplied parameters \n{bash_cmd_rdy}")
        
    #if it worked we restore the previous status quo:
    
    """
    try:
        
        #remove them from working dir back to normal storage dir.
        os.remove(f"{path_to_pdbs}/PCA/add_chain_label_ali")
        os.remove(f"{path_to_pdbs}/PCA/write_CA")
        os.remove(f"{path_to_pdbs}/PCA/run_pca_domenico")
        os.remove(f"{path_to_pdbs}/PCA/PCA_pipeline.sh")
        
        #change back into previous work dir.
        os.chdir(f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs")
        
    except Exception as error:
        
        os.chdir(f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs")
        print("Something did not work out at reshuffling .exe back into their former location.")
        print(error)
    """

In [90]:
def _remove_deviating_ca(path_to_pdbs:str, ref_struc_pdb_id:str):
    """helper function to remove deviating C-Alpha number files."""
    
    onlyfiles = [f for f in os.listdir(f"{path_to_pdbs}/") if os.path.isfile(os.path.join(f"{path_to_pdbs}/", f))]
    only_pdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
    
    
    print(f"this is template path: {path_to_pdbs}/{ref_struc_pdb_id}.pdb")
    template_path = f"{path_to_pdbs}/{ref_struc_pdb_id}.pdb"
        
    parser = PDBParser(QUIET=True)
    
    prot_name = "noname"
    structure = parser.get_structure(prot_name, template_path)
    
    i = 0
    for res in structure.get_atoms():
        #we only count CA. if structures deviate from that.. remove them.
        if res.get_id() == "CA":
            i += 1
    
    
    for pdbs in only_pdbs:
        
        try:
            parser = PDBParser(QUIET=True)
    
            prot_name = "noname"
            structure = parser.get_structure(prot_name, f"{path_to_pdbs}/{pdbs}")
        
            j = 0
            for res in structure.get_atoms():
            #we only count CA. if structures deviate from that.. remove them.
                if res.get_id() == "CA":
                    j += 1
        
            #this means it has different CA
            if j != i:
                print(pdbs)
                print(j)
                os.remove(f"{path_to_pdbs}/{pdbs}")
                
                
        except Exception as error:
            print(error)
            
            #if we cant open it.. its faulty. remove it.
            os.remove(f"{path_to_pdbs}/{pdbs}")

In [91]:
#generalize this plot function.

def _plot_PCA(path_to_PCA:str):
    
    """Quick function to plot the first 2 PC and their corresponding explained variances."""
    
    expl_var = f"{path_to_PCA}/pc_variances.txt"
    pca = f"{path_to_PCA}/exp_ensemble_proj_PC.txt"
    
    structure_path = f"{path_to_PCA}/ensemble.txt"
    
    structures = []
    
    title_path = path_to_PCA.split("/")
    gene = title_path[-4]
    oligomer = title_path[-3]
    position = title_path[-2]
    
    
    expl_var_vals = []
    with open(expl_var, "r") as var:
        for entries in var:
            entries = entries.replace("\n","")
            expl_var_vals.append(float(entries))
    
    
    with open(structure_path, "r") as struc_in:
        for lines in struc_in:
            pdb = lines.replace("\n"," ")
            pdb = pdb[0:4]
            structures.append(pdb)
            
    #we compute the first 10.
    PCA = defaultdict(list)
    
    with open(pca, "r") as pc_read:
        for lines in pc_read:
            #lets collect each column as a separate list
            
            #each member in the list will correspond to a structure and its associated PC value.
            for i in range(0,10):
                PCA[i+1].append(float(lines.split()[i]))
    
    
    
    plt.plot(PCA[1], PCA[2], "o")
    
    zipped_PCA = list(zip(PCA[1],PCA[2]))
    
    
    accept_lst = ["3w5b", "2oa0", "6hxb"]

    for i, (x,y) in enumerate(zipped_PCA):
        
        label = "{}".format(structures[i])
        
        if label in accept_lst:
            print(label)
            plt.annotate(label, # this is the text
                 (x, y), # these are the coordinates to position the label
                 textcoords="offset points", # how to position the text
                 xytext=(-10,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center
            print(x,y)
            plt.plot(x,y , "o",color="red")
            
    plt.title(f"{gene}\n{oligomer} {position}")
    plt.xlabel(f"PCA 1 (variance {expl_var_vals[0]}%)")
    plt.ylabel(f"PCA 2 (variance {expl_var_vals[1]}%)")
    
    
    plt.savefig(f"{path_to_PCA}/PC_plot2.png")
    plt.clf()

#path_to_PCA = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/PCA"   
    
#_plot_PCA(path_to_PCA=path_to_PCA)

In [92]:
#path_to_PCA = "/home/micnag/bioinformatics/test/test_pca"   
    
#_plot_PCA(path_to_PCA=path_to_PCA)

In [93]:
def porcupine_plot(path_to_PCA:str):
    """Function to create porcupine plots"""
    
    
    path_to_pca_comp = f"{path_to_PCA}/PCs.txt"
    
    path_to_pca_proj = f"{path_to_PCA}/exp_ensemble_proj_PC.txt"
    
    pca_components = np.loadtxt(path_to_pca_comp, usecols=[0,1])
    pca_scores = np.loadtxt(path_to_pca_proj, usecols=[0,1])
    
    normalized_pca_components = pca_components / np.linalg.norm(pca_components, axis=1)[:, np.newaxis]
    
    #print(normalized_pca_components[0:10])
    
    plt.figure(figsize=(8, 6))
    plt.scatter(pca_scores[:, 0], pca_scores[:, 1], color='blue', label='Data Points')

    for i in range(len(pca_scores)):
        x0, y0 = pca_scores[i]
        dx, dy = normalized_pca_components[i]
        plt.arrow(x0, y0, dx, dy, color='red', width=0.02, head_width=0.1, length_includes_head=True)
    
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('Porcupine Plot - PCA Data')
    plt.legend()
    plt.grid(True)
    plt.show()

In [94]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/PCA"

#porcupine_plot(path)

# REPAIR STATION MODELLER.


In [95]:
#recursive helper function to measure gaps
def _catchup(idx:int, resseq:int, start:int):
    
    '''This helper function will start iteratively catching up to the gap by incrementing
    each iteration by 1 until the idx matches the current resseq
    The amount of iterations needed to get back to the resseq is returned together
    with the start and end of the gap.'''
    
    #if we have a gap.
    if idx != resseq:
        #close gapsize by 1 and recall function recursively.
        idx += 1
        return _catchup(idx,resseq, start) #needs function call otherwise return:None
    
    #if we sealed the gap we return the report.
    return(idx, resseq, start)

In [96]:
def check_gaps_single_struc(path:str, report=True)-> dict:
    ''' This function should take all pdbs (already removed from het atoms)
        and sort them according to a fixed max_gap parameter.
        If this gap length is exceeded, we trash the structure.
        Otherwise, we separate in two folders:
        structures that are completely intact and those that need repair
        but have gaps smaller than max_gap.
        If report = True we will also write out a report documenting the gaps.'''
    
    #setup
    #set path to folderpath
    
    parser = PDBParser(QUIET=True)
    
    prot_path = path.split(sep="/")
    
    #last entry of path is 4r23_A_0.pdb. so we need 4r23_A_0 
    prot_name = prot_path[-1][:-4]
    
    '''main part'''
    #print(onlyfiles)
    
    gap_dict = defaultdict(str)
    

    fullpath = f"{path}"   # ../P0633/1c0f.pdb
    
    try:
        structure = parser.get_structure(prot_name, fullpath)
        
        #we will return the gaps list at the end of function (explained there at return)
        gaps = []
        start_res = []
        '''Just used to grab the first residue in each chain!'''
        for model in structure:
            for chain in model:
                chains = chain.get_residues()
                for residue in chains:
                    tmp = residue.get_full_id()[3][1]
                    start_res.append(tmp)
                    break
       
        i = 0 #we iterate through the list of start_res given that we can have many chains!
        # e.g each chain can start with a different residue... e.g pos 5 pos 10 ect.
        for model in structure:
            
            for chain in model:
                
                #we store the chain id for later purpose of repairing
                chain_id = chain.get_full_id()[2]
                
                idx = start_res[i] #get first residue position
                
                i += 1 #shift pointer by 1 to get next start residue pos in next chain iteration.
                
                """THIS PART NEEDS MAJOR OVERHAUL!""" #done mostly
                for res in chain.get_residues():
                
                    resseq = res.get_full_id()[3][1]
                    #print(idx, resseq)
                    
                    if idx != resseq: #this means we have a gap!

                        if idx > resseq:
                            continue
                        #store start of gap for later return
                        start = idx
                    
                        #we try to catch it then
                        res = _catchup(idx,resseq, start) #recursive call
                    
                        diffval = res[0]-res[2] #diff between end of gap and start of gap = gaplength
                        
                        gaps.append((chain_id, res[2]-1, res[0], diffval)) # end - start
                        #print(gaps)
                        idx = res[0]+1 #update to bring idx back to current resseq
                        continue
                    
                    idx += 1
                    if idx < resseq:
                        print(f"idx is {idx}, resseq is {resseq}")
                        break
                        
            gap_dict[prot_name] = gaps
    except Exception as error:
            print(error)
    
        
    return gap_dict

In [97]:
def repair_viability(gap_dict:dict, max_gap=7):
    
    """Quick check if the pdb can be salvaged or if it should be left as it is."""
    
    gap_lengths = []
    
    for pdb, gaps in gap_dict.items():
        
        if len(gaps) == 0:
            return False
        for gap in gaps:
            single_gap = gap[3]
            gap_lengths.append(single_gap)
            
    gap_lengths_sort = sorted(gap_lengths, reverse=True)
    
    #if we find that the max gap is too large we return False else True
    return False if gap_lengths_sort[0] > max_gap else True
    

## REPAIR

+ build model from fasta files
+ rechain new model 
+ cut according to residues that we need (i.e remove N and C-terminal overhangs, which is only spaghetti)

In [98]:
#initial works but needs overhaul!

#seems to work but should only be used on cases where we have small gaps.

def repairing_residues(pdb_id_target:str, path:str):
    """Function should take only those proteins that have
    a) gaps with less than 7 residues missing per gap.
    
    Input:
    path to pdb_folder
    gap_dict file
    
    Output:
    produces repaired structures
    """
    
    #onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    #pdbs = [corr_file for corr_file in onlyfiles if corr_file[-4:] == ".pdb"]
    
    log.none() #no stdout spam
    env = Environ() #setup env for modelling
    
    aln = Alignment(env) # setup alignment within the env.
    
    mdl = Model(env)  #setup the model object where we will append our pdbs into.
     
    current_pth = os.getcwd()
    
    
    try:
        
        os.mkdir(f"{path}/{pdb_id_target}")
        #print(f"{path}/{pdb_id_target}")
        #print(f"{path}/{pdb_id_target}/{pdb_id_target}")
        shutil.copy(f"{path}/{pdb_id_target}.pdb", f"{path}/{pdb_id_target}/{pdb_id_target}.pdb")
    except:
        print("already there")
    
    
    
    os.chdir(f"{path}/{pdb_id_target}")
    print(os.getcwd())
    
    env.io.atom_files_directory = ['.','../.']
    
    start, stop, chain = _start_stop_fasta(pdb_id_target=pdb_id_target, path=f"{path}/{pdb_id_target}")
    
    #write fasta
    #grab start + end pos.
    #grab fasta from a different path.
    
    gene_name = get_gene_fasta(f"{pdb_id_target[0:4]}") #gives back the gene_name
    
    gene_name_id, gene_id = get_gene_name_uniprot(gene_name)

    fasta_seq = get_gene_fasta(gene_id) #this should work.
    
    if fasta_seq == None:
        return #we cant progress anyfurther then.
    
    code = f"{pdb_id_target}"
    
    #print("this will be start and stop")
    #print(start, stop)
    
    
    mdl.read(file=code, model_segment=(f"{start}:{chain}", f"{stop}:{chain}"))
    
    aln.append_model(mdl, align_codes=code, atom_files=code)
    
    #print("this is pdb_id_target and chain")
    #print(f"{pdb_id_target[0:4]}", chain)
    
    #this is the old version which grabs full fasta.
    #fasta_seq = get_gene_fasta_from_pdb_id(f"{pdb_id_target[0:4]}", chain=chain)
    
    #print("this is fasta seq")
    #print(fasta_seq)
    
    with open(f"./{pdb_id_target}x.fasta", "w") as fastaout:
        fastaout.write(f">{pdb_id_target}x\n")
        fastaout.write(fasta_seq)
    
    
    aln_code = f"{pdb_id_target}x"
    
    aln.append(file=f"./{pdb_id_target}x.fasta", align_codes=aln_code, 
               alignment_format="fasta")
    
    aln.salign(overhang=30, gap_penalties_1d=(-450, -50),
    alignment_type="tree", output="ALIGNMENT")
    
    ##aln.malign(gap_penalties_1d=(-500, -300))
    ##aln.malign3d(gap_penalties_3d=(0.0, 2.0))
    aln.write(file=f"{pdb_id_target}.ali")

    #lets model
    
    #we will fish one out of the pdb list and align it against all others.
    
    a = AutoModel(env,
                  alnfile = f"{pdb_id_target}.ali",
                  knowns = f"{pdb_id_target}",
                  sequence = aln_code)
    
    a.starting_model = 1
    a.ending_model = 1
    a.make()
    
    os.chdir(current_pth)
    
    #If successful:
    
    pdbs = []
    onlyfiles = [f for f in os.listdir(f"{path}/{pdb_id_target}") if os.path.isfile(os.path.join(f"{path}/{pdb_id_target}", f)) and f[-4:] == ".pdb"]
    pdbs_sorted = sorted(onlyfiles, key=len)
    #print(pdbs_sorted)
    #the old one needs to be removed.
    #print("this is the remove path")
    #print(f"{path}/{pdb_id_target}/{pdbs_sorted[0]}")
    #os.remove(f"{path}/{pdb_id_target}/{pdbs_sorted[0]}")
    #the new one needs to be renamed
    shutil.copy(f"{path}/{pdb_id_target}/{pdbs_sorted[1]}",f"{path}/{pdb_id_target}/original_{pdbs_sorted[1]}")
    os.rename(f"{path}/{pdb_id_target}/{pdbs_sorted[1]}", f"{path}/{pdb_id_target}/{pdbs_sorted[0]}")
    
    #If rebuilt, we need to RENUMBER AGAIN (TAKE CORRECT SHIFT INTO ACCOUNT)
    #now move the result back to the original location and overwrite the gap containing struc.
    os.remove(f"{path}/{pdbs_sorted[0]}")
    #print(f"we removed {path}/{pdbs_sorted[0]}")
    
    #DONT RENUMBER, but instead CUT OFF according to start stop. RENUMBERING NOT needed because FASTA is already providing 1-END numbering:
    
    
    _cut_proper_models(path_to_struc=f"{path}/{pdb_id_target}/{pdbs_sorted[0]}",
                   start_num = start, end_num = stop,
                       chain=chain)
    
    #_renumber_struc(path_to_struc=f"{path}/{pdb_id_target}/{pdbs_sorted[0]}",
    #               start_num = start, chain=chain)
    
    shutil.copy(f"{path}/{pdb_id_target}/{pdbs_sorted[0]}",f"{path}/{pdbs_sorted[0]}")
    #print(f"we moved {path}/{pdbs_sorted[0]}/{pdbs_sorted[0]} to {path}/{pdbs_sorted[0]}")
    #back to normal working dir
    os.chdir(current_pth)
    #os.rename and maybe copy back to original file dest.
    
    #else. ignore and continue.
    

In [99]:
 def _start_stop_fasta(pdb_id_target:str, path:str):
    
    """This function reads in a structure and returns start / end of it to limit the seq which 
    will be used to rebuild gapped structures"""
    

    fullpath = f"{path}/{pdb_id_target}.pdb"
    
    parser = PDBParser(QUIET=True)
    protname = pdb_id_target
    chain_corr = pdb_id_target[5:6]
    
    #chain_corr = "A"  #THIS IS ALWAYS A BECAUSE MODELLER MAKES ALL RESULT FILES BE CHAIN A. DONT CHANGE IT TO WHATEVER CHAIN IT REALLY IS.
   
    structure = parser.get_structure(protname, fullpath)
    
    positions = []
    seq = []
    
    for model in structure:
        for chain in model:
            if chain.get_id() == chain_corr:
                for residues in chain:
                    res_num = residues.get_id()[1]
                    if res_num > 0:
                        positions.append(res_num) #we discard negative entries
                        seq.append(residues.get_resname())
    
    start, stop = positions[0], positions[-1]
    #gives back start and stop.
    return start, stop, chain_corr

#_start_stop_fasta(pdb_id_target="2duk_A_0.pdb", path="/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/")

In [100]:
def _renumber_struc(path_to_struc:str,
                    start_num:str, 
                    chain:str):
    
    #renumbering fresh struc:
    
    shiftres_location = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/pdb_shiftres_by_chain.py"
    
    #rechain again.
    parser = PDBParser(QUIET=True)
                
    prot_name = f"default"
                
    #open the correct pdb and rechain it.
    structure_template = parser.get_structure(prot_name, path_to_struc)
        
    new_chain = chain 
        
    for models in structure_template:
        for chains in models:
                
            chains.id = "_"
            
            chains.id = new_chain
            
            io = PDBIO()
            
            io.set_structure(structure_template)
            
            io.save(path_to_struc)
    
    #we continue
    
    print("This is path, start num and chain")
    print(f"{path_to_struc}")
    
    print(f"{start_num}")
    print(f"{chain}")
    
    bash_cmd = f"python {shiftres_location} {path_to_struc} {start_num} {chain}"
    
    bash_cmd_rdy = bash_cmd.split()
    
    with open(f"{path_to_struc}_tmp", "w") as fh_tmp:
        result = run(bash_cmd_rdy, stdout=fh_tmp, stderr=PIPE, 
             universal_newlines=True)
    
    #now replace the original one with the temp file.
    os.replace(f"{path_to_struc}_tmp", f"{path_to_struc}")
    

In [101]:
def _cut_proper_models(path_to_struc:str,
                   start_num:str, end_num:str,
                       chain:str):
    
    
    class ResidueSelect(Bio.PDB.Select):
        def accept_residue(self, res):
            #we accept all residues that are within start and stop 
            if res.id[1] >= int(start_num) and res.id[1] <= int(end_num) and res.parent.id == new_chain:  #i changed chain to new_chain
                return True
            #but we reject those that are before (N-terminal overhang) or after (C-terminal overhang)
            #these are anyhow just spaghetti model.
            else:
                return False

    #rechain again.
    parser = PDBParser(QUIET=True)
                
    prot_name = f"default"
                
    #open the correct pdb and rechain it.
    structure_template = parser.get_structure(prot_name, path_to_struc)
        
    new_chain = chain 
        
    for models in structure_template:
        for chains in models:
                
            chains.id = "_"
            
            chains.id = new_chain
            
            io = PDBIO()
            
            io.set_structure(structure_template)
            
            io.save(path_to_struc, ResidueSelect())
    
    

### WE NEED TO UPDATE THE FASTA PROCESSING IN ORDER TO INCLUDE ALL CHAINS PROPERLY.

Needs to take chain A and B separately in order to not mess up.

# ALL OLIGOMER PIPELINE 
(PCA prep + PCA + PLOT)

In [102]:
def all_oligomer_runs(path:str, min_seq_len_dict:dict):
    
    """This function basically will call a bunch of other functions 
    for each potential oligomeric state and each position within them."""
    
    oligodirectories = ["monomer",
                        "dimer",
                        "trimer",
                        "tetramer",
                        "pentamer",
                        "hexamer",
                        "heptamer",
                        "oktamer",
                        "nonamer",
                        "decamer",
                        "undecamer",
                        "dodecamer",
                        "tridecamer",
                        "tetradecamer",
                        "pentadecamer",
                        "hexadecamer",
                        "heptadecamer",
                        "oktadecamer",
                        "nonadecamer",
                        "eicosamer"
    ]
    
    relevant_dirs = []
    
    for file in os.listdir(path):
        if file in oligodirectories:
            d = os.path.join(path, file)
            relevant_dirs.append(d)
            
    
    print("this is relevant dirs")
    #now we go through each of those oligomer dirs and for each position:
    
    
    #lets see here why we dont go into some higher oligomers pca.
    print(relevant_dirs)
    
    dir_dictionary = defaultdict(list)
    
    for dirs in relevant_dirs:

        dir_dictionary[dirs] = os.listdir(dirs)
    
    
    print("this is dir dictionary")
    print(dir_dictionary)
    
    path_dict = defaultdict()
    
    for keys, vals in dir_dictionary.items():
        subdir_list = []
        for subdir in vals:
            new_path = keys + "/" + subdir
            subdir_list.append(new_path)

        path_dict[keys] = subdir_list 

    
    for keys, vals in path_dict.items():
        print("this is path_dict.items")
        print(keys, vals)
        for dir_paths in vals:
            
            #now we need to parse through ALL of these directories:
            
            onlyfiles = [f for f in os.listdir(dir_paths) if os.path.isfile(os.path.join(dir_paths, f))]
            
            onlypdbs = [f for f in onlyfiles if f[-4:] == ".pdb"]
            
            print("This is onlypdbs from within the run_all_oligomers")
            print(onlypdbs)
            if len(onlypdbs) < 5:
                #if thats the case that we dont have more than 2 structures 
                #we go to the next dir.
                continue
            
            # '/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/monomer/pos_31_97'
            path_split = dir_paths.split("/")
            oligostate = path_split[-2] #this corresponds to monomer e.g
            position_ = path_split[-1] #thi sis pos_31_97 e.g
            
            search_oligo_pos = f"{oligostate}>{position_}"
            
            print("this is min seq len dict at search oligo pos")
            print(min_seq_len_dict[search_oligo_pos])
            min_seq_len = min_seq_len_dict[search_oligo_pos]
            print(min_seq_len)
            
            
            #first we grab which positions are gap and whats the max seq length will be.
            remove_pos, delete_pdbs = read_msa_file_version_1(dir_paths)
            
            
            print("this is remove pos")
            print(remove_pos)
            print("this is delete_pdbs")
            print(delete_pdbs)
            
            #then we will prepare those structures, cut gaps and will be left with uniform length
            #pdbs that have 1:1 correspondence between each residue against all pdbs.
            
            ref_struc_pdb_id = prep_structures_pca(path_to_strucs=dir_paths, 
                                remove_pos=remove_pos,delete_pdbs=delete_pdbs,
                                max_seq_len = min_seq_len) #min_seq_len is length of shortes structure in the ensemble.
            
            if ref_struc_pdb_id == None:
                continue #means we are done here.
            
            print("we enter chains_of ref")
            print("this is dir path pca and ref_struc_pdb_id")
            print(dir_paths, ref_struc_pdb_id)
            chains_of_ref = _get_chain_labels(dir_paths, ref_struc_pdb_id)
            
            
            print("we enter now domenico_new")
            
            dir_path_pca = f"{dir_paths}/PCA"
            
            
            print(dir_path_pca, ref_struc_pdb_id[:-4], chains_of_ref)
            
            #ref_struc [0:4] = 4 digit code
            
            #grep -a because temp output file in domenicos code after gromacs alignment contained
            #binary stuff.
            
            #try:
            #    PCA_domenico_new(path_to_pdbs=dir_path_pca,
            #                 ref_struc_pdb_id=ref_struc_pdb_id[:-4],
            #                 chains_of_reference_struc=chains_of_ref)

            #
    
            #    #plot results as well:
            #
            #    #here we need dir_path_pca but above in PCA_domenico_new
            #    #we only need dir paths!!!!!
            #    #_plot_PCA(path_to_PCA=dir_path_pca)
            #    
            #except Exception as error:
            #    print(error)
            #    continue

In [103]:
def _get_chain_labels(dir_paths:str, ref_struc_pdb_id:str):
    
    """Helper function to extract chain label info"""
    
    parser = PDBParser(QUIET=True)
                
    prot_name = f"default"
               
    path_to_pdb = f"{dir_paths}/{ref_struc_pdb_id}"
    
    #open the correct pdb and rechain it.
    structure_template = parser.get_structure(prot_name, path_to_pdb)
    
    
    chains_str = ""
    for models in structure_template:
        for chains in models:
            chains_str += chains.id
    
    
    return chains_str

In [104]:
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/"
#all_oligomer_runs(path)

In [105]:
def write_report_gaps(report_file_loc:str, result_gaps:dict):
    """needs documentation."""
    with open(report_file_loc, "w") as fh_report:
        for keys, vals in result_gaps.items():
            fh_report.write(str(keys[0:6]))
            fh_report.write(",")
            #split list into string sep = ;
            if len(vals) == 0: #means we have no gaps
                fh_report.write("No_gap")
                fh_report.write("\n")
                continue
            #start and endgap are the positions that are still PRESENT in the structure. so first missing res is start+1
            for entries in vals: #is a tuple containing (CHAIN, STARTGAP, ENDGAP, LEN GAP)
                for k in entries:
                    fh_report.write(str(k)+" ")  #k = entry of tuple
                fh_report.write(";")    
            fh_report.write("\n")

# Mutational analysis functions.

In [106]:
def read_mutations_for_mapping(directory_to_search:str, prot_name:str):
    
    """NEEDS TO BE WORKED ON / IS JUST A TEMPLATE NOW."""
    
    path="/home/micnag/bioinformatics/mutational_collection_cosmic/mutations/overall_mutations"
    
    #fetches the primary gene name that is used to store mutations in csv folder.
    gene_name = get_gene_name_uniprot(uniprot_id=prot_name)
    
    #print(gene_name)
    # Read all files from a directory, and read your input argument
    files = os.listdir(directory_to_search)
    
    # Sort file names by name
    files = sorted(files) 

    all_iso_forms = []
    
    #seems to work
    for file_name in files:

        if file_name.startswith(gene_name):
            all_iso_forms.append(file_name)
    
    
    #print(all_iso_forms)

    mutation_pos_count_dict = defaultdict(int)
    
    for isoform in all_iso_forms:
        mutation_iso_dict = defaultdict(int)
        with open(f"{path}/{isoform}", "r") as fh_test:
            for entries in fh_test:
                mut_list = entries.split(",")
            
    
            #this should do the trick. sorts based on position in increasing order.
            mut_sorted = sorted(mut_list, key=lambda x: int(x[1:-1]), reverse=False)
    
            #for entries in mut_sorted:
            #    print(entries)
    
            #count mutations per position
            for entries in mut_sorted:
                pos = int(entries[1:-1])
                mutation_iso_dict[pos] += 1
                
        mutation_pos_count_dict[isoform] = mutation_iso_dict
    #print(mutation_pos_count_dict)

    #for keys, vals in mutation_pos_count_dict.items():
        #print(keys, vals)
        #for isoforms, muts in keys.items():
        #    print(isoforms)
        #    print(muts)
            
    ##test purpose
    #with open("/home/micnag/bioinformatics/test/RNASET2_muts.csv", "w") as fh_out:
    #    for keys, vals in mutation_pos_count_dict.items():
    #        fh_out.write(str(keys))
    #        fh_out.write(",")
    #        fh_out.write(str(vals))
    #        fh_out.write("\n")
    
    #print(mutation_pos_count_dict)
    return mutation_pos_count_dict

In [107]:
def surr_mutations(pdb_file:str, mutation_dict:dict,  
                   outpath:str, cutoff=8, protname="default"):
    
    
    #print(mutation_dict)
    
    mutated_neighbours = defaultdict(list)
    
    
    
    for keys, vals in mutation_dict.items():
        tmp = []
        for position, freq in vals.items():
            tmp.append(position)
        
        mutated_neighbours[keys] = tmp
        
    with open(pdb_file, "r") as pdbfile1:

        # first we need to make extract all atoms from our pdb file.
        
        parser = PDBParser(QUIET=True)
        
        structure = parser.get_structure(protname, pdbfile1)

        # Selection.unfold_entities(<structure object>, <level of information that you want>)
        # other levels are "C" for chain, "R" for residue, "A" for atom and so on.
        

        atom_list = Bio.PDB.Selection.unfold_entities(structure, "A")
        
        # lets get the coordinates of all atoms now
        
        #for each atom we store his parent residue
        atom_coords = [(atom.get_coord(), atom.get_parent()) for atom in atom_list]
        
        for atoms in atom_list:
            print(atoms.get_id())
        # we provide as argument here the Selection.unfold.entities object which has all atoms.
        ns = Bio.PDB.NeighborSearch(atom_list)  # this class object has the .search() method defined in its __init__
        
        
        
        #here we store all neighbours
        general_neighbours = defaultdict(list)
        
        for atoms in atom_coords:
            
            f'''For each atom we will make a search for all surrounding atoms that are within {cutoff} A radius.'''
            parent_res = atoms[1].get_id()[1]  #returns the residue number.
            
            proximal_atoms = ns.search(atoms[0], 8, "R")
            
            # I SET HERE search for atoms[0] because atoms is a tuple containing of coordinates
            # and parent residue name see line 75 + 76 #print(atom_coords[0])

            f"""Synthax: ns.search(<target object>, <Cutoff to be searched for>, 
            <type of information level that should be returned>
            R means we dont want the single atoms that are within {cutoff}A 
            found but instead their corresponding residues. For all atoms we would set <A> instead of <R>"""

            # this function searches through a target (in our case each atom as we loop through all available atoms)
            # and returns a list with all atoms within specified atoms .
            
                
            for residues in proximal_atoms:  # we go through all residues that were found within cutoff A
            
                id_x = residues.get_id()[1]
                # get_id gives us a tuple with shape ("", "residue number", "optinal flag").
                # Out of this tuple we want the residue id which is [1]
                # we only want residues that we dont have already in the list.
                # Makes no sense to add stuff that is already in there
                general_neighbours[parent_res].append(id_x)
                
        
        # if we have all we append the whole list to the dictionary. we take the atoms parent residue name as a key.
        non_redundant_neighbours = defaultdict(int)
        
        
        for keys, vals in general_neighbours.items():
            unique_hits = list(set([x for x in vals if x != keys]))
            
            non_redundant_neighbours[keys] = unique_hits
        
        
        
        #now lets check how many mutations are within them.
        
        for isoform, mutations in mutated_neighbours.items():
            print(isoform)
            print(mutations)
            with open(f"{outpath}/{isoform[:-4]}_mutated_surroundings.tsv", "w") as fh_out:
                for position, neighbours in non_redundant_neighbours.items():
                
                    #make sets of both (just to use intersection)
                    neighbours = set(neighbours)
                    mutations = set(mutations)
                    
                    shared_mut_pos = neighbours.intersection(mutations)
                    shared_mut_lst = list(sorted(list(shared_mut_pos)))
                    print(position)
                    print(shared_mut_lst)
                    
                    fh_out.write(str(position))
                    fh_out.write("\t")
                    for hits in shared_mut_lst:
                        
                        fh_out.write(str(hits))
                        fh_out.write(" ")
                    
                    fh_out.write("\t")
                    fh_out.write(str(len(shared_mut_lst)))
                    fh_out.write("\n")

In [108]:
def surr_mutations_calpha_1(pdb_file:str, mutation_dict:dict,  
                   outpath:str, NMA_info:dict, cutoff=8, protname="default"):
    
    #print(mutation_dict)
    mutated_neighbours = defaultdict(list)
    
    for keys, vals in mutation_dict.items():
        tmp = []
        for position, freq in vals.items():
            tmp.append(position)
        
        mutated_neighbours[keys] = tmp
        
    
    #print(mutated_neighbours)
    with open(pdb_file, "r") as pdbfile1:

        # first we need to make extract all atoms from our pdb file.
        parser = PDBParser(QUIET=True)
        
        structure = parser.get_structure(protname, pdbfile1)

        # Selection.unfold_entities(<structure object>, <level of information that you want>)
        # other levels are "C" for chain, "R" for residue, "A" for atom and so on
        atom_list = Bio.PDB.Selection.unfold_entities(structure, "A")
        
        # lets get the coordinates of all atoms now
        #only CALPHA
        atom_coords = [atom for atom in atom_list if atom.get_id() == "CA"]
    
        # we provide as argument here the Selection.unfold.entities object which has all atoms.
        ns = Bio.PDB.NeighborSearch(atom_coords)  # this class object has the .search() method defined in its __init__
        
        #here we store all neighbours
        general_neighbours = defaultdict(list)
        
        for atoms in atom_coords:
            
            f'''For each atom we will make a search for all surrounding atoms that are within {cutoff} A radius.'''
            parent_res = atoms.get_parent().get_id()[1]  #returns the residue number.
            
            proximal_atoms = ns.search(atoms.get_coord(), 8, "A")
            
            # I SET HERE search for atoms[0] because atoms is a tuple containing of coordinates
            # and parent residue name see line 75 + 76 #print(atom_coords[0])

            f"""Synthax: ns.search(<target object>, <Cutoff to be searched for>, 
            <type of information level that should be returned>
            R means we dont want the single atoms that are within {cutoff}A 
            found but instead their corresponding residues. For all atoms we would set <A> instead of <R>"""

            # this function searches through a target (in our case each atom as we loop through all available atoms)
            # and returns a list with all atoms within specified atoms .
            
            for atms in proximal_atoms:  # we go through all residues that were found within cutoff A
                
                id_x = atms.get_id()
                # get_id gives us a tuple with shape ("", "residue number", "optinal flag").
                # Out of this tuple we want the residue id which is [1]
                # we only want residues that we dont have already in the list.
                # Makes no sense to add stuff that is already in there
                if id_x == "CA":
                    general_neighbours[parent_res].append(atms.get_parent().get_id()[1])
                
        
        # if we have all we append the whole list to the dictionary. we take the atoms parent residue name as a key.
        non_redundant_neighbours = defaultdict(int)
        
        for keys, vals in general_neighbours.items():
            unique_hits = list(set([x for x in vals if x != keys]))
            
            non_redundant_neighbours[keys] = unique_hits
        
        
        #now lets check how many mutations are within them.
        
        for isoform, mutations in mutated_neighbours.items():
            #print(mutations)
            
            if NMA_info:
                
                NMA_dict = defaultdict()
                NMA_scores = [x[1] for x in NMA_info] # we grab the scores knowing that it contains for each residue from start to finish a score.
                #these scores are RENUMBERED FROM POS 1 even if POS 1 in the protein has resnum 134 e.g
            else:
                NMA_scores = []
            
            
            
            df_mutation = pd.DataFrame()
            
                
            #will contain all the neighbouring CA of each position as a list of lists.
            neighbour_list = []
            #these are the res nums of the CAs
            prot_idx = []
            
            #surrounding mutated res list
            mutated_res_neighbours = []
            
            #number of mutated neighbours
            num_mutated_neighbours = []
            
            #NMA score list
            NMA_score_attached_vals = []
            
            
            i = 0 
            for position, neighbours in non_redundant_neighbours.items():
                
                #make sets of both (just to use intersection)
                
                neighbours = set(neighbours)
                
                neighbour_list.append(list(neighbours))
                prot_idx.append(position)
            
                mutations = set(mutations)
                
                shared_mut_pos = neighbours.intersection(mutations)
                shared_mut_lst = list(sorted(list(shared_mut_pos)))
                
                mutated_res_neighbours.append(shared_mut_lst)
                
                num_mutated_neighbours.append(len(shared_mut_lst))
                
                if len(NMA_scores) != 0:
                    
                    NMA_score_attached_vals.append(NMA_scores[i])
                    
                    i += 1
                    
                else:
                    
                    NMA_score_attached_vals.append(np.NaN)
                    i += 1
            
            #print(neighbour_list)
            
            df_mutation["neighbour_CAs"] = [x for x in list(neighbour_list)]
            df_mutation["mutated_neighbours_CAs"] = [x for x in list(mutated_res_neighbours)] if len(mutated_res_neighbours) > 0 else np.NaN
            df_mutation["num_mutated_neighbours"] = [int(x) for x in list(num_mutated_neighbours)]
            df_mutation["NMA_scores"] = NMA_score_attached_vals
            df_mutation.index = prot_idx
            #print(pdb_file)
            df_mutation.to_csv(outpath, sep="\t")
            #print(df_mutation.head())
            
            

In [109]:
def run_mutational_mapping_new(dir_to_proteins:str, main_prot_name:str, include_NMA=True):
    
     #good candidate to put all of them into 1 function. Next time wrap it up.
    """mutational mapping part"""
    
    try:
        os.mkdir(f"{dir_to_proteins}/Mutational_mapping")
    
    except Exception as error:
        #print(error)
        pass
    
    #this dir path needs to be hardcoded because it is constant in every case.4
    dir_to_read = "/home/micnag/bioinformatics/mutational_collection_cosmic/mutations/overall_mutations"
    
    #path = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/{main_prot_name}"

    if include_NMA:
        
        NMA_dir = f"{dir_to_proteins}/NMA"
        try:
            dirs_to_check = [d for d in os.listdir(NMA_dir) if os.path.isdir(os.path.join(NMA_dir, d))]
            
            #print(dirs_to_check)
        except Exception as error:
            print(error)
        
        NMA_dict = defaultdict()
        
        for dirs in dirs_to_check:
            path_to_check = f"{dirs[:-4]}.ca.pdb_bfactor_res.txt"
            try:
                #we append a dic as val to key from NMA_dict
                NMA_scores_per_struc = []
                
                with open(f"{NMA_dir}/{dirs}/{path_to_check}", "r") as fh_nma_in:
                    for lines in fh_nma_in:
                        lines = lines.split()
                        pos, score = lines[0], lines[1]
                        NMA_scores_per_struc.append((int(pos), float(score)))
                    
                    #we append the new dict to a global dict.
                    NMA_dict[dirs] = NMA_scores_per_struc
                        
            except Exception as error:
                #print(f"we could not open {NMA_dir}/{dirs}/{path_to_check}")
                continue
        
        
    #for strucs, combo_scores in NMA_dict.items():
    #    print(strucs)
    #    print(combo_scores)
    
    
    
    all_file_input = [f for f in os.listdir(dir_to_proteins) if os.path.isfile(os.path.join(dir_to_proteins, f))]
    #grab only pdbs
    pdb_mapping_inputs = [f for f in all_file_input if f[-4:] == ".pdb"]
    
    #grab corresponding mutational dictionary for each gene
    mut_dictionary = read_mutations_for_mapping(directory_to_search=dir_to_read, prot_name=main_prot_name)
    
    #now lets map on each structure for each oligomer.
    #check if there are pdb structures to map
    if len(pdb_mapping_inputs) != 0:
        #if yes map for each position and oligomer.
        for pdb_files in pdb_mapping_inputs:
            
            
            if include_NMA:
                try:
                    NMA_info = NMA_dict[pdb_files]
                except:
                    NMA_info = False
            else:
                NMA_info = False
            
            
            #only CA-based. 8 A cutoff.
            
            pdb_file_path = f"{dir_to_proteins}/{pdb_files}"
            
            surr_mutations_calpha_1(pdb_file=pdb_file_path, mutation_dict=mut_dictionary, protname=main_prot_name,  
                      outpath=f"{dir_to_proteins}/Mutational_mapping/{pdb_files[:-4]}.tsv", cutoff=8,
                                 NMA_info=NMA_info)
            
    

In [110]:
#test new block mutational mapping

#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN/monomer/pos_1_181"
#run_mutational_mapping_new(dir_to_proteins=path, main_prot_name="NUD4B_HUMAN", include_NMA=True)

In [111]:
#testing mutational mapping block.

#dir_to_read = "/home/micnag/bioinformatics/mutational_collection_cosmic/mutations/overall_mutations"

#pdb_file = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/mutational_mapping/3w5b.pdb"

#outpath = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/mutational_mapping"

#mut_dictionary = read_mutations_for_mapping(directory_to_search=dir_to_read, prot_name="AT2A1_HUMAN")
#surr_mutations_calpha(pdb_file=pdb_file, mutation_dict=mut_dictionary, protname="AT2A1_HUMAN",  
#                   outpath=outpath, cutoff=8)
#mutational_freq_hist(mut_dictionary)

In [112]:
def mutational_freq_hist(mutational_dict:dict):
    
    #print(mutational_dict)
    
    tmp_dict = defaultdict()
    
    for keys, vals in mutational_dict.items():
        tmp_list = []
        for key, val in vals.items():
            tmp_list.append((key,val))
        tmp_sort = sorted(tmp_list, key=lambda x: x[1], reverse=True)
        tmp_dict[keys] = tmp_sort
    

    all_isoform_dicts = defaultdict(list)
    only_muts = defaultdict(list)
    
    for keys, vals in tmp_dict.items():
        min_pos = 1
        max_pos = max([x[0] for x in vals])
        
        for (pos, mut) in vals:
            all_isoform_dicts[keys].append((pos, mut))
            only_muts[keys].append((pos, mut))
            
        for i in range(min_pos, max_pos):
            if i not in [x[0] for x in vals]:
                all_isoform_dicts[keys].append((i, 0))
    
    
    overlap_mutations = defaultdict(int)
    
    for keys, vals in only_muts.items():
        
        for muts in vals:
            
            overlap_mutations[muts[0]] += muts[1]
    
    
    
    shared_list = []
    
    #for keys, vals in all_isoform_dicts.items():
    #    #setup min max to find the boundaries for our mutations on the protein.
    #    min_pos = min([int(x[1]) for x in vals])
    #    max_pos = max([int(x[1]) for x in vals])
    #    #width = (max_pos-min_pos)
    #
    #    mutations_list = [x[1] for x in vals]
    #    avg_mut = np.average(mutations_list)
    #    #print(min_pos, max_pos, avg_mut)
    #    shared_list.append([x for x in vals])
        
    #print(shared_list)
        
    #result = set(shared_list[0]).intersection(*shared_list)
    #print(result)


    """MIGHT BE NEEDED LATER"""
    fig = plt.figure(figsize=(100, 50))
    ax = fig.add_subplot(111)
    
    #threshold_avg = avg_mut
    #threshold_1_sd = avg_mut + 1*np.std(mutations_list)
    #threshold_2_sd = avg_mut + 2*np.std(mutations_list)
    #threshold_3_sd = avg_mut + 3*np.std(mutations_list)
    
    
    #below_avg = []
    #sd_1 = []
    #sd_2 = []
    #sd_3 = []
    
    #for keys, vals in mutational_dict.items():
    #    if vals > threshold_3_sd:
    #        sd_3.append((keys, vals))
    #        continue
    #    if vals > threshold_2_sd:
    #        sd_2.append((keys, vals))
    #        continue
    #    if vals > threshold_1_sd:
    #        sd_1.append((keys, vals))
    #        continue
    #    if vals < threshold_avg:
    #        below_avg.append((keys, vals))
    #        continue
    #
    #x_below_avg = [x[0] for x in below_avg]
    #y_below_avg = [x[1] for x in below_avg]
    #
    #x_sd_1 = [x[0] for x in sd_1]
    #y_sd_1 = [x[1] for x in sd_1]
    #
    #x_sd_2 = [x[0] for x in sd_2]
    #y_sd_2 = [x[1] for x in sd_2]
    #
    #x_sd_3 = [x[0] for x in sd_3]
    #y_sd_3 = [x[1] for x in sd_3]
    
    #ax.bar(x_below_avg,y_below_avg, color = 'grey', width=1,label='Below avg')
    #ax.bar(x_sd_1,y_sd_1, color = 'yellow', width=1, label='Above 1 SD') 
    #ax.bar(x_sd_2,y_sd_2, color = 'orange', width=1, label='Above 2 SD')
    #ax.bar(x_sd_3,y_sd_3, color = 'red', label='Above 3 SD')
    
    ax.bar(overlap_mutations.keys(), overlap_mutations.values())
    
    #ax.bar(mutational_dict.keys(), mutational_dict.values(), color=["red", "blue"])
    #ax.set_title(f"Number of mutations in 8A vicinity", {'fontsize': 120})
    #ax.set_xlabel('Position', fontsize=80)
    #ax.set_ylabel('Mutations in Neighbourhood [Mutations / 8A surroundings]', fontsize=80)
    #this does not take into account if the protein is actually longer.  needs improvement if this is important.
    #ax.hlines(avg_mut, xmin=0, xmax=max_pos, color="grey", linestyles="dotted",linewidth=3)
    #ax.hlines(avg_mut+2*np.std(mutations_list), xmin=0, xmax=max_pos, color="r", linestyles="dashed",linewidth=4)
    #ax.hlines(avg_mut+3*np.std(mutations_list), xmin=0, xmax=max_pos, color="r", linestyles="dashed",linewidth=5)
    #ax.tick_params(labelsize=60)
    #ax.xaxis.set_ticks(np.arange(0, max_pos, 10))
    #ax.legend(fontsize=80, borderpad=2)
    

In [113]:
#dir_to_read = "/home/micnag/bioinformatics/mutational_collection_cosmic/mutations/overall_mutations"

#mut_dictionary = read_mutations_for_mapping(directory_to_search=dir_to_read, prot_name="RNT2_HUMAN")
#mutational_freq_hist(mut_dictionary)

In [114]:
#function to cluster PCA automatically


def PCA_clustering(path_to_PCA:str):
    
    """Function will take in the PCA data and perform affinity propagation clustering."""
    
    
    expl_var = f"{path_to_PCA}/pc_variances.txt"
    pca = f"{path_to_PCA}/exp_ensemble_proj_PC.txt"
    
    
    #centers = [[1, 1], [-1, -1], [1, -1]]
    #X, labels_true = make_blobs(
    #        n_samples=300, centers=centers, cluster_std=0.5, random_state=0)
    
    
    #read in the first 2 PC from PCA file.
    pca_array = np.loadtxt(pca, dtype="float",
                           usecols=[0,1])
    
    pca_mean = np.mean(pca_array)
    pca_std = np.std(pca_array)
    print(pca_mean)
    print(pca_std)
    
    pca_normalized = (pca_array-pca_mean)/pca_std
    
    #print(pca_normalized)
    #normalize array
    #pca_array_normalized = stats.zscore(pca_array, axis=None)
    
    
    
    af = AffinityPropagation(random_state=0,max_iter=1000,damping=0.7).fit(pca_normalized)
    
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_
    
    n_clusters_ = len(cluster_centers_indices)
    
    
    print(cluster_centers_indices)
    print(labels)
    
    cnt_dict = defaultdict()
    
    hits = set(labels)
    
    for x in hits:
        cnt_dict[x] = 0
        
    for x in labels:
        cnt_dict[x] += 1
        
    
    highest_groups = []
    for keys, vals in cnt_dict.items():
        highest_groups.append((keys, vals))
        
    
    highest_sort = sorted(highest_groups, key=lambda x : x[1], reverse=True)
    
    print(highest_sort)
    #print("Estimated number of clusters: %d" % n_clusters_)
    #print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
    #print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
    #print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
    #print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
    #print(
    #    "Adjusted Mutual Information: %0.3f"
    #    % metrics.adjusted_mutual_info_score(labels_true, labels)
    #)
    #print(
    #    "Silhouette Coefficient: %0.3f"
    #    % metrics.silhouette_score(X, labels, metric="sqeuclidean")
    #)

    
    plt.close("all")
    plt.figure(1)
    plt.clf()
    
    colors = plt.cycler("color", plt.cm.viridis(np.linspace(0, 1, 4)))
    
    for k, col in zip(range(n_clusters_), colors):
        class_members = labels == k
        cluster_center = pca_array[cluster_centers_indices[k]]
        plt.scatter(
            pca_array[class_members, 0], pca_array[class_members, 1], color=col["color"], marker="."
        )
        plt.scatter(
            cluster_center[0], cluster_center[1], s=14, color=col["color"], marker="o"
        )
        for x in pca_array[class_members]:
            plt.plot(
                [cluster_center[0], x[0]], [cluster_center[1], x[1]], color=col["color"]
            )
    
    plt.title("Estimated number of clusters: %d" % n_clusters_)
    plt.show()


In [115]:
#path_to_PCA = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/PCA"
#PCA_clustering(path_to_PCA=path_to_PCA)

In [116]:
def PCA_clustering_2(path_to_PCA:str):
    
    """Function will take in the PCA data and perform affinity propagation clustering."""
    
    
    expl_var = f"{path_to_PCA}/pc_variances.txt"
    pca = f"{path_to_PCA}/exp_ensemble_proj_PC.txt"
    
    
    np.random.seed(0)
    #centers = [[1, 1], [-1, -1], [1, -1]]
    #X, labels_true = make_blobs(
    #        n_samples=300, centers=centers, cluster_std=0.5, random_state=0)
    
    
    #read in the first 2 PC from PCA file.
    pca_array = np.loadtxt(pca, dtype="float",
                           usecols=[0,1])
    
    pca_mean = np.mean(pca_array)
    pca_std = np.std(pca_array)
    
    #normalize
    pca_normalized = (pca_array-pca_mean)/pca_std
    
    
    clust = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.05)
    
    # Run the fit
    clust.fit(pca_normalized)
    labels_050 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=0.5)
    
    labels_200 = cluster_optics_dbscan(
    reachability=clust.reachability_,
    core_distances=clust.core_distances_,
    ordering=clust.ordering_,
    eps=2)

    space = np.arange(len(pca_array))
    reachability = clust.reachability_[clust.ordering_]
    labels = clust.labels_[clust.ordering_]
    
    print("these are the number of conformers we found")
    print(len(set([x for x in labels if x >= 0])))
    print(set(labels))
    
    
    plt.figure(figsize=(10, 7))
    G = gridspec.GridSpec(2, 3)
    ax1 = plt.subplot(G[0, :])
    ax2 = plt.subplot(G[1, 0])
    ax3 = plt.subplot(G[1, 1])
    ax4 = plt.subplot(G[1, 2])
    
    # Reachability plot
    colors = ["g.", "r.", "b.", "y.", "c."]
    for klass, color in zip(range(0, 5), colors):
        Xk = space[labels == klass]
        Rk = reachability[labels == klass]
        ax1.plot(Xk, Rk, color, alpha=0.3)
    ax1.plot(space[labels == -1], reachability[labels == -1], "k.", alpha=0.3)
    ax1.plot(space, np.full_like(space, 2.0, dtype=float), "k-", alpha=0.5)
    ax1.plot(space, np.full_like(space, 0.5, dtype=float), "k-.", alpha=0.5)
    ax1.set_ylabel("Reachability (epsilon distance)")
    ax1.set_title("Reachability Plot")
    
    # OPTICS
    colors = ["g.", "r.", "b.", "y.", "c."]
    for klass, color in zip(range(0, 5), colors):
        Xk = pca_array[clust.labels_ == klass]
        ax2.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
    ax2.plot(pca_array[clust.labels_ == -1, 0], pca_array[clust.labels_ == -1, 1], "k+", alpha=0.1)
    ax2.set_title("Automatic Clustering\nOPTICS")
    
    # DBSCAN at 0.5
    colors = ["g.", "r.", "b.", "c.", "y.", "m."]
    for klass, color in zip(range(0, 4), colors):
        Xk = pca_array[labels_050 == klass]
        ax3.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
    ax3.plot(pca_array[labels_050 == -1, 0], pca_array[labels_050 == -1, 1], "k+", alpha=0.1)
    ax3.set_title("Clustering at 0.5 epsilon cut\nDBSCAN")
    
    # DBSCAN at 2.
    colors = ["g.", "m.", "y.", "c."]
    for klass, color in zip(range(0, 4), colors):
        Xk = pca_array[labels_200 == klass]
        ax4.plot(Xk[:, 0], Xk[:, 1], color, alpha=0.3)
    ax4.plot(pca_array[labels_200 == -1, 0], pca_array[labels_200 == -1, 1], "k+", alpha=0.1)
    ax4.set_title("Clustering at 2.0 epsilon cut\nDBSCAN")
    
    plt.tight_layout()
    plt.show()
    

In [117]:
#path_to_PCA = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994/PCA"
#PCA_clustering_2(path_to_PCA=path_to_PCA)

In [118]:
def mutational_upsetplot(mutational_dict:dict, save_file=True):
    
    #print(mutational_dict)
    
    tmp_dict = defaultdict()
    
    isoforms_names = []
    for keys, vals in mutational_dict.items():
        tmp_list = []
        #store names of isoforms.
        if not isoforms_names:
            isoforms_names.append(keys)
        for key, val in vals.items():
            tmp_list.append((keys,key,val))
        tmp_sort = sorted(tmp_list, key=lambda x: x[1], reverse=True)
        tmp_dict[keys] = tmp_sort
        
        
        
    all_isoform_dicts = defaultdict(list)
    only_muts = defaultdict(list)
    
    for keys, vals in tmp_dict.items():
        min_pos = 1
        max_pos = max([x[1] for x in vals])
        
        for (isoform, pos, mut) in vals:
            all_isoform_dicts[keys].append((isoform[:-4],pos, mut))
            only_muts[keys].append((pos, mut))
            
        for i in range(min_pos, max_pos):
            if i not in [x[0] for x in vals]:
                all_isoform_dicts[keys].append((isoform[:-4], i, 0))
    

    tmp_ = []
    for keys, vals in all_isoform_dicts.items():
        for x in vals:
            tmp_.append(x)
    
    
    sorted_full_isoforms = sorted(tmp_, key=lambda x: x[1], reverse=False)
    
    max_val = sorted_full_isoforms[-1][1]
      
    muts = defaultdict(int)
    isoforms = defaultdict(list)
    
    for entries in sorted_full_isoforms:
        if entries[2] == 0:
            continue
        muts[(entries[1])] += entries[2]
        isoforms[str(entries[1])].append(entries[0])
        
    datalst = []
    members = []
    
    for i in range(1, max_pos):
        datalst.append(muts[i])
        members.append(isoforms[str(i)])

    #mut_freq_overlap = from_memberships(members[0:5], data=data[0:5])
    
    example = from_memberships(
    members, data=datalst)
    
    
    
    UpSet.plot(example, subset_size="sum", orientation="horizontal",
        sort_by="degree", sort_categories_by="cardinality",
        facecolor="navy",shading_color="lightgray",
        show_counts=True,other_dots_color=.2)
    
    title = min(isoforms_names)[:-4]
    plt.suptitle(title)
    
    #if save_file:
        #plt.savefig("/home/micnag/bioinformatics/test/myplot.pdf") 

In [119]:
def select_c_alpha(path:str):
    
    
    #sel only c_alpha
    class C_alpha_only(Select):
        def __init__(self, *args):
            super().__init__(*args)
            
        #overload accept_residue inherited from Select with this conditional return
        def accept_atom(self, atom):
            return 1 if atom.id == "CA" else 0
        
        #overloaded to only accept positive residue numbering.
        def accept_residue(self, residue):      
            return 1 if residue.id[1] > 0 else 0    
        
    lst =  [('VAL',"V"), ('ILE',"I"), ('LEU',"L"), ('GLU',"E"), ('GLN',"Q"),
                    ('ASP',"D"), ('ASN',"N"), ('HIS',"H"), ('TRP',"W"), ('PHE',"F"), ('TYR',"Y"), 
                    ('ARG',"R"), ('LYS',"K"), ('SER',"S"), ('THR',"T"), ('MET',"M"), ('ALA',"A"), 
                    ('GLY',"G"), ('PRO',"P"), ('CYS',"C")]
    
    canonical_aas = defaultdict(lambda: "X", lst)
    #filelst    path
    #5ltu_A.pdb /home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN
    
    parser = PDBParser(QUIET=True)
    
    prot_name = f"default"
    
    #print("we are here")
    #print(fullpath)
    structure = parser.get_structure(prot_name, path)
    
    
    io = PDBIO()
            
    io.set_structure(structure)
    
    #savepath needs to be changed.
    io.save(path, C_alpha_only())
    
    #after save we reload it and then WE COUNT.
    
    parser = PDBParser(QUIET=True)
    
    prot_name = f"default"
    
    #print("we are here")
    #print(fullpath)
    

    #here we grab sequence.
    structure = parser.get_structure(prot_name, path)
    
    struc_full = [canonical_aas[x.get_resname()] for x in structure.get_residues()]
    #print(struc_full)
    struc_full = "".join(struc_full)
    
    #here we grab ranges
    structure = parser.get_structure(prot_name, path)
    struc_range = [x.get_id()[1] for x in structure.get_residues()]
    struc_start, struc_stop = struc_range[0], struc_range[-1]
    #print(struc_start, struc_stop)
    
    return struc_start, struc_stop, struc_full
    

In [120]:
#select_c_alpha("/home/micnag/pdb_neg_test.pdb")

In [121]:
#dir_to_read = "/home/micnag/bioinformatics/mutational_collection_cosmic/mutations/overall_mutations"

#mut_dictionary = read_mutations_for_mapping(directory_to_search=dir_to_read, prot_name="RNT2_HUMAN")
#mutational_upsetplot(mut_dictionary)

# Normal mode analysis block

In [122]:
def NMA_pipeline(path:str):
    
    baselocation = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/NMA_pipeline"
        
    #we work in a clean work dir.
    try:
        os.mkdir(f"{path}/NMA")
    except Exception as error:
        print(error)
        
    #fetch all file names that we need to shuffle.
    files_to_shuffle = [f for f in os.listdir(baselocation) if os.path.isfile(os.path.join(baselocation, f))]
    
    #we move all required files to the work dir.
    for files in files_to_shuffle:
        #copy to target folder
        shutil.copy(f"{baselocation}/{files}", f"{path}/NMA")
        
    #lets start NMA for all structure
    os.chdir(f"{path}/NMA")
    
    result_files = ["evec.dat", "constants.dat", "hessian.dat"]

    complete_struc_path = f"{path}/PCA/clean_ensemble"
    
    onlyfiles = [f for f in os.listdir(complete_struc_path) if os.path.isfile(os.path.join(complete_struc_path, f)) and f[-4:] == ".pdb"]

    #set b factor to 0 otherwise this interfers with memory allocation if the columns are not separated (occ. and b fac)
    
    
    for pdbs in onlyfiles:
        print("we run now nma with")
        print(pdbs)

        #first we set b fac to 0
        _set_b_factors_0(path=f"{complete_struc_path}/{pdbs}")
        
        run_nma = f"./get_nma.sh {complete_struc_path}/{pdbs}"
        
        bash_cmd = run_nma.split()
        
        try:
            
            #lets keep it here to be save
            os.chdir(f"{path}/NMA")
            result = run(bash_cmd, stdout=PIPE, stderr=PIPE,
                universal_newlines=True)
        
            print(result.stdout)
            print(result.stderr)
            
            os.mkdir(f"{path}/NMA/{pdbs}")
            
            #we move the CA-only structure also for representation.
            
            shutil.move(f"{complete_struc_path}/{pdbs[0:4]}.ca.pdb", f"{path}/NMA/{pdbs}")
            
            #we move the result files to another dir where we work later on for visualization.
            for resultf in result_files:
                shutil.move(f"{complete_struc_path}/{resultf}", f"{path}/NMA/{pdbs}")
            
            #now we call prep nma scores to compute the b factor associated values.
            prep_nma_scores(dir_with_normal_modes=f"{path}/NMA/{pdbs}", nm=10)
            
        except Exception as error:
            print(error)


In [123]:
def prep_nma_scores(dir_with_normal_modes:str, nm=10):
    
    
    #./ptraj_evec_bfactor_cum.pl nma 1kju.ca.pdb evec.dat 10 testbfactorpdb
    pdb_list= [f for f in os.listdir(dir_with_normal_modes) if os.path.isfile(os.path.join(dir_with_normal_modes, f)) and f[-4:] == ".pdb"]
    
    #from here we grab our executables and scripts
    baselocation = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/NMA_pipeline"
    
    try:
        
        shutil.copy(f"{baselocation}/ptraj_evec_bfactor_cum.pl", f"{dir_with_normal_modes}")
        
        #so we execute the script from here.
        os.chdir(dir_with_normal_modes)
        
        run_perl = f"./ptraj_evec_bfactor_cum.pl nma {pdb_list[0]} evec.dat {nm} {pdb_list[0][:-4]}"
    
        bash_cmd = run_perl.split()
        
        with open(f"{dir_with_normal_modes}/{pdb_list[0]}_bfactor_res.txt", "w") as bfout:
        
            result = run(bash_cmd, stdout=bfout, stderr=PIPE,
                universal_newlines=True)
        
        
        
    except Exception as error:
        print(error)
    

In [124]:
#path = "/home/micnag/bioinformatics/nma_test/NMA/1kju.pdb"
#prep_nma_scores(dir_with_normal_modes=path)

In [125]:
#path = "/home/micnag/bioinformatics/nma_test"

#NMA_pipeline(path=path)

# Main function body

In [164]:
def fetch_struc(query:str, templates:str, seq_sim:str, query_start:str, query_end:str,
                temp_start:str, temp_end:str,
                run_NMA:bool, run_PCA:bool)-> None:
    
    ''' Input: dictionary containing:
        
        Key: Gene_name / Uniprot ID
        
        Values: [0]: All rcsb ids of homologs and their respective chains e.g 8DGD_A for chain A of 8DGD
                [1]: Seq similarity [0,1]
                
        We parse through all mmseq2 hit rcsb ids and retrieve associated structures.
        First retrieved structure will be used as REFERENCE structure.
        This makes sense because mmseqs2 hits are sorted by seq similarity and highest seq
        similarity is listed first (assuming this seq similarity hit corresponds to the best 
        structural similarity hit).
        Then we select the correct chain and compute RMSD / TM score for each respective pair against
        the Reference structue. We use the first entry corresponding to the highest seq similarity as template to align other structures against.
    '''
    
    
    #go through all steps and check logic and redundancy.
    
    uniprot_id = query
    #Used to store protein name for matching with cosmic_results later downstream 
    main_prot_name = get_gene_name_uniprot(query) 
    
    #domain dict will be used to create sub directories later.
    domain_dict = _split_domains(templates, temp_start, temp_end, query_start, query_end)
    
    #print("this is the output of split domains.")
    #print(domain_dict)
    
    
    print(main_prot_name)
    
    path = f"/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/{main_prot_name}"
    
    #here we store the protein original fasta
    main_prot_seq = get_gene_fasta(query)
    
    #print(main_prot_seq)
    
    #setup directories and makes batch download pdb input file
    chain_dict = setup_directories_prepare_input_files(gene_name = main_prot_name,
                                          prot_fasta = main_prot_seq,
                                          domain_dict = domain_dict)
    
    

    #print(f"we work currently in the directory: {main_prot_name}")
    print("we start download")
    #we download both pdbs as well as mmcif files here.
    batch_download_pdbs(gene_name=main_prot_name)
    
    print("we start shifts")
    #correct eventual shifts
    shift_dict = new_get_shifts(pdbfolder=path)
    
    #make new dirs within gene directory for higher oligomers if available.

    #get_single_chains_pdbs(pdb_dict=chain_dict, path=path, 
    #                       domain_dict=domain_dict, shift_dict=shift_dict)
    
    print("we start renumber whole strucs")
    #try to renumber whole structures before building oligomers
    renumber_whole_structures(shift_dict=shift_dict, path=path)
    
    
    #builds biological assemblies from asymmetric units
    oligostates = get_biological_assemblies_atomium(path=path, 
                                            gene_name=main_prot_name,
                                            main_iso_seq=main_prot_seq,
                                            main_protein_seq=main_prot_seq)
    
    #print(oligostates)
    
    print("start make_oligo_dirs")
    
    #lets make the required directories.
    make_oligo_dirs(domain_dict, oligostates, path, domain_dict)
    
    
    
    
    #prepare templates for each domain and each oligomeric state respectively.
    template_dict = prepare_references(
                       main_gene_name=main_prot_name,
                       templates=templates,
                       seq_sim = seq_sim,
                       query_start=query_start,
                       query_end=query_end,
                       temp_start =temp_start,
                       temp_end =temp_end,
                       path=path, oligodict=oligostates)  

    #compute tm scores and rmsd
    
    
    
    min_seq_len_dict, templates_new_dict = USAlign(templates=template_dict, gene_name=main_prot_name, path=path, report=True)
    
    print("this is new templates dict:")
    print(f"{templates_new_dict}")
    #now we run the PCA part.
    
    """DO A MAJORITY VOTE TO DECIDE ON THE RELEVANT OLIGOMERIC STATE. WE ONLY CARRY ON THERE FROM HERE On."""
    
    
    
    #oligo_state_to_check = path/to/most/structures
    oligo_state_to_check, template = _relevant_oligomers(path=path, templates_new_dict=templates_new_dict)
    # FUNCTION CALL (ALL_DIRS_WITH_STRUCS):
    # RETURN : MAJORITY CLASS 
    # CONTINUE with majority class ONLY.
    
    print("This is most common state:")
    print(oligo_state_to_check)
    print("this is template")
    print(template)

    #check first for conservation

    # steps for conservation

    try:
        #run mmseq to find homolog sequences

        print("we start mmseq now!")
        mmseq_fasta_result = mmseq_multi_fasta(uniprot_id=uniprot_id, outdir=oligo_state_to_check)
        #get 3 different conservation scores in a pandas df.
        conserv_df = get_conservation(path_to_msa=mmseq_fasta_result)

        print("we got a df!")
        conserv_df.to_csv(f"{oligo_state_to_check}/conservation_df.csv")
    except Exception as error:
        print(error)
    
        

    
    if run_PCA:
        
        run_pca_for_all_oligomers(path=oligo_state_to_check, 
                              template=template,
                              main_prot_seq=main_prot_seq, 
                              prot_name=main_prot_name)
                             
    if run_NMA:
        
        #first we compute the normal modes
        #NMA_pipeline(path=oligo_state_to_check)
        pass
    #all_oligomer_runs(path=path, min_seq_len_dict=min_seq_len_dict)
    
    """Lets run the mutational mapping block from here.
    
        Lets gather all mutations first and merge them together."""


    #step 1 gather mutations from cosmic:

    cosmic_df = get_cosmic_mutations(gene_name=main_prot_name) 
    
    try:
        cosmic_df.to_csv(f"{path}/cosmic_mutations.csv")
    except Exception as error:
        print(error)
        
    #step 2 map gnomad mutations.

    gnomad_table_path = map_gnomad(Gene_name=main_prot_name, outpath=oligo_state_to_check)

    #step 3 extract mutations from gnomad
    gnomad_df, gnomad_mutation_dict = gnomad_to_pandas(Gene_name=main_prot_name,
                 path_to_tsv=gnomad_table_path, 
                 fasta_seq=main_prot_seq)

    try:
        gnomad_df.to_csv(f"{path}/gnomad_mutations.csv")
    except Exception as error:
        print(error)
    
    #step 4 gather mutations from clinvar
    clinvar_df = map_clinvar(Gene_name=main_prot_name)

    """
    Gene_name:str, clinvar_df:pd.DataFrame,
                         gnomad_mut_dict:dict,
                         clinvar_mapped_df_path:str)
    """

    clinvar_map_outpath = f"{path}/clinvar_intermediate.csv"


    try:
        clinvar_df.to_csv(clinvar_map_outpath)
        print(clinvar_df.shape)
        
    except Exception as error:
        print(error)
        
    # Search for mutations in gnomad if they are present so we obtain info about allele freq ect.

    #set this so we dont get error if we encounter try except issues.
    updated_clinvar_df = None
    cbioport_df = None

    try:
        list_to_be_searched, clinvar_df = map_clinvar_to_gnomad(Gene_name=main_prot_name, clinvar_df=clinvar_df,
                         gnomad_mut_dict=gnomad_mutation_dict, clinvar_mapped_df_path=clinvar_map_outpath)

    except Exception as error:
        # Update clinvar muts that were found 1 step before.
        updated_clinvar_df = update_clinvar_muts_based_on_gnomad(clinvar_df=clinvar_df, gnomad_dict=gnomad_mutation_dict)
    
    try:
        updated_clinvar_df.to_csv(f"{path}/clinvar_mutations.csv")
    except Exception as error:
        print(error)
    
    try:
        gene_name = get_hugo_name(uniprot_id)
        print(f"This is gene name in hugo: {gene_name}")
        cbioport_df = get_cbioportal_info(gene_name=gene_name)

        cbioport_df.to_csv(f"{path}/cbioport_mutations.csv")
    except Exception as error:
        print(error)

    try:
        print(f"This is cosmic df: {cosmic_df.shape}")
    except Exception as error:
        print(error)
    try:
        print(f"This is updated_clinvar df: {updated_clinvar_df.shape}")
    except Exception as error:
        print(error)
    try:
        print(f"This is gnomad df: {gnomad_df.shape}")
    except Exception as error:
        print(error)
    try:
        print(f"This is bioportal df: {cbioport_df.shape}")
    except Exception as error:
        print(error)

In [127]:
#df_cosmic = get_cosmic_mutations(gene_name="NUDT4B")

In [128]:
#df_cosmic

# MUTATIONAL MAPPING NEW

In [129]:
def get_cosmic_mutations(gene_name:str):

    path = "/home/micnag/bioinformatics/cosmic/Cosmic_GenomeScreensMutant_v98_GRCh38.tsv"
    
    usecols=['GENE_SYMBOL',
         'MUTATION_AA', 'MUTATION_DESCRIPTION',
       'MUTATION_ZYGOSITY', 'LOH', 'CHROMOSOME', 'GENOME_START', 'GENOME_STOP']

    df = dd.read_csv(path, sep="\t", dtype={'CHROMOSOME': 'object',
       'MUTATION_ZYGOSITY': 'object', 'GENOME_START': 'float64',
       'GENOME_STOP': 'float64',
       'LOH': 'object'}, usecols=usecols)

    #we need to switch these tuples and then map the 1letter aa code to 3letter aa for later compatibility.
    lst =  [('Val',"V"), ('Ile',"I"), ('Leu',"L"), ('Glu',"E"), ('Gln',"Q"),
                    ('Asp',"D"), ('Asn',"N"), ('His',"H"), ('Trp',"W"), ('Phe',"F"), ('Tyr',"Y"), 
                    ('Arg',"R"), ('Lys',"K"), ('Ser',"S"), ('Thr',"T"), ('Met',"M"), ('Ala',"A"), 
                    ('Gly',"G"), ('Pro',"P"), ('Cys',"C")]

    lst = [(y, x) for x, y in lst]

    canonical_aas = defaultdict(lambda: "X", lst)

    df_re = df[df["MUTATION_DESCRIPTION"].str.contains("missense")]
    
    df_re = df_re[df_re["GENE_SYMBOL"] == f"{gene_name}"]

    meta = ('Gene name', 'str') 
    df_re['CHROMOSOME'] = df_re['CHROMOSOME'].astype('object')
    df_re['WT_AA'] = df_re['MUTATION_AA'].str[2].apply(lambda x: canonical_aas[x], meta=meta)
    df_re['MUTATION_POSITION'] = df_re['MUTATION_AA'].str[3:-1]
    df_re['MUTATED_AA'] = df_re['MUTATION_AA'].str[-1].apply(lambda x: canonical_aas[x], meta=meta)

    df_re = df_re.drop("MUTATION_AA", axis=1)
    
    cosmic_df = df_re.compute()

    cosmic_df["GENOME_START"] = cosmic_df["GENOME_START"].astype(int)
    cosmic_df["GENOME_STOP"] = cosmic_df["GENOME_STOP"].astype(int)
    
    return cosmic_df

In [130]:
def map_gnomad(Gene_name:str, outpath:str):

    outpath = f"{outpath}/gnomad_datatable.txt"
    #outpath="/home/micnag/bioinformatics/hail_trials/newline_test.tsv"
    
    path = '/home/micnag/bioinformatics/gnomad_raw_data/gnomad_data.mt'
    #path1 = "/home/micnag/bioinformatics/gnomad_raw_data/gnomad_data_2.mt"
    mt = hl.read_matrix_table(path)
    
    substring1 = Gene_name
    substring2 = "missense"
    
    
    mt = mt.annotate_rows(Gene_names=mt.info.vep.map(
        lambda x: x.split("\|")[3]) ,
                      type_of_change = mt.info.vep.map(
        lambda x: x.split("\|")[1]) , 
                      AA_change = mt.info.vep.map(
        lambda x: x.split("\|")[11]) , 
                      ENST_identifier= mt.info.vep.map(
        lambda x: x.split("\|")[6])

    ) 
             
    filtered_mt_2 = mt.filter_rows(
    
    #hl.any(lambda x: hl.str(x).contains(substring3), mt.AA_change)
    hl.any(lambda x: hl.str(x).contains(substring1), mt.info.vep) &
    hl.any(lambda x: hl.str(x).contains(substring2), mt.info.vep)
    
    )
                     
    filtered_mt_3 = filtered_mt_2.annotate_rows(
        Allele_count_int = filtered_mt_2.info.AC,
        Allele_frequency_float = filtered_mt_2.info.AF,
        Allele_number_int = filtered_mt_2.info.AN,
        Gene_name_str = _replace_empty(filtered_mt_2.Gene_names), 
        Mutation_change_str = _replace_empty(filtered_mt_2.AA_change),
        Type_of_change_str = _replace_empty(filtered_mt_2.type_of_change))
    
    
    
    rows_to_keep = ["Gene_name_str", "Mutation_change_str", "Type_of_change_str", "Allele_count_int",
                "Allele_frequency_float", "Allele_number_int"]


    selected_rows = filtered_mt_3.select_rows(
        Allele_count_int=filtered_mt_3.Allele_count_int,
        Allele_frequency_float=filtered_mt_3.Allele_frequency_float,
        Allele_number_int=filtered_mt_3.Allele_number_int,
        Gene_name_str=hl.str(filtered_mt_3.Gene_name_str),
        Mutation_change_str=hl.str(filtered_mt_3.Mutation_change_str),
        Type_of_change_str=hl.str(filtered_mt_3.Type_of_change_str)
            )

    save_buffer = selected_rows.select_rows(*rows_to_keep)
    
    select_rows_out = save_buffer.rows()
    
    select_rows_out.export(outpath)
    
    #this is the location where we save the results.
    return outpath

In [131]:
def gnomad_to_pandas(Gene_name:str,path_to_tsv:str, fasta_seq:str):
    
    df = pd.read_csv(path_to_tsv, sep="\t")
    pd.set_option('display.float_format', '{:.2e}'.format)
    
    #modifying and streamlining input for further downstream compatibility.
    df['Allele_frequency_float'] = df['Allele_frequency_float'].str.strip('[]').astype(float)
    df['Allele_count_int'] = df['Allele_count_int'].str.strip('[]').astype(int)
    
    df["Type_of_change_str"] = df["Type_of_change_str"].astype("string")
    df["Type_of_change_str"] = df["Type_of_change_str"].str.replace("[", "")
    df["Type_of_change_str"] = df["Type_of_change_str"].str.replace("]", "")
    df["Type_of_change_str"] = df["Type_of_change_str"].str.replace("\"", "")
    
    df["Mutation_change_str"] = df["Mutation_change_str"].astype("string")
    df["Mutation_change_str"] = df["Mutation_change_str"].str.replace("[", "")
    df["Mutation_change_str"] = df["Mutation_change_str"].str.replace("]", "")
    df["Mutation_change_str"] = df["Mutation_change_str"].str.replace("\"","")
    
    df["Gene_name_str"] = df["Gene_name_str"].astype("string")
    df["Gene_name_str"] = df["Gene_name_str"].str.replace("[", "")
    df["Gene_name_str"] = df["Gene_name_str"].str.replace("]", "")
    df["Gene_name_str"] = df["Gene_name_str"].str.replace("\"","")

    #required to parse the info.
    lst =  [('Val',"V"), ('Ile',"I"), ('Leu',"L"), ('Glu',"E"), ('Gln',"Q"),
                    ('Asp',"D"), ('Asn',"N"), ('His',"H"), ('Trp',"W"), ('Phe',"F"), ('Tyr',"Y"), 
                    ('Arg',"R"), ('Lys',"K"), ('Ser',"S"), ('Thr',"T"), ('Met',"M"), ('Ala',"A"), 
                    ('Gly',"G"), ('Pro',"P"), ('Cys',"C")]
    
    canonical_aas = defaultdict(lambda: "X", lst)

    Mut_dict = defaultdict()

    for x, y, z, allele_cnt, allele_freq, allele_num \
        in zip(df["Gene_name_str"], df["Mutation_change_str"], df["Type_of_change_str"],
          df["Allele_count_int"], df["Allele_frequency_float"], df["Allele_number_int"]):
        
        tmp = x.split(",")
        tmp2 = y.split(",")
        tmp3 = z.split(",")
        #print(tmp, tmp2, tmp3)
        #print(allele_cnt, allele_freq, allele_num)
        for x1, y1, z1 in zip(tmp, tmp2, tmp3):
            
            #print(x1, y1, z1)
            if x1 == Gene_name and y1[0:4] == "ENSP":
                gene_code = x1
                mutation_code = y1[20:]
                type_change = z1
                
                result = _check_proper_transcript(fasta_seq=fasta_seq,
                                                  mutation_code=mutation_code, 
                                                  dict_to_check=canonical_aas, allele_count=allele_cnt)
                
                #print(gene_code, mutation_code, type_change)
                if result:
                    Mut_dict[mutation_code] = (allele_cnt, allele_freq, allele_num)
        
    
    #we return the dictionary with out mutations and their respective allele_cnts, allele_freqs, allele_nums
    return df, Mut_dict

In [132]:
def map_clinvar(Gene_name:str):
    
    path = "/home/micnag/bioinformatics/clinvar_trials/variant_summary.txt"

    use_cols = ["Type", "Name", "GeneSymbol",
           "ClinicalSignificance", "PhenotypeList",
           "Assembly", "ChromosomeAccession", 
           "Chromosome", "Start", "Stop"]

    column_data_types = {
    "Type": str,
    "Name": str,
    "GeneSymbol": str,
    "ClinicalSignificance": str,
    "PhenotypeList": str,
    "Assembly": str,
    "ChromosomeAccession": str,
    "Chromosome": str,
    "Start": int,
    "Stop": int
    }

    df_work = pd.read_csv(path, sep="\t", usecols=use_cols, dtype=column_data_types)
    
    df_work.loc[:, "AA_change"] = df_work["Name"].str.split().str.get(-1)
    df_work.loc[:, "AA_change"] = df_work["AA_change"].str.replace("(", "")
    df_work.loc[:, "AA_change"] = df_work["AA_change"].str.replace(")", "")
    
    df_work.loc[:,"Original_AA"] = df_work["AA_change"].str[2:5]
    df_work.loc[:,"Modified_AA"] = df_work["AA_change"].str[-3:]
    df_work['Position'] = pd.to_numeric(df_work['AA_change'].str[5:-3], errors='coerce')
    
    # Drop rows with NaN values in the 'Position' column
    df_work.dropna(subset=['Position'], inplace=True)
    df_work['Position'] = df_work['Position'].astype(int)
    
    df_work["Genomic_location"] = df_work["Chromosome"] + ":" + df_work["Start"].astype(str)
    df_work["gnomad_aa_change"] = "p." + df_work["Original_AA"] + df_work["Position"].astype(str) + df_work["Modified_AA"]
    
    df_work = df_work.drop("AA_change", axis=1)
    df_work = df_work.drop("Name", axis=1)
    df_work = df_work.drop("Chromosome", axis=1)
    df_work = df_work.drop("Start", axis=1)
    df_work = df_work.drop("Stop", axis=1)
    
    df_work["Allele_count"] = [np.nan] * len(df_work)
    df_work["Allele_number"] = [np.nan] * len(df_work)
    df_work["Allele_frequency"] = [np.nan] * len(df_work)
    
    
    accepted_residues = ["Ala", "Gly", "Ser", "Leu", "Pro",
                    "Ile", "Val", "Phe", "Tyr", "Trp",
                     "His", "Thr", "Asn", "Gln", "Asp", 
                     "Glu","Cys", "Met", "Lys", "Arg"]
    
    
    #filtering based on our Gene name.
    df_filtered = df_work[(df_work["Type"] == "single nucleotide variant") & 
        (df_work["GeneSymbol"] == Gene_name) &
        (df_work["Assembly"] == "GRCh37") & 
        (df_work['Original_AA'].isin(accepted_residues)) &
        (df_work['Modified_AA'].isin(accepted_residues)) ]
    
    return df_filtered

In [133]:
def map_clinvar_to_gnomad(Gene_name:str, clinvar_df:pd.DataFrame,
                         gnomad_mut_dict:dict,
                         clinvar_mapped_df_path:str):
    
    path = '/home/micnag/bioinformatics/gnomad_raw_data/gnomad_data.mt'
    #path1 = "/home/micnag/bioinformatics/gnomad_raw_data/gnomad_data_2.mt"
    mt = hl.read_matrix_table(path)
    
    hits = []

    to_be_searched = []
    found = defaultdict(tuple)

    for keys, vals in gnomad_mut_dict.items():
        #print(keys, vals)
        hits.append(keys)

    for x, y in zip(clinvar_df["gnomad_aa_change"], clinvar_df["Genomic_location"]):
        if x[2:] in hits:
            found[y] = (gnomad_mut_dict[x[2:]])
        else:
            to_be_searched.append((x, y))
            #print(x,"not found")

    #now we try to search for those hits that were not found in the gnomad dataset.
    hit_test = [x[1] for x in to_be_searched]


    if len(hit_test) == 0:
        return None
        
    #print(len(hit_test)) #126 but only 26 annotated in gnomad.
    loci_to_filter_hail = [hl.parse_locus(locus) for locus in hit_test]

    #we annotate these rows that bare information that is interesting for us.
    mt = mt.annotate_rows(Gene_names=mt.info.vep.map(
        
        lambda x: x.split("\|")[3]) ,
                          
                      type_of_change = mt.info.vep.map(
        lambda x: x.split("\|")[1]) , 
                          
                      AA_change = mt.info.vep.map(
        lambda x: x.split("\|")[11]) , 
                          
                      ENST_identifier= mt.info.vep.map(
        lambda x: x.split("\|")[6]),
        
                      Allele_count_int = mt.info.AC,
                          
                      Allele_frequency_float = mt.info.AF,
                          
                      Allele_number_int = mt.info.AN
    ) 

    
    filtered_mt_2 = mt.filter_rows(
    
        #hl.any(lambda x: hl.str(x).contains(substring3), mt.AA_change)
        #hl.any(lambda x: hl.str(x).contains(substring1), mt.info.vep) &
        #hl.any(lambda x: hl.str(x).contains(substring2), mt.info.vep) &
        hl.literal(loci_to_filter_hail).contains(mt.locus)
    
    ) 
    
    selected_row_fields = ["Allele_count_int", "Allele_number_int", "Allele_frequency_float"]

    #selected_rows = selected_rows.rows()

    selected_row_field_1 = filtered_mt_2.select_rows(*selected_row_fields)

    sel_output = selected_row_field_1.rows()

    sel_output.export(clinvar_mapped_df_path)
    
    #we return the outpath for further downstream work
    return to_be_searched, clinvar_mapped_df_path

In [134]:
def update_clinvar_muts_based_on_gnomad(clinvar_df:pd.DataFrame,
                                       gnomad_dict:dict):

    clinvar_rev_path = "/home/micnag/bioinformatics/hail_trials/reverse_mapping_test.tsv"
    
    df_reload = pd.read_csv(clinvar_rev_path,
                            sep="\t")

    #df_reload.shape

    #df_reload.columns
    df_reload['Allele_frequency_float'] = df_reload['Allele_frequency_float'].str.strip('[]').astype(float)
    df_reload['Allele_count_int'] = df_reload['Allele_count_int'].str.strip('[]').astype(int)

    for x, allele_c, allele_num, allele_freq in zip(df_reload["locus"],
                                                    df_reload["Allele_count_int"],
                                                    df_reload["Allele_number_int"],
                                                    df_reload['Allele_frequency_float']):
        #print(x, allele_c, allele_freq, allele_num)
        if x not in gnomad_dict.keys():
            gnomad_dict[x] = (allele_c, allele_freq, allele_num)
        else:
            continue
    
    
    #setup dummies for mapping.
    clinvar_df["Allele_count"] = [np.nan] * len(clinvar_df)
    clinvar_df["Allele_number"] = [np.nan] * len(clinvar_df)
    clinvar_df["Allele_frequency"] = [np.nan] * len(clinvar_df)
    
    for k, v in gnomad_dict.items():
    
        #k = chr:position e.g 16:17464758  and v = (Allele_cnt, Allele_freq, Allele_num)
        clinvar_df.loc[clinvar_df["Genomic_location"] == k, "Allele_count"] = int(v[0])
        clinvar_df.loc[clinvar_df["Genomic_location"] == k, "Allele_frequency"] = v[1]
        clinvar_df.loc[clinvar_df["Genomic_location"] == k, "Allele_number"] = int(v[2])
    
    
    return clinvar_df

In [135]:
def _check_proper_transcript(fasta_seq:str, mutation_code:str,
                             dict_to_check:dict, allele_count:int):
    
    #print(mutation_code)
    try:
        position = int(mutation_code[3:-3])
        
    except Exception as error:
        return False
    
    #quality filter to return ONLY variants that passed the filter criteria.
    if allele_count == 0:
        return False
    
    aa_according_to_gnomad = mutation_code[0:3]
    
    aa_to_check = dict_to_check[aa_according_to_gnomad]
    
    aa_according_to_fasta = fasta_seq[position-1] #-1 because of python starting with 0
    
    #print(aa_to_check, aa_according_to_fasta)
    if aa_to_check == aa_according_to_fasta:
        return True
    else:
        return False

In [136]:
def _replace_empty(arr):
    return hl.map(lambda x: hl.if_else(x == "", "empty", x), arr)

In [137]:
def _relevant_oligomers(path:str, templates_new_dict:dict):
    
    """This function basically will call a bunch of other functions 
    for each potential oligomeric state and each position within them."""
    
    oligodirectories = ["monomer",
                        "dimer",
                        "trimer",
                        "tetramer",
                        "pentamer",
                        "hexamer",
                        "heptamer",
                        "oktamer",
                        "nonamer",
                        "decamer",
                        "undecamer",
                        "dodecamer",
                        "tridecamer",
                        "tetradecamer",
                        "pentadecamer",
                        "hexadecamer",
                        "heptadecamer",
                        "oktadecamer",
                        "nonadecamer",
                        "eicosamer"
    ]
    
    relevant_dirs = []
    
    for file in os.listdir(path):
        if file in oligodirectories:
            d = os.path.join(path, file)
            relevant_dirs.append(d)
    
    dir_dictionary = defaultdict(list)
    
    for dirs in relevant_dirs:
        
        dir_dictionary[dirs] = os.listdir(dirs)
        
    
    #print("this is dir dictionary")
    #print(dir_dictionary)
    
    path_to_check = []
    
    for keys, vals in dir_dictionary.items():
        #key = basepath to monomer e.g vals = list of ranges
        subdir_list = []
        
        for subdir in vals:
            new_path = keys + "/" + subdir
            
            
            if os.listdir(new_path):
                #print(new_path)
                path_to_check.append(new_path)
            else:
                
                print(new_path, "is empty")
                continue
    
    
    path_majority_vote = []
    
    for check_paths in path_to_check:
        
        len_files = len([f for f in os.listdir(check_paths) if os.path.isfile(os.path.join(check_paths, f)) and f[-4:] == ".pdb"])
        
        path_majority_vote.append((check_paths, len_files))

    
    
    majority_vote = sorted(path_majority_vote, key= lambda x: x[1], reverse=True)
    
    majority_path = majority_vote[0][0]

    majority_key = majority_vote[0][0].split("/")
    majority_key = majority_key[-2] + "/" + majority_key[-1]

    template = templates_new_dict[majority_key][0]
    return majority_path, template

In [138]:
def _set_b_factors_0(path:str):
    
    parser = PDBParser(QUIET=True)
    
    prot_name = f"test"
    
    fullpath = f"{path}"
    
    structure = parser.get_structure(prot_name, fullpath)
    
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    atom.set_bfactor(0.0)
    
    
    io = PDBIO()
    io.set_structure(structure)
    
    savepath = fullpath
        
    #we save all structures to the monomeric category.
    io.save(savepath)




In [139]:
def run_pca_for_all_oligomers(path:str, 
                             template:str,
                             main_prot_seq:str,
                             prot_name:str):
    
        
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f[-4:] == ".pdb"]
    
    #if there are more than 5 structures we do PCA
    if len(onlyfiles) >= 5:
        #structure_based_cutting(path_to_pdbs:str, template:str, main_prot_seq:str):
        
        template_new = structure_based_cutting(path_to_pdbs=path, 
                               template=template,
                               main_prot_seq=main_prot_seq)
        
        print(template_new)
        #now if it worked we can run PCA
        work_dir, protein, num_struc = pca_laura_pipeline(path=f"{path}/PCA",
                           template=template_new,
                           protein=prot_name)
        
        try:
            _plot_PCA_new(input_dir=work_dir, protein=protein, num_struc=num_struc)
        except Exception as error:
            
            print("we could not run _plot_PCA_new")
            print(error)
            
        

In [140]:
#run_pca_for_all_oligomers("/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/ADH1G_HUMAN/dimer",
#                         template_dict=None,
#                         main_prot_seq=None,
#                         prot_name=None)

1. map_gnomad()  #we map all gnomad mutations to a hailtable and select only those relevant gene muts.
2. gnomad_to_pandas()  #we convert the found mutations of gnomad to a pandas df.
3. map_clinvar()     #we search for all clinvar mutations based on the same gene_name. 
4. map_clinvar_to_gnomad()  # we search for those mutations that dont already have associated frequencies if they might be present in gnomad
5. update_clinvar_muts_based_on_gnomad() #now we update our previous clinvar df with those new found frequencies (if found)


# testblock to retrieve mutations.

In [141]:
#step 1 gather mutations from cosmic:

#main_prot_name = "FLNC"
#main_prot_seq = get_gene_fasta(main_prot_name)
#
#path = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/AT2A1_HUMAN/monomer/pos_1_994"
#oligo_state_to_check = path
#
#clinvar_mapped_df_path = f"{path}/clinvar_map.txt"
#
#cosmic_df = get_cosmic_mutations(gene_name=main_prot_name) 
#
#cosmic_df.head()
#
#try:
#    cosmic_df.to_csv(f"{path}/cosmic_mutations.csv")
#except Exception as error:
#    print(error)
##    
##step 2 map gnomad mutations.
#
#gnomad_table_path = map_gnomad(Gene_name=main_prot_name, outpath=oligo_state_to_check)
##
###step 3 extract mutations from gnomad
#gnomad_df, gnomad_mutation_dict = gnomad_to_pandas(Gene_name=main_prot_name,
#             path_to_tsv=gnomad_table_path, 
#             fasta_seq=main_prot_seq)
##
##
###print(gnomad_mutation_dict)
#gnomad_df.head()
#try:
#    gnomad_df.to_csv(f"{path}/gnomad_mutations.csv")
#except Exception as error:
#    print(error)
##
##step 4 gather mutations from clinvar
#clinvar_df = map_clinvar(Gene_name=main_prot_name)
### Search for mutations in gnomad if they are present so we obtain info about allele freq ect.
#list_to_be_searched, clinvar_mapped_df_path = map_clinvar_to_gnomad(Gene_name=main_prot_name, clinvar_df=clinvar_df,
#                     gnomad_mut_dict=gnomad_mutation_dict,clinvar_mapped_df_path=clinvar_mapped_df_path)
#
#
#clinvar_df.head()
#
### Update clinvar muts that were found 1 step before.
#updated_clinvar_df = update_clinvar_muts_based_on_gnomad(clinvar_df=clinvar_df, gnomad_dict=gnomad_mutation_dict)
##try:
##    updated_clinvar_df.to_csv(f"{path}/clinvar_mutations.csv")
##except Exception as error:
##    print(error)
#
#updated_clinvar_df.head()
#
##print(f"This is cosmic df: {cosmic_df.shape}")
##
##print(f"This is updated_clinvar df: {updated_clinvar_df.shape}")
##
##print(f"This is gnomad df: {gnomad_df.shape}")
#

In [142]:
#_for_all_oligomers(path="/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/NUD4B_HUMAN", template_dict=template_dict)

In [143]:
#repair flnc strucs

def FLNC_repair(path_to_pdb:str, stop_pos:int, main_prot_seq:str):
    """Function should take only those proteins that have
    a) gaps with less than 7 residues missing per gap.
    
    Input:
    path to pdb_folder
    gap_dict file
    
    Output:
    produces repaired structures
    """

    #onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    #pdbs = [corr_file for corr_file in onlyfiles if corr_file[-4:] == ".pdb"]
    
    log.none() #no stdout spam
    env = Environ() #setup env for modelling
    
    aln = Alignment(env) # setup alignment within the env.
    
    mdl = Model(env)  #setup the model object where we will append our pdbs into.
     
    current_pth = os.getcwd()
    
    #print("we are inside mini repair")
    #print(pdb_basep)

    #['', 'home', 'micnag', 'result_test_struc_align', 'original_7e7s_A.pdb']
    #print(pdb_basep)
    #/home/micnag/result_test_struc_align
    #print(pdb_id_target)
    #7e7s_A
    #print(pdb_id_chain)
    #A
    
    # we need this for downstream split.
    pdb_id_target = "FLNc_ABD"
    
    pdb_id_chain = "A"
    
    
    #this is the new version which grabs correct seq per chain.
    
    #if original ensemble: we fetch the right uniprot sequence else we fetch whatever other homolog protein it is.

    #either we grab this!
    fasta_seq = main_prot_seq
        
    print(fasta_seq)
    print(len(fasta_seq))
    
    os.chdir("/home/micnag/bioinformatics/FLNC_project/new_structures_modeller_repaired")
    print(os.getcwd())
    
    
    #start stop grab:
    
    pdb_parser = Bio.PDB.PDBParser(QUIET = True)
        
    #sample struc
    sample_structure = pdb_parser.get_structure("sample", path_to_pdb)
    
    sample_res = sample_structure.get_residues()
    
    sample_list = [x.get_id()[1] for x in sample_res]
    
    start = sample_list[0] #first residue.
    stop = len(fasta_seq) #max amount of fasta seq.
    
    print(start, stop)
    
    env.io.atom_files_directory = ['.','../.']


    #write fasta
    #grab start + end pos.
    
    #start, stop, chain = _start_stop_fasta(pdb_id_target=pdb_id_target, 
    #                                                   path=f"{pdb_basep}/{pdb_id_target}")
    
    code = f"{pdb_id_target}"
    
    #print("this will be start and stop")
    #print(start, stop)
    
    
    mdl.read(file=code, model_segment=(f"{start}:{pdb_id_chain}", f"{stop}:{pdb_id_chain}"))
    
    aln.append_model(mdl, align_codes=code, atom_files=code)
    
    #print("this is pdb_id_target and chain")
    #print(f"{pdb_id_target[0:4]}", chain)
    
    #this is the old version which grabs full fasta.
    #fasta_seq = get_gene_fasta_from_pdb_id(f"{pdb_id_target[0:4]}", chain=chain)
    
    #print("this is fasta seq")
    #print(fasta_seq)
    
    with open(f"./{pdb_id_target}x.fasta", "w") as fastaout:
        fastaout.write(f">{pdb_id_target}x\n")
        fastaout.write(fasta_seq)
    
    
    aln_code = f"{pdb_id_target}x"


    aln.append(file=f"./{pdb_id_target}x.fasta", align_codes=aln_code, 
               alignment_format="fasta")
    
    aln.salign(overhang=30, gap_penalties_1d=(-450, -50),
    alignment_type="tree", output="ALIGNMENT")
    
    ##aln.malign(gap_penalties_1d=(-500, -300))
    ##aln.malign3d(gap_penalties_3d=(0.0, 2.0))
    aln.write(file=f"{pdb_id_target}.ali")

    #lets model
    
    #we will fish one out of the pdb list and align it against all others.
    
    a = AutoModel(env,
                  alnfile = f"{pdb_id_target}.ali",
                  knowns = f"{pdb_id_target}",
                  sequence = aln_code)
    
    a.starting_model = 1
    a.ending_model = 1
    a.make()
    

In [144]:
#main_prot_seq = "MMNNSGYSDAGLGLGDETDEMPSTEKDLAEDAPWKKIQQNTFTRWCNEHLKCVGKRLTDLQRDLSDGLRLIALLEVLSQKRMYRKFHPRPNFRQMKLENVSVALEFLEREHIKLVSIDSKAIVDGNLKLILGLIWTLILHYSISMPMWEDEDDEDARKQTPKQRLLGWIQNKVPQLPITNFNRDWQDGKALGALVDNCAPGLCPDWEAWDPNQPVENAREAMQQADDWLGVPQVIAPEEIVDPNVDEHSVMTYLSQFPKAKL"

#path = "/home/micnag/bioinformatics/FLNC_project/new_structures_modeller_repaired/FLNc_ABD.pdb"

#FLNC_repair(path_to_pdb=path, stop_pos=262, main_prot_seq=main_prot_seq)

# integrate cbioportal for patient information

In [145]:
#gene_name = get_hugo_name("P00533")
#print(gene_name)
#result_df = get_cbioportal_info(gene_name=gene_name)
#result_df.head(10)

#print(result_df.shape)
#['alleleSpecificCopyNumber', 'aminoAcidChange', 'center', 'chr', 'driverFilter', 
# 'driverFilterAnnotation', 'driverTiersFilter', 'driverTiersFilterAnnotation', 
# 'endPosition', 'entrezGeneId', 'gene', 'keyword', 'molecularProfileId', 'mutationStatus', 
# 'mutationType', 'namespaceColumns', 'ncbiBuild', 'normalAltCount', 'normalRefCount', 
# 'patientId', 'proteinChange', 'proteinPosEnd', 'proteinPosStart', 'referenceAllele', 
# 'refseqMrnaId', 'sampleId', 'startPosition', 'studyId', 'tumorAltCount', 'tumorRefCount', 
# 'uniquePatientKey', 'uniqueSampleKey', 'validationStatus', 'variantAllele', 'variantType']

In [146]:
# Seq alignment through mmseq2 and retrieval of multi pdb fasta.

def mmseq_multi_fasta(uniprot_id:str, outdir:str, 
                      sensitivity=7, filter_msa=0,
                     query_id = 0.6):

    """
    uniprot_id: The unique uniprot identifier used to fetch the corresponding fasta file that will be used as a template for mmseq2

    outdir: location where result files will be stored.

    sensitivity: mmseq2 specific parameter that goes from 1-7. The higher the more sensitive the search.

    filter_msa = 0 default. if 1 hits are stricter.

    query_id = 0.6 [0, 1]  the higher the more identity with query is retrieved. 1 means ONLY the query hits while 0 means take everything possible.
    
    """


    #we blast with this fasta as query.
    trgt_fasta_seq = get_gene_fasta(uniprot_id)


    #Make outdir for all required files.
    try:
        os.mkdir(outdir)

    except Exception as error:
        print(error)

    #we need to write it out to file.
    with open(f"{outdir}/{uniprot_id}_fasta.fa", "w") as fasta_out:
        fasta_out.write(f">{uniprot_id}\n")
        fasta_out.write(trgt_fasta_seq)
    
    #fetch pre downloaded database from a parent folder.

    msa_file = None
    new_location = None
    
    try:
        DB_storage_location = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/swissprot_DB"
        #shutil.copy(previous_path, savepath)
        
        bash_curl_cmd = f"mmseqs createdb {outdir}/{uniprot_id}_fasta.fa {DB_storage_location}/query_fastaDB"
    
        bash_curl_cmd_rdy = bash_curl_cmd.split()

        print(bash_curl_cmd)
        print("we start now")
        
        #run first cmd which setups query database based on our input fasta file
        result_setup_query_db = run(bash_curl_cmd_rdy, stdout=PIPE, stderr=PIPE, 
                             universal_newlines=True)

        print("this worked")
        print(result_setup_query_db.stderr)

        bash_curl_cmd_2 = f"mmseqs search {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/tmp -s {sensitivity}"
    
        bash_curl_cmd_rdy_2 = bash_curl_cmd_2.split()
        
        #run 2nd cmd which blasts against swiss_DB and generates the resultDB (i.e our hits that were found)
        result_setup_blast_db = run(bash_curl_cmd_rdy_2, stdout=PIPE, stderr=PIPE, 
                             universal_newlines=True)

        print("this worked as well")
        print(result_setup_blast_db.stderr)
        #mmseqs result2flat resultDB resultDB_flat --threads 4
        #mmseqs result2fasta pdb70_mmseqs2.fasta resultDB_flat resultDB.fasta --threads 4

        # mmseqs convertalis queryDB targetDB resultDB result.fasta

        #mmseqs result2flat queryDB targetDB resultDB result_flat.txt

        #bash_curl_cmd_4 = f"mmseqs result2flat {DB_storage_location}/result_DB {DB_storage_location}/result_DB_flat --use-fasta-header TRUE"

        #bash_curl_cmd_rdy_4 = bash_curl_cmd_4.split()

        #result_setup_flat_convert = run(bash_curl_cmd_rdy_4, stdout=PIPE, stderr=PIPE, 
        #                     universal_newlines=True)
        
        #print("here works still")
        #bash_curl_cmd_3 = f"mmseqs convertalis {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/{uniprot_id}_result.m8"
        #mmseqs convertalis query_fastaDB PDB_DB result_DB "$input_fasta"_result.m8
        #bash_curl_cmd_rdy_3 = bash_curl_cmd_3.split()
        
        #result_setup_flat_convert = run(bash_curl_cmd_rdy_3, stdout=PIPE, stderr=PIPE, 
        #                     universal_newlines=True)

        #mmseqs convert2fasta DB_clu_rep DB_clu_rep.fasta
        bash_curl_cmd_5 = f"mmseqs result2msa {DB_storage_location}/query_fastaDB {DB_storage_location}/swiss_DB {DB_storage_location}/result_DB {DB_storage_location}/{uniprot_id}_out.fasta --msa-format-mode 3 --filter-msa {filter_msa} --qid {query_id}" 

        bash_curl_cmd_5_rdy = bash_curl_cmd_5.split()

        result_setup_msa_convert = run(bash_curl_cmd_5_rdy, stdout=PIPE, stderr=PIPE, 
                             universal_newlines=True)
        #print(result_setup_flat_convert.stderr)

        #delete last line.. required.
        sed_cmd = f'sed -e 1,4d -e $d {DB_storage_location}/{uniprot_id}_out.fasta'
        
        bash_curl_cmd_6_rdy = sed_cmd.split()

        #f"{DB_storage_location}/{uniprot_id}_new_out.fasta"
        with open(f"{DB_storage_location}/{uniprot_id}_new_out.fasta", "w") as new_fasta:
            result_truncation = run(bash_curl_cmd_6_rdy, stdout=new_fasta, stderr=PIPE, 
                             universal_newlines=True)

        # Specify the path to your MSA file
        msa_file = f"{DB_storage_location}/{uniprot_id}_new_out.fasta"


        #transfer the msa file to another location and delete useless files.
        # we need to delete : all uniprot* files. 
        # all query*. All result* 

        new_location = f"{outdir}/{uniprot_id}.fasta"

        
        shutil.copy(msa_file, new_location)

        remove_files_and_dirs_msa(DB_storage_location, uniprot_id=uniprot_id)
        
    except Exception as error:
        print(error)


    #we want the path to msa_file for downstream analysis.
    return new_location

In [147]:
def remove_files_and_dirs_msa(directory, uniprot_id):

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        # Check if the item is a file and matches the specified patterns
        if os.path.isfile(file_path) and (filename.startswith("query") or filename.startswith("result") or filename.startswith(uniprot_id)):
            os.remove(file_path)
            print(f"Removed file: {file_path}")
        
        # Check if the item is a directory and has the name "tmp"
        elif os.path.isdir(file_path) and filename == "tmp":
            shutil.rmtree(file_path)
            print(f"Removed directory and its contents: {file_path}")


In [148]:
def get_conservation(path_to_msa:str):

    '''
    path_to_msa:str    path to the multiple sequence alignment file. The first entry is expected to be used as reference.
    canal: object that contains the msa seq. Will be used with canal.analysis to compute conservation scores.
    
    '''
    
    canal = Canal(fastafile=path_to_msa, #Multiple sequence alignment (MSA) of homologous sequences
          ref=0, #Position of reference sequence in MSA, use first sequence always
          startcount=0, # ALways 0 because our seqs are always from 1 - end
          verbose=False) # no verbosity 
    
    result_cons = canal.analysis(method="all")
    #this is a pandas df that contains 3 cols : 3 diff conservation score computations.
    return result_cons

# steps for conservation

1) mmseq_multi_fasta
2) remove_files_and_dirs_msa
3) get_conservation

In [149]:
#outdir = "/home/micnag/bioinformatics/mmseq_dir_pipe_test"
#uniprot_id = "Q14315"

#out_path_msa = mmseq_multi_fasta(uniprot_id =uniprot_id, 
#                outdir=outdir)

#get_conservation(path_to_msa=out_path_msa)

In [250]:
def get_cluster_df(PCA_12_proj_file:str, prot_names:str):
    
    '''

    Input:

    PCA_12_proj_file: Path to the file that contains the projected PC1 and PC2 values as column entries.

    prot_names: The path to the file that contains the names of the associated proteins (4 digit pdb code) that are used in the PCA.

    Output:

    pandas.DataFrame ready for downstream applications. 

    Cols: prot_name , PC1, PC2
    
    '''

    PC_lst = []
    prot_lst = []
    #lets store the PC 1 2 coords as a list consisting of tuples [(PC1, PC2), (PC1, PC2),.....n] for all n structures.
    with open(PCA_12_proj_file, "r") as pca_input:
        for lines in pca_input:
            lines = lines.split()
            PC_lst.append((lines[2], lines[3]))

    #here we have the associated names + chains so we also add them to a list for later retrieval.
    with open(prot_names, "r") as prot_input:
        for pdb_chain in prot_input:
            pdb_chain = pdb_chain.replace("\n","")
            prot_lst.append(pdb_chain)

    
    pca_df = pd.DataFrame()
    pca_df["prot_name"] = prot_lst
    pca_df["PC1"] = [x[0] for x in PC_lst]
    pca_df["PC2"] = [x[1] for x in PC_lst]

    
    #pca_df.columns
    #print(pca_df.head())
    return pca_df

In [ ]:
def cluster_pca_hdbscan(pca_df:pd.DataFrame):

    # Assuming pca_df has columns 'PC1' and 'PC2'
    X = pca_df[['PC1', 'PC2']].values
    y = pca_df.iloc[:,0]

    # Create an HDBSCAN model
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=5)
    
    # Fit the model and get cluster labels
    hdbscan_labels = hdbscan_model.fit_predict(X)
    
    # Check the cluster labels using a loop
    for label, pdb in zip(hdbscan_labels, y):
        print("Data point belongs to cluster:", label, "and is pdb:",pdb)
    
    # Count the number of clusters (excluding noise, labeled as -1)
    num_clusters = len(set(hdbscan_labels)) - (1 if -1 in hdbscan_labels else 0)
    print("Number of Clusters:", num_clusters)
    
    
    
    

In [251]:
path_pca = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/ALOX5/monomer/pos_1_674/PCA/ALOX5.mode_12.proj"
path_ensemble = "/home/micnag/bioinformatics/rcsb_retrieved_pdbs/rcsb_fetched_structures/ALOX5/monomer/pos_1_674/PCA/ensemble.txt"

pca_df = get_cluster_df(PCA_12_proj_file=path_pca,
                           prot_names=path_ensemble)
#print(pca_df)

cluster_pca_hdbscan(pca_df)

Data point belongs to cluster: -1
Data point belongs to cluster: -1
Data point belongs to cluster: -1
Data point belongs to cluster: -1
Data point belongs to cluster: -1
Data point belongs to cluster: -1
Number of Clusters: 0


# RUN MAIN

In [150]:
#First: download all pdb.. /home/cond... do the tm score and rmsc comp...then select based on that...
#then we will repair gaps for those retrieved structures.

path = "/home/micnag/bioinformatics/uniprot/mmseq_protein_hits_raw.json" #testcase

hits = open(path, "r")
testcase = json.load(hits)
key_lst = []
val_lst = []
i = 0
for keys, vals in testcase.items():
    i +=1
    if keys == "P00533":
        print(i)
        pass


2025


In [252]:
## First: download all pdb.. /home/cond... do the tm score and rmsc comp...then select based on that...
#then we will repair gaps for those retrieved structures.

path = "/home/micnag/bioinformatics/uniprot/mmseq_protein_hits_raw.json" #testcase

hits = open(path, "r")
testcase = json.load(hits)
key_lst = []
val_lst = []
i = 0
for keys, vals in testcase.items():
    i +=1
    if keys == "P04049":
        print(i)
        pass
    key_lst.append(keys)
    val_lst.append(vals)


#we parse from here through all.
hit_list = list(zip(key_lst,val_lst))

#2024:2025 egfr

#try 6:7 good case.
#so is 202:203
#24-25 is faulty. so is 34-35 and 37:38

#555-556 seems good case to illustrate.
#5803 = ACSM2A gene


#O14983 SERCA 744:745
#O75311 GLRA 1401:1402
#P48167 GLRB 4400:4401
#P00326 ADHG 2004:2005
#P06756 Integrin checknumbers 2356:2357  THIS case is totally catastrophic. #final stuff to fix for mixed heteromers.
i = 0
#Q8N142 is Adenylosuccinate synthetase isozyme 1 9096:9097
#2569:2570 = LOX5
#calmodulin 1 2689:2690  !rerun again when NMR is properly working.

#P02788 Lactotransferin 2175:2176
#P02911 LAO-binding protein try as well. 
#Q8TEX9 Importin-4 10086:10087
#Q14974 Real importin subunit beta with strucs: 6418:6419

#Q16774 guanylate kinase 6766:6767   ! rerun again when NMR is properly working! seems interesting.
#raf1 + complexes
print(len(hit_list))
#744:745 serca
#2203:2204 kraf
for keys, entries in hit_list[744:745]:
    #print(keys, entries)
    query, templates, seq_sim, query_start, query_end, temp_start, temp_end = keys, entries[0],\
    entries[1],entries[2],entries[3],entries[4],entries[5] 
    
    print(query)
    #print(sorted(templates))
    os.chdir("/home/micnag/bioinformatics") #just to start properly.
    
    #2203:2204 == RAF1
    fetch_struc(query, templates, seq_sim, query_start, query_end, temp_start, temp_end, run_NMA=True, run_PCA=True)


2204
14913
O14983
300


These are the ranges that we deal with: [(1, 994), (1, 993), (2, 993), (23, 849), (5, 989), (5, 825), (68, 862), (19, 830), (15, 766), (23, 769), (148, 770), (66, 771), (290, 336), (87, 783), (78, 769), (36, 77), (243, 278), (749, 780), (86, 115)] 

length of dictionary before recursive_reduction: 19
length of dictionary after recursive_reduction: 6

These are the merged ranges: [Interval(1, 994), Interval(290, 336), Interval(36, 77), Interval(243, 278), Interval(749, 780), Interval(86, 115)]
ATP2A1
we are in get gene fasta
we start download
we start shifts
1wpg_A 0
1wpg_B 0
1wpg_C 0
1wpg_D 0
7d91_A 0
7d91_B 0
7d91_G 0
4res_A 0
4res_C 0
4res_B 0
4res_D 0
4res_E 0
4res_G 0
3w5b_A 0
8d3u_A 0
8d3u_B 0
8d3u_G 0
3nal_A 0
7nxf_A 0
7zre_A 0
7zre_B 0
7zre_C 0
7zre_D 0
5xa8_A 0
5xa9_A 0
3n23_A 5
3n23_C 5
3n23_B 0
3n23_D 0
3n23_E 0
3n23_G 0
7x24_A 0
7x24_B 0
1iwo_A 0
1iwo_B 0
3j08_A 0
3j08_B 0
7d93_A 0
7d93_C 0
7d93_B 0
7d93_D 0
7d93_E 0
7d93_G 0
2by4_A 0
4ux2_A 0
4ux2_B 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/micnag/anaconda3/envs/cosmic_env/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/micnag/anaconda3/envs/cosmic_env/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/micnag/anaconda3/envs/cosmic_env/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o1.pyLoadReferencesFromDataset

# TO DO LOG


### repair and check for repairability.

+ repair modeller pipeline should work : done
+ rechaining / renumbering after modelling: done
+ funnelling repaired chains into full biological ensembles: done


### NOW WE NEED TO RENUMBER AFTER REPAIR!

REPAIR IS DONE AND WORKS.


## NEXT STEP:

remove duplicate TEMPLATE from MSA USALIGN.
example whats wrong:

>2q9p.pdb:A	L=132	d0=4.26	seqID=0.874	TM-score=0.93282
---RTYDGDGYKKRAACLCFRSESEEEVLLVSSSRHPDRWIVPGGGMEPEEEPSVAAVREVCEEAGVKGTLGRLVGIF-ENQERKHRTYVYVLIVTEVLEDWEDSV-NIGRKREWFKIEDAIKVLQYHKPVQASYFE----------
>2q9p.pdb:A	L=132	d0=4.26	seqID=0.874	TM-score=0.93282	*
---RTYDGDGYKKRAACLCFRSESEEEVLLVSSSRHPDRWIVPGGGMEPEEEPSVAAVREVCEEAGVKGTLGRLVGIF-ENQERKHRTYVYVLIVTEVLEDWEDSV-NIGRKREWFKIEDAIKVLQYHKPVQASYFE----------
>3mcf.pdb:A	L=136	d0=4.33	seqID=0.801	TM-score=0.85092
FK----------KRAACLCFRSEREDEVLLVSSSRYPDRWIVPGGGMEPEEEPGGAAVREVYEEAGVKGKLGRLLGVFEQNQDPKHRTYVYVLTVTELLEDWEDSV-SIGRKREWFKVEDAIKVLQCHKPVHAEYLEKLKLGGSPTN
...

2q9p is twice there


+ **ABOVE BUG IS ALREADY FIXED** - 12/06/2023


### Implement according structural cutting based on 1:1 correspondence residues (i.e remove all that dont have 1:1 correspondence)

+ do the structural alignment and cut all residues that dont have 1:1 correspondence: **done**
+ prepare structures to only have those residues i.e all structures will end up with the same amount of residues : **done**
+ run domenicos PCA pipeline and integrate it into workflow: **done**
+ plot pca results (PCA1 PCA2): **done**


## Assemble FULL pipeline and run preliminary test runs.


pipeline assembled:

bugfixes:

+ fixed fetch fasta during repair:

previous fetch fasta would always grab full length protein fasta which would lead to full length protein repair reconstruction. This is not desired. We want only to restore the structure present (irrespective if the structure is only a fragment of the full length protein)

TBD:

+ multichain alignment in msa
Currently we only align chain A. Multichain alignment are not possible but we can run for all chains against all respective chains e.g chain A vs A / B vs B etc? Done



establish 1 to 1 full length alignments with USAlign and then cut according to the results of n - 1 seq alignments against the reference structure.

+ for multichain alignment, output file looks different. 
come up with a correct way to parse output file and make sure that chains are properly aligned!


+ set up proper PCA plots for each oligomer against its respective oligomers.
#


+ for NMR modells, consider that high seq similarity is equivalently good and means same protein but Calmodulin 1 e.g 6y95 and 6y94 are seen as different with low tm score ! mistake !they are same sequence and protein.


+ NOW NMR modells are taken as separate PDBS into account and will also be part of the ensemble!
fixed on 19.7.2023


# <b> TODO: </b>
NEED FOR OLIGOMERS TO DO SEQUENCE BASED GROUPING FOR EACH CHAIN AGAINST EACH CHAIN... A VS A... B VS B

check EGFR case for repair mutated residues. there is an issue with a CY0 modified residue that will result in a weird alanine.
_mutate function needs to be checked.

2j5e.pdb shows what is wrong...


+ check the alignment if it cuts properly the residues that it SHOULD CUT. There might be an issue with the residue selection that are KEPT FOR PCA.


+ pipeline mistakes small ligand bounds in other chains as oligomers... should be monomeric still.. 
THIS ERROR IS FIXED ALREADY.




NEXT THURSDAY:

Continue with HAIL and incorporation of GNOMAD DATABASE INFO.
Mutational mapping new is construction site now.
https://hail.is/tutorial.html
Check NUMBERING on the pdb structures. SEEMS TO BE OFF (maybe for repaired structures)????

DECOUPLE NMA RUNS FROM PCA... PCA IS A WHOLE OWN STORY...
WE RUN NMA ANALYSIS ON THE FRESH POPULATION OF STRUCTURES, irrespective of gaps. TAKE STRUCS FROM THE POS X-Y DIRECTORY. DONT REPAIR (AT FIRST)